In [ ]:
pip install osmnx

# Датасет

## Загрузка и визуализация датасета

In [ ]:
import osmnx as ox
import matplotlib.pyplot as plt

# Настройки osmnx
ox.settings.log_console = True
ox.settings.use_cache = True
ox.settings.timeout = 300

def download_moscow_region_optimized():
    """Скачивание Москвы и Московской области"""

    # Получаем границы регионов
    print("Получаем границы Москвы и Московской области...")
    places = [
        "Москва, Россия",
        "Московская область, Россия"
    ]

    # Получаем GeoDataFrame с границами
    regions_gdf = ox.geocode_to_gdf(places)

    # Визуализируем границы
    regions_proj = ox.projection.project_gdf(regions_gdf)
    fig1, ax1 = plt.subplots(figsize=(10, 10))
    regions_proj.plot(ax=ax1, fc="gray", ec="blue", alpha=0.5)
    ax1.set_title("Границы Москвы и Московской области")
    ax1.axis("off")
    plt.show()

    # Получаем объединенный полигон
    combined_polygon = regions_gdf.unary_union

    # Скачиваем дорожную сеть для объединенного региона
    print("Скачиваем дорожную сеть...")
    graph = ox.graph_from_polygon(
        combined_polygon,
        network_type='drive',  # только автомобильные дороги
        simplify=True
    )

    return graph

# Альтернативный способ - через graph_from_places (проще)
def download_moscow_region_simple():
    """Простой способ скачивания через graph_from_places"""

    places = [
        "Москва, Россия",
        "Московская область, Россия"
    ]

    graph = ox.graph_from_places(
        places,
        network_type='drive',  # публичные дороги для автомобилей
        simplify=True
    )

    return graph

# Скачиваем данные
print("Скачиваем...")
combined_graph = download_moscow_region_optimized()
print("Успешно!")

# Основная информация о графе
print(f"Количество узлов: {combined_graph.number_of_nodes()}")
print(f"Количество рёбер: {combined_graph.number_of_edges()}")

# Визуализируем дорожную сеть
print("Визуализация дорожной сети...")
fig2, ax2 = ox.plot_graph(
    combined_graph,
    node_size=0,           # скрываем узлы
    edge_linewidth=0.5,
    edge_color='#0066cc',
    bgcolor='white',
    show=False,
    close=False
)
ax2.set_title("Дорожная сеть Москвы и Московской области")
plt.show()

# Сохраняем граф
output_file = "moscow_region_drive_network.graphml"
ox.save_graphml(combined_graph, output_file)
print(f"Граф сохранен как '{output_file}'")

## Анализ датасета

In [ ]:
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from collections import Counter

In [ ]:
# Загружаем граф
graph = ox.load_graphml("moscow_region_drive_network.graphml")

In [ ]:
# 1. Основные характеристики
print(f"Количество перекрестков (узлов): {graph.number_of_nodes():,}")
print(f"Количество дорог (рёбер): {graph.number_of_edges():,}")
print(f"Плотность сети: {nx.density(graph):.6f}")

In [ ]:
# 2. Анализ типов дорог
def analyze_highway_types(graph):
    highway_types = []
    road_lengths = {}
    oneway_count = 0

    for u, v, data in graph.edges(data=True):
        highway = data.get('highway', 'unknown')
        length = data.get('length', 0)

        # Считаем односторонние дороги
        if data.get('oneway', False) == True:
            oneway_count += 1

        # Обрабатываем highway типы
        if isinstance(highway, list):
            for hw in highway:
                highway_types.append(hw)
                road_lengths[hw] = road_lengths.get(hw, 0) + length
        else:
            highway_types.append(highway)
            road_lengths[highway] = road_lengths.get(highway, 0) + length

    return highway_types, road_lengths, oneway_count

highway_types, road_lengths, oneway_count = analyze_highway_types(graph)

# Статистика по типам дорог
type_counts = Counter(highway_types)
total_edges = len(highway_types)
total_length_km = sum(road_lengths.values()) / 1000

print(f"Всего дорог: {total_edges:,} сегментов")
print(f"Общая длина: {total_length_km:,.1f} км")
print(f"Односторонних дорог: {oneway_count} ({oneway_count/total_edges*100:.1f}%)")

print("\nТоп-10 типов дорог по количеству:")
for hw_type, count in type_counts.most_common(10):
    percentage = (count / total_edges) * 100
    print(f"• {hw_type:20}: {count:6} сегментов ({percentage:5.1f}%)")

print("\nТоп-10 типов дорог по длине:")
for hw_type in sorted(road_lengths.keys(), key=lambda x: road_lengths[x], reverse=True)[:10]:
    length_km = road_lengths[hw_type] / 1000
    percentage = (road_lengths[hw_type] / sum(road_lengths.values())) * 100
    print(f"• {hw_type:20}: {length_km:8.1f} км ({percentage:5.1f}%)")

In [ ]:
# 3. Дополнительные показатели
lengths = [data.get('length', 0) for u, v, data in graph.edges(data=True)]
if lengths:
    print(f"Средняя длина дороги: {np.mean(lengths):.1f} м")
    print(f"Медианная длина дороги: {np.median(lengths):.1f} м")
    print(f"Минимальная длина: {np.min(lengths):.1f} м")
    print(f"Максимальная длина: {np.max(lengths):.1f} м")
    print(f"Стандартное отклонение: {np.std(lengths):.1f} м")

In [ ]:
# 5. Степени узлов
degrees = [degree for node, degree in graph.degree()]
print(f"Средняя степень узла: {np.mean(degrees):.2f}")
print(f"Максимальная степень: {np.max(degrees)}")
print(f"Минимальная степень: {np.min(degrees)}")

# Анализ распределения степеней
degree_counts = Counter(degrees)
print("\nРаспределение степеней узлов:")
for degree, count in degree_counts.most_common(10):
    print(f"  Степень {degree:2}: {count:5} узлов ({count/graph.number_of_nodes()*100:5.1f}%)")

In [ ]:
# 6. Плотность

# Конвертируем в GeoDataFrames
nodes_gdf, edges_gdf = ox.graph_to_gdfs(graph)

# Рассчитываем bounding box
minx, miny, maxx, maxy = nodes_gdf.total_bounds
width_km = (maxx - minx) * 111  # приблизительно в км (1 градус ~ 111 км)
height_km = (maxy - miny) * 111
area_approx = width_km * height_km

print(f"Приблизительная площадь покрытия: {area_approx:,.0f} км²")
print(f"Плотность узлов: {graph.number_of_nodes()/area_approx:.1f} узлов/км²")
print(f"Плотность дорог: {total_length_km/area_approx:.1f} км дорог/км²")

In [ ]:
# 7. Визуализация основных типов дорог
def plot_highway_types_simplified(graph):
    """Упрощенная визуализация с группировкой типов дорог"""

    # Группируем типы дорог для лучшей читаемости
    def get_road_category(highway_type):
        if isinstance(highway_type, list):
            highway_type = highway_type[0]

        main_roads = ['motorway', 'trunk', 'primary', 'secondary']
        local_roads = ['tertiary', 'unclassified', 'residential']
        service_roads = ['service', 'living_street']

        if highway_type in main_roads:
            return 'main_roads'
        elif highway_type in local_roads:
            return 'local_roads'
        elif highway_type in service_roads:
            return 'service_roads'
        else:
            return 'other'

    # Цвета для категорий
    color_map = {
        'main_roads': 'red',
        'local_roads': 'blue',
        'service_roads': 'green',
        'other': 'gray'
    }

    # Создаем списки для визуализации
    edge_colors = []
    categories_count = Counter()

    for u, v, data in graph.edges(data=True):
        highway = data.get('highway', 'other')
        category = get_road_category(highway)
        categories_count[category] += 1
        edge_colors.append(color_map[category])

    # Визуализация
    fig, ax = plt.subplots(figsize=(15, 12))

    ox.plot_graph(
        graph,
        ax=ax,
        node_size=0,
        edge_color=edge_colors,
        edge_linewidth=0.7,
        edge_alpha=0.8,
        bgcolor='white',
        show=False,
        close=False
    )

    # Легенда
    from matplotlib.lines import Line2D
    legend_elements = []
    for category, color in color_map.items():
        if category in categories_count:
            count = categories_count[category]
            legend_elements.append(
                Line2D([0], [0], color=color, lw=3,
                      label=f"{category} ({count} roads)")
            )

    ax.legend(handles=legend_elements, loc='upper right', fontsize=12)
    ax.set_title("Дорожная сеть Московского региона\n(Группировка по типам дорог)", fontsize=16)

    plt.tight_layout()
    plt.show()

    return categories_count

# Запускаем визуализацию
categories_count = plot_highway_types_simplified(graph)

print("\nРаспределение по категориям дорог:")
for category, count in categories_count.most_common():
    percentage = count / total_edges * 100
    print(f"  {category:15}: {count:6} дорог ({percentage:5.1f}%)")

## Склады

In [ ]:
import requests
import pandas as pd
import time
import json
import os

def geocode_yandex(address, api_key):
    """Геокодирование через Яндекс API"""
    base_url = "https://geocode-maps.yandex.ru/1.x/"
    params = {
        "apikey": api_key,
        "geocode": address,
        "format": "json",
        "lang": "ru_RU"
    }

    try:
        print(f"Геокодируем: {address}")
        response = requests.get(base_url, params=params, timeout=10)
        data = response.json()

        # Парсим ответ
        found = data['response']['GeoObjectCollection']['metaDataProperty']['GeocoderResponseMetaData']['found']
        if int(found) > 0:
            # Берем первый результат
            pos = data['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos']
            # Яндекс возвращает в формате "долгота широта"
            lon, lat = map(float, pos.split())
            print(f"Найдено: {lat}, {lon}")
            return lat, lon
        else:
            print(f"Яндекс не нашел координаты для: {address}")
            return None, None
    except Exception as e:
        print(f"Ошибка при геокодировании {address} через Яндекс: {e}")
        return None, None

def save_to_json(df, filename):
    """Сохраняет DataFrame в JSON файл"""
    result = []
    for _, row in df.iterrows():
        result.append({
            'name': row['name'],
            'address': row['address'],
            'latitude': row['latitude'],
            'longitude': row['longitude']
        })

    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=2)
    print(f"Данные сохранены в файл: {filename}")

In [ ]:
# API ключ
YANDEX_API_KEY = "мой api"

In [ ]:
# Обаботка распределительных и фулфилмент-центров (РЦ/РФЦ)
# Список складов РЦ/РФЦ
warehouses_data = [
    {"name": "ПАВЛО_СЛОБОДСКОЕ_РЦ", "address": "143581, Московская обл., Истринский район, с. Павловская Слобода, территория квартала 0050343, зд. 2"},
    {"name": "ХОРУГВИНО_РФЦ", "address": "141533, Московская обл., Солнечногорский р-н, с.п. Пешковское, дер. Хоругвино, стр. 32/2"},
    {"name": "Хоругвино_НЕГАБАРИТ", "address": "141533, Московская обл., Солнечногорский р-н, с.п. Пешковское, дер. Хоругвино, стр. 32/2"},
    {"name": "Новая_Рига_РФЦ", "address": "Россия, Московская обл, г. Истра, с. Петровское, территория МПСК Ориентир-Запад, зд. 1A"},
    {"name": "Жуковский", "address": "140182, Московская область, г. Жуковский, район Замоскворечье, д. 457, стр. 5"},
    {"name": "Софьино", "address": "Российская Федерация, Московская область, Раменский городской округ, квартал 4/218, стр. 2/1"},
    {"name": "Ногинск", "address": "142440, РФ, МО, Богородский городской округ, р.п. Обухово, тер. Обухово-Парк, дом 2, строение 1"},
    {"name": "Гривно", "address": "142184, МО, г.о. Подольск, д. Гривно, тер. пром. парка Гривно, д. 1, к. 13"},
    {"name": "Пушкино-1", "address": "Россия, Московская область, Пушкинский городской округ, г. Пушкино, Ярославское шоссе, д. 216"},
    {"name": "Пушкино-2", "address": "Россия, Московская область, г.о. Пушкинский, г. Пушкино, Ярославское шоссе, д. 218"}
]

warehouses_df = pd.DataFrame(warehouses_data)

# Геокодируем РЦ/РФЦ
print("Используем Яндекс API для геокодирования РЦ/РФЦ")
coordinates = []
for i, address in enumerate(warehouses_df['address']):
    print(f"\n--- Обрабатываем {i+1}/{len(warehouses_df)} ---")
    lat, lon = geocode_yandex(address, YANDEX_API_KEY)
    coordinates.append((lat, lon))
    time.sleep(0.5)  # Пауза между запросами

# Добавляем координаты в DataFrame
warehouses_df['latitude'] = [coord[0] for coord in coordinates]
warehouses_df['longitude'] = [coord[1] for coord in coordinates]

# Проверяем результаты РЦ/РФЦ
print("\n" + "-"*40)
print("Результаты:")
print("-"*40)
print(warehouses_df[['name', 'latitude', 'longitude']])

# Сохраняем РЦ/РФЦ в файл
save_to_json(warehouses_df, "warehouses_rc_rfc_coordinates.json")

In [ ]:
# Обаботка сортировочных центров (СЦ)
# Список сортировочных центров (СЦ) в Москве и МО
sorting_centers_data = [
    {"name": "Анненский", "address": "г. Москва, Анненский проезд, д. 3, стр. 1"},
    {"name": "Варшавская", "address": "г. Москва, 1-й Варшавский пр-д, д. 2, стр. 9А"},
    {"name": "Железнодорожный", "address": "обл. Московская, г. Балашиха, д. Пестово, д. 14А"},
    {"name": "Истра", "address": "Московская обл., г. Истра, д. Давыдовское, ул. Дачная, стр. 3"},
    {"name": "Королев", "address": "Московская обл., г. Королев, ул. Силикатная, д. 10А"},
    {"name": "Огуднево", "address": "Московская область, городской округ Щёлково, деревня Огуднево, ул. Полевая, д. 1/1"},
    {"name": "Щербинка", "address": "обл. Московская, г. Подольск, д. Борисовка, пром. зона ПромТехАльянс, д. 1, стр. 2"},
    {"name": "Челобитьево", "address": "г. Мытищи, д. Челобитьево, Осташковское шоссе, владение 15, стр. 1"},
    {"name": "Нижнее Велино", "address": "обл. Московская, р-н Раменский, д. Нижнее Велино, ш. Старо-Рязанское, корпус 2, стр. 2"},
    {"name": "Воровского", "address": "Московская область, р.п. имени Воровского, ул. Мира, д. 5, склад 16"},
    {"name": "Львовский (ТСЦ)", "address": "142155, обл. Московская, г. Подольск, пром. зона Львовский, ул. Московская, д. 69, стр. 5"},
    {"name": "Химки", "address": "141401, обл. Московская, г. Химки, пр-д Коммунальный, 30А, стр. 1"},
    {"name": "Сухарево", "address": "Московская обл., Городской округ Мытищи, д. Сухарево, д. 135"},
    {"name": "Чёрная Грязь", "address": "Московская обл., Солнечногорский р-н, дер. Черная Грязь, ул. Сходненская, стр. 1"},
    {"name": "Балабаново", "address": "Калужская область, г. Балабаново, Киевское шоссе, д. 96, стр. 2"},
    {"name": "МКШВ", "address": "г. Москва, ул. Рябиновая, д. 44, стр. 28"},
    {"name": "Скотопрогонная", "address": "г. Москва, ул. Скотопрогонная, д. 35, стр. 3"}
]

sorting_centers_df = pd.DataFrame(sorting_centers_data)

# Геокодируем СЦ
print("Используем Яндекс API для геокодирования СЦ...")
coordinates_sc = []
for i, address in enumerate(sorting_centers_df['address']):
    print(f"\n--- Обрабатываем СЦ {i+1}/{len(sorting_centers_df)} ---")
    lat, lon = geocode_yandex(address, YANDEX_API_KEY)
    coordinates_sc.append((lat, lon))
    time.sleep(0.5)  # Пауза между запросами

# Добавляем координаты в DataFrame
sorting_centers_df['latitude'] = [coord[0] for coord in coordinates_sc]
sorting_centers_df['longitude'] = [coord[1] for coord in coordinates_sc]

# Проверяем результаты СЦ
print("\n" + "-"*40)
print("Результаты:")
print("-"*40)
print(sorting_centers_df[['name', 'latitude', 'longitude']])

# Сохраняем СЦ в файл
save_to_json(sorting_centers_df, "sorting_centers_coordinates.json")

In [ ]:
# Финальная статистика
print(f"Обработано РЦ/РФЦ: {len(warehouses_df)}")
print(f"Обработано СЦ: {len(sorting_centers_df)}")
print(f"Всего объектов: {len(warehouses_df) + len(sorting_centers_df)}")

# Создаем объединенный DataFrame
all_warehouses_df = pd.concat([warehouses_df, sorting_centers_df], ignore_index=True)
print(f"Объединенный файл создан: {len(all_warehouses_df)} объектов")

# Сохраняем объединенный файл
save_to_json(all_warehouses_df, "all_warehouses_coordinates.json")

In [ ]:
import matplotlib.pyplot as plt
import osmnx as ox

# 1. Карта с 10 складами (РЦ/РФЦ)
fig1, ax1 = plt.subplots(figsize=(15, 12))

# Рисуем дорожную сеть
ox.plot_graph(graph,
              ax=ax1,
              node_size=0,  # не показываем узлы
              edge_color='lightgray',
              edge_linewidth=0.5,
              bgcolor='white',
              show=False,
              close=False)

# Рисуем 10 складов
ax1.scatter(warehouses_df['longitude'],
            warehouses_df['latitude'],
            c='red', s=100, edgecolors='black', linewidth=1.5,
            label='Склады РЦ/РФЦ')

# Подписываем склады
for idx, row in warehouses_df.iterrows():
    ax1.annotate(row['name'][:15] + '...' if len(row['name']) > 15 else row['name'],
                (row['longitude'], row['latitude']),
                xytext=(5, 5), textcoords='offset points',
                fontsize=9, fontweight='bold',
                bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))

ax1.set_title('10 складов РЦ/РФЦ на карте Московского региона', fontsize=14, fontweight='bold')
ax1.set_xlabel('Долгота')
ax1.set_ylabel('Широта')
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('warehouses_10_map.png', dpi=300, bbox_inches='tight')
plt.show()

# 2. Карта со всеми складами
fig2, ax2 = plt.subplots(figsize=(15, 12))

# Рисуем дорожную сеть
ox.plot_graph(graph,
              ax=ax2,
              node_size=0,
              edge_color='lightgray',
              edge_linewidth=0.5,
              bgcolor='white',
              show=False,
              close=False)

# Рисуем все склады разными цветами
# РЦ/РФЦ - красные, СЦ - синие
colors = []
for idx, row in all_warehouses_df.iterrows():
    if idx < len(warehouses_df):  # первые 10 - РЦ/РФЦ
        colors.append('red')
    else:  # остальные - СЦ
        colors.append('blue')

ax2.scatter(all_warehouses_df['longitude'],
            all_warehouses_df['latitude'],
            c=colors, s=100, edgecolors='black', linewidth=1.5)

# Подписываем только начальный склад и несколько крупных
for idx, row in all_warehouses_df.iterrows():
    # Подписываем начальный склад
    if idx == 0:
        ax2.annotate(row['name'][:20] + '...' if len(row['name']) > 20 else row['name'],
                    (row['longitude'], row['latitude']),
                    xytext=(5, 5), textcoords='offset points',
                    fontsize=9, fontweight='bold',
                    bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))
    # Подписываем еще 5 случайных складов
    elif idx in [5, 10, 15, 20, 25] and idx < len(all_warehouses_df):
        ax2.annotate(row['name'][:15] + '...' if len(row['name']) > 15 else row['name'],
                    (row['longitude'], row['latitude']),
                    xytext=(5, 5), textcoords='offset points',
                    fontsize=8,
                    bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.5))

# Добавляем легенду
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], marker='o', color='w', label='Склады РЦ/РФЦ',
           markerfacecolor='red', markersize=10, markeredgecolor='black'),
    Line2D([0], [0], marker='o', color='w', label='Сортировочные центры',
           markerfacecolor='blue', markersize=10, markeredgecolor='black')
]
ax2.legend(handles=legend_elements, loc='upper right')

ax2.set_title(f'Все склады на карте ({len(all_warehouses_df)} объектов)',
             fontsize=14, fontweight='bold')
ax2.set_xlabel('Долгота')
ax2.set_ylabel('Широта')
ax2.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('warehouses_all_map.png', dpi=300, bbox_inches='tight')
plt.show()

# Классические методы

In [ ]:
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import itertools
from collections import defaultdict
import json
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Загрузка дорожной сети из файла
graph = ox.load_graphml("moscow_region_drive_network.graphml")
print("Дорожный граф загружен из файла")

# Получаем данные об узлах и ребрах графа
nodes, edges = ox.graph_to_gdfs(graph)
print(f"Граф содержит: {graph.number_of_nodes()} узлов, {graph.number_of_edges()} дорог")

# Загрузка данных о складах из JSON файла
with open('warehouses_rc_rfc_coordinates.json', 'r', encoding='utf-8') as f:
    warehouses_json = json.load(f)

# Создаем DataFrame из JSON данных
warehouses_data = []
for warehouse in warehouses_json:
    warehouses_data.append({
        "name": warehouse['name'],
        "latitude": warehouse['latitude'],
        "longitude": warehouse['longitude']
    })

warehouses_df = pd.DataFrame(warehouses_data)
print(f"Загружено {len(warehouses_df)} складов из JSON файла")

# Определяем начальную точку (ПАВЛО_СЛОБОДСКОЕ_РЦ)
START_WAREHOUSE = 0
print(f"Начальная точка: {warehouses_df.iloc[START_WAREHOUSE]['name']}")

In [ ]:
warehouses_df

"Подключаем" склады к ближайщим узлам

In [ ]:
#анализ подключения складов
def calculate_geodesic_distance(lat1, lon1, lat2, lon2):
    """
    Вычисляет расстояние между двумя точками на Земле по формуле гаверсинусов
    Возвращает расстояние в метрах
    """
    from math import radians, sin, cos, sqrt, atan2

    R = 6371000  # Радиус Земли в метрах

    # Перевод координат в радианы
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)

    # Разницы координат
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Формула гаверсинусов
    a = sin(dlat/2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))

    return R * c

def analyze_warehouse_connections(graph, warehouse_point, warehouse_node, warehouse_name, radius_meters=2000):
    """
    Детальный анализ подключения конкретного склада к дорожной сети

    Args:
        graph: дорожный граф
        warehouse_point: координаты склада (lat, lon)
        warehouse_node: привязанный узел графа
        warehouse_name: название склада
        radius_meters: радиус анализа в метрах

    Returns:
        dict: детальная информация о подключении
    """
    # Получаем данные привязанного узла
    node_data = graph.nodes[warehouse_node]
    node_point = (node_data['y'], node_data['x'])  # (lat, lon)

    # Вычисляем расстояние от склада до узла
    distance_to_node = calculate_geodesic_distance(
        warehouse_point[0], warehouse_point[1],
        node_point[0], node_point[1]
    )

    # Анализируем окрестности узла (подграф в радиусе)
    subgraph = ox.graph.graph_from_point(
        node_point,
        dist=radius_meters,
        network_type='drive'
    )

    # Анализ типов дорог в окрестностях
    if subgraph.number_of_edges() > 0:
        highway_types = []
        for u, v, data in subgraph.edges(data=True):
            highway = data.get('highway', 'unknown')
            if isinstance(highway, list):
                highway_types.extend(highway)
            else:
                highway_types.append(highway)
        road_type_counts = pd.Series(highway_types).value_counts()
    else:
        road_type_counts = pd.Series()

    # Анализ подключенных дорог к конкретному узлу
    connected_roads_info = []
    for neighbor in graph.neighbors(warehouse_node):
        edge_data = graph.get_edge_data(warehouse_node, neighbor)
        if edge_data:
            # Берем первую дорогу (может быть несколько между узлами)
            first_edge = list(edge_data.values())[0]
            road_type = first_edge.get('highway', 'unknown')
            if isinstance(road_type, list):
                road_type = road_type[0]
            length = first_edge.get('length', 0)
            name = first_edge.get('name', 'Без названия')
            connected_roads_info.append({
                'neighbor_node': neighbor,
                'road_type': road_type,
                'length_m': length,
                'name': name
            })

    return {
        'warehouse_name': warehouse_name,
        'warehouse_coords': warehouse_point,
        'node_id': warehouse_node,
        'node_coords': node_point,
        'distance_to_node_m': distance_to_node,
        'subgraph_stats': {
            'nodes': subgraph.number_of_nodes(),
            'edges': subgraph.number_of_edges()
        },
        'road_types_in_radius': road_type_counts,
        'directly_connected_roads': connected_roads_info,
        'analysis_radius_m': radius_meters
    }

In [ ]:
# Привязка складов к узлам графа и детальный анализ
warehouse_nodes = []
detailed_connections = []

for idx, warehouse in warehouses_df.iterrows():
    point = (warehouse['latitude'], warehouse['longitude'])

    # Находим ближайший узел дорожной сети к складу
    nearest_node = ox.distance.nearest_nodes(graph, point[1], point[0])
    warehouse_nodes.append(nearest_node)

    # Детальный анализ подключения
    connection_analysis = analyze_warehouse_connections(
        graph, point, nearest_node, warehouse['name']
    )
    detailed_connections.append(connection_analysis)

    print(f"{warehouse['name']:25} -> Узел {nearest_node}")
    print(f"  Расстояние до узла: {connection_analysis['distance_to_node_m']:.1f} м")
    print(f"  Подключен к {len(connection_analysis['directly_connected_roads'])} дорогам")

warehouses_df['node_id'] = warehouse_nodes

In [ ]:
# Детальный анализ для конкретного склада
first_connection = detailed_connections[0]  # Первый склад - нашей начальной точки
print(f"Склад: {first_connection['warehouse_name']}")
print(f"Узел графа: {first_connection['node_id']}")
print(f"Координаты склада: {first_connection['warehouse_coords']}")
print(f"Координаты узла: {first_connection['node_coords']}")
print(f"Расстояние до узла: {first_connection['distance_to_node_m']:.1f} м")

print(f"\nДорожная сеть в радиусе {first_connection['analysis_radius_m']} м:")
print(f"  - Узлов: {first_connection['subgraph_stats']['nodes']}")
print(f"  - Дорог: {first_connection['subgraph_stats']['edges']}")

print("\nТипы дорог в окрестностях:")
if not first_connection['road_types_in_radius'].empty:
    for road_type, count in first_connection['road_types_in_radius'].head(5).items():
        print(f"  - {road_type}: {count} дорог")
else:
    print("  - Нет данных о дорогах")

print("\nНепосредственные подключения:")
if first_connection['directly_connected_roads']:
    for i, road in enumerate(first_connection['directly_connected_roads'], 1):
        print(f"  {i}. К узлу {road['neighbor_node']}:")
        print(f"     Тип: {road['road_type']}")
        print(f"     Длина: {road['length_m']:.0f} м")
        print(f"     Название: {road['name']}")
else:
    print("  - Нет непосредственных подключений")

In [ ]:
# Анализ качества подключения складов
def analyze_connection_quality(detailed_connections):
    """Анализирует качество подключения складов к дорожной сети"""

    good_connections = 0
    medium_connections = 0
    poor_connections = 0

    for connection in detailed_connections:
        distance = connection['distance_to_node_m']
        road_count = len(connection['directly_connected_roads'])

        # Критерии качества
        if distance <= 200 and road_count >= 2:
            quality = "ХОРОШЕЕ"
            good_connections += 1
        elif distance <= 500 and road_count >= 1:
            quality = "СРЕДНЕЕ"
            medium_connections += 1
        else:
            quality = "ПЛОХОЕ"
            poor_connections += 1

        print(f"{connection['warehouse_name']:25} -> {quality} "
              f"({distance:.0f} м, {road_count} дорог)")

    print(f"\nИТОГО: Хороших: {good_connections}, Средних: {medium_connections}, "
          f"Плохих: {poor_connections}")

# Запускаем анализ
analyze_connection_quality(detailed_connections)

In [ ]:
# вычисление матрицы расстояния между складами
def calculate_route_distance(route, distance_matrix):
    """
    Вычисляет общее расстояние маршрута на основе матрицы расстояний
    """
    total_distance = 0
    for i in range(len(route) - 1):
        from_node = route[i]
        to_node = route[i+1]
        total_distance += distance_matrix[from_node][to_node]
    return total_distance

# Создаем матрицу расстояний между всеми складами
n_warehouses = len(warehouse_nodes)
distance_matrix = np.zeros((n_warehouses, n_warehouses))

print("Вычисляем попарные расстояния между складами")
for i in range(n_warehouses):
    for j in range(i+1, n_warehouses):
        try:
            # Вычисляем кратчайшее расстояние по дорожной сети
            distance = nx.shortest_path_length(graph, warehouse_nodes[i], warehouse_nodes[j], weight='length')
            distance_matrix[i][j] = distance
            distance_matrix[j][i] = distance
        except nx.NetworkXNoPath:
            # Если пути нет, ставим большое число
            distance_matrix[i][j] = 1e9
            distance_matrix[j][i] = 1e9
            print(f"Предупреждение: нет пути между складом {i} и складом {j}")

print("Матрица расстояний создана!")
print(f"Размер матрицы: {distance_matrix.shape}")

# Выводим примеры расстояний
print("\nПримеры расстояний между складами:")
for i in range(min(3, n_warehouses)):
    for j in range(i+1, min(4, n_warehouses)):
        dist_km = distance_matrix[i][j] / 1000
        print(f"  {warehouses_df.iloc[i]['name']} -> {warehouses_df.iloc[j]['name']}: {dist_km:.1f} км")

Реализация классических алгоритмов рещения TSP

In [ ]:
# Список для хранения результатов всех алгоритмов
results = []

Алгоритм 1: Жадный алгоритм


In [ ]:
def greedy_tsp(distance_matrix, start_index=0):
    """
    Жадный алгоритм ближайшего соседа для решения TSP

    Алгоритм:
    1. Начинаем с начальной точки
    2. На каждом шаге выбираем ближайший непосещенный город
    3. Повторяем пока не посетим все города
    4. Возвращаемся в начальную точку

    Сложность: O(n^2)
    """
    start_time = time.time()

    n = distance_matrix.shape[0]
    visited = [False] * n
    route = [start_index]
    visited[start_index] = True
    current = start_index

    # Последовательно выбираем ближайшие непосещенные города
    while len(route) < n:
        nearest_distance = float('inf')
        nearest_city = -1

        for next_city in range(n):
            if not visited[next_city] and distance_matrix[current][next_city] < nearest_distance:
                nearest_distance = distance_matrix[current][next_city]
                nearest_city = next_city

        route.append(nearest_city)
        visited[nearest_city] = True
        current = nearest_city

    # Замыкаем маршрут - возвращаемся в начальную точку
    route.append(start_index)
    total_distance = calculate_route_distance(route, distance_matrix)

    end_time = time.time()
    computation_time = end_time - start_time

    return route, total_distance, computation_time

In [ ]:
greedy_route, greedy_distance, greedy_time = greedy_tsp(distance_matrix, START_WAREHOUSE)

results.append({
    'method': 'Жадный алгоритм',
    'route': greedy_route,
    'distance_km': greedy_distance / 1000,
    'time_sec': greedy_time,
    'description': 'На каждом шаге выбирает ближайший непосещенный склад'
})

print(f"Завершено: {greedy_distance/1000:.2f} км за {greedy_time:.6f} сек")
print(f"Маршрут: {greedy_route}")

Алгоритм 2: 2-OPT Локальный поиск

In [ ]:
def tsp_2opt(route, distance_matrix, max_iterations=1000):
    """
    Алгоритм 2-opt для улучшения существующего маршрута

    Алгоритм:
    1. Берем существующий маршрут
    2. Пытаемся улучшить его, переставляя пары ребер
    3. Если находим улучшение, принимаем его
    4. Повторяем пока есть улучшения или до достижения максимума итераций

    Сложность: O(n^2 * max_iterations)
    """
    start_time = time.time()

    best_route = route.copy()
    best_distance = calculate_route_distance(route, distance_matrix)
    improved = True
    iterations = 0

    while improved and iterations < max_iterations:
        improved = False

        # Перебираем все возможные пары индексов для перестановки
        for i in range(1, len(route) - 2):
            for j in range(i + 1, len(route) - 1):
                if j - i == 1:
                    continue

                # Создаем новый маршрут с перевернутой подпоследовательностью
                new_route = route[:i] + route[i:j+1][::-1] + route[j+1:]
                new_distance = calculate_route_distance(new_route, distance_matrix)

                # Если нашли улучшение, обновляем лучший маршрут
                if new_distance < best_distance:
                    best_route = new_route
                    best_distance = new_distance
                    improved = True
                    route = new_route
                    break

            if improved:
                break

        iterations += 1

    end_time = time.time()
    computation_time = end_time - start_time

    return best_route, best_distance, computation_time

In [ ]:
opt2_route, opt2_distance, opt2_time = tsp_2opt(greedy_route, distance_matrix)

results.append({
    'method': '2-OPT',
    'route': opt2_route,
    'distance_km': opt2_distance / 1000,
    'time_sec': greedy_time + opt2_time,
    'description': 'Улучшает существующий маршрут перестановкой пар ребер',
    'improvement': f"{(greedy_distance - opt2_distance) / greedy_distance * 100:.1f}%"
})

print(f"Завершено: {opt2_distance/1000:.2f} км за {opt2_time:.6f} сек")
print(f"Улучшение: {results[-1]['improvement']}")
print(f"Маршрут: {opt2_route}")

Алгоритм 3: Метод минимального остовного дерева (MST)

In [ ]:
def mst_tsp(distance_matrix, start_index=0):
    """
    Метод минимального остовного дерева для приближенного решения TSP

    Алгоритм:
    1. Строим полный граф городов
    2. Находим минимальное остовное дерево
    3. Обходим дерево в глубину для получения маршрута
    4. Удаляем повторяющиеся города

    Сложность: O(n^2) для плотных графов
    """
    start_time = time.time()

    n = distance_matrix.shape[0]

    # Строим полный граф городов
    G = nx.Graph()
    for i in range(n):
        for j in range(i+1, n):
            G.add_edge(i, j, weight=distance_matrix[i][j])

    # Строим минимальное остовное дерево
    mst = nx.minimum_spanning_tree(G)

    # Обход в глубину для получения маршрута
    route_dfs = list(nx.dfs_preorder_nodes(mst, start_index))

    # Убеждаемся, что посетили все города
    visited = set(route_dfs)
    missing = set(range(n)) - visited
    if missing:
        route_dfs.extend(missing)

    # Замыкаем маршрут
    route_dfs.append(start_index)
    total_distance = calculate_route_distance(route_dfs, distance_matrix)

    end_time = time.time()
    computation_time = end_time - start_time

    return route_dfs, total_distance, computation_time

In [ ]:
mst_route, mst_distance, mst_time = mst_tsp(distance_matrix, START_WAREHOUSE)

results.append({
    'method': 'MST',
    'route': mst_route,
    'distance_km': mst_distance / 1000,
    'time_sec': mst_time,
    'description': 'Строит минимальное остовное дерево и обходит его'
})

print(f"Завершено: {mst_distance/1000:.2f} км за {mst_time:.6f} сек")
print(f"Маршрут: {mst_route}")

Алгоритм 4: Генетический алгоритм

In [ ]:
def genetic_algorithm_tsp(distance_matrix, start_index=0, pop_size=50, generations=100, mutation_rate=0.1):
    """
    Генетический алгоритм для решения TSP

    Алгоритм:
    1. Создаем начальную популяцию случайных маршрутов
    2. Оцениваем качество каждого маршрута (фитнес-функция)
    3. Отбираем лучшие маршруты для размножения
    4. Применяем кроссовер и мутацию для создания нового поколения
    5. Повторяем заданное количество поколений

    Сложность: O(pop_size * generations * n)
    """
    start_time = time.time()

    n = distance_matrix.shape[0]

    def initialize_population():
        """Создает начальную популяцию случайных маршрутов"""
        population = []
        for _ in range(pop_size):
            route = list(range(n))
            route.remove(start_index)
            np.random.shuffle(route)
            route = [start_index] + route + [start_index]
            population.append(route)
        return population

    def fitness(route):
        """Фитнес-функция: обратная пропорция расстоянию (чем короче маршрут, тем лучше)"""
        return 1 / calculate_route_distance(route, distance_matrix)

    def order_crossover(parent1, parent2):
        """Оператор упорядоченного кроссовера (OX)"""
        p1 = parent1[1:-1]  # Убираем начальную/конечную точку
        p2 = parent2[1:-1]

        size = len(p1)
        child = [-1] * size

        # Выбираем случайный сегмент из первого родителя
        start, end = sorted(np.random.choice(range(size), 2, replace=False))
        child[start:end+1] = p1[start:end+1]

        # Заполняем оставшиеся позиции генами из второго родителя
        pointer = (end + 1) % size
        for gene in p2:
            if gene not in child:
                while child[pointer] != -1:
                    pointer = (pointer + 1) % size
                child[pointer] = gene

        return [start_index] + child + [start_index]

    def mutate(route):
        """Оператор мутации: перестановка двух случайных городов"""
        if np.random.random() < mutation_rate:
            idx1, idx2 = np.random.choice(range(1, len(route)-1), 2, replace=False)
            route[idx1], route[idx2] = route[idx2], route[idx1]
        return route

    # Основной цикл генетического алгоритма
    population = initialize_population()
    best_route = None
    best_distance = float('inf')

    for generation in range(generations):
        # Оценка качества популяции
        fitness_scores = [fitness(route) for route in population]

        # Отбор лучшей особи текущего поколения
        current_best_idx = np.argmax(fitness_scores)
        current_best_route = population[current_best_idx]
        current_best_distance = calculate_route_distance(current_best_route, distance_matrix)

        # Обновление глобального лучшего решения
        if current_best_distance < best_distance:
            best_distance = current_best_distance
            best_route = current_best_route.copy()

        # Создание нового поколения
        new_population = [best_route.copy()]  # Элитизм: сохраняем лучшую особь

        while len(new_population) < pop_size:
            # Турнирный отбор
            tournament_size = 3
            tournament_indices = np.random.choice(range(pop_size), tournament_size)
            tournament_fitness = [fitness_scores[i] for i in tournament_indices]
            parent1_idx = tournament_indices[np.argmax(tournament_fitness)]
            parent2_idx = tournament_indices[np.argmax(tournament_fitness)]

            parent1 = population[parent1_idx]
            parent2 = population[parent2_idx]

            # Кроссовер и мутация
            child = order_crossover(parent1, parent2)
            child = mutate(child)

            new_population.append(child)

        population = new_population

    end_time = time.time()
    computation_time = end_time - start_time

    return best_route, best_distance, computation_time

In [ ]:
ga_runs = []
for run in range(5):
    print(f"Запуск {run+1}/5...", end=" ")
    ga_route, ga_distance, ga_time = genetic_algorithm_tsp(
        distance_matrix, START_WAREHOUSE, pop_size=50, generations=100
    )
    ga_runs.append({
        'route': ga_route,
        'distance_km': ga_distance / 1000,
        'time_sec': ga_time
    })
    print(f"{ga_distance/1000:.2f} км")

# Анализируем результаты генетического алгоритма
ga_distances = [run['distance_km'] for run in ga_runs]
best_ga_idx = np.argmin(ga_distances)
best_ga = ga_runs[best_ga_idx]

results.append({
    'method': 'Генетический алгоритм',
    'route': best_ga['route'],
    'distance_km': best_ga['distance_km'],
    'time_sec': np.mean([run['time_sec'] for run in ga_runs]),
    'description': 'Эволюционный поиск с кроссовером и мутацией',
    'statistics': {
        'best_km': best_ga['distance_km'],
        'mean_km': np.mean(ga_distances),
        'std_km': np.std(ga_distances),
        'runs': len(ga_runs)
    }
})

print(f"Лучший результат: {best_ga['distance_km']:.2f} км")
print(f"Статистика: среднее {np.mean(ga_distances):.2f} ± {np.std(ga_distances):.2f} км")
print(f"Лучший маршрут: {best_ga['route']}")

Анализ и визуализация

In [ ]:
# Создаем расширенный DataFrame с дополнительной статистикой
results_data = []
for r in results:
    row_data = {
        'Метод': r['method'],
        'Расстояние_км': r['distance_km'],
        'Время_сек': r['time_sec'],
        'Описание': r['description'],
    }

    results_data.append(row_data)

# Создаем DataFrame
results_df = pd.DataFrame(results_data)

# Сортируем по расстоянию
results_df = results_df.sort_values('Расстояние_км')

In [ ]:
results_df

In [ ]:
# Создаем графики
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# График 1: Расстояние по методам
results_df.set_index('Метод')['Расстояние_км'].plot(
    kind='bar', ax=axes[0], color='skyblue',
    title='Сравнение длины маршрутов по методам',
    ylabel='Расстояние (км)'
)
axes[0].tick_params(axis='x', rotation=45)

# Добавляем значения на столбцы первого графика
for i, v in enumerate(results_df['Расстояние_км']):
    axes[0].text(i, v + 5, f'{v:.1f} км', ha='center', va='bottom', fontsize=9)

# График 2: Время выполнения
results_df.set_index('Метод')['Время_сек'].plot(
    kind='bar', ax=axes[1], color='lightcoral',
    title='Время выполнения алгоритмов',
    ylabel='Время (сек)'
)
axes[1].tick_params(axis='x', rotation=45)

# Добавляем значения на столбцы второго графика
for i, v in enumerate(results_df['Время_сек']):
    axes[1].text(i, v + 0.001, f'{v:.4f} сек', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig('pandas_analysis_results.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Детальный анализ маршрутов
for result in results:
    method = result['method']
    route = result['route']
    distance = result['distance_km']

    print(f"\n{method}:")
    print(f"Общее расстояние: {distance:.2f} км")
    print(f"Количество посещений: {len(route)} точек")
    print(f"Порядок посещения: ", end="")

    # выводим порядок посещения
    for i, point in enumerate(route[:-1]):
        if i == 0:
            print(f"{warehouses_df.iloc[point]['name']}", end="")
        else:
            print(f" -> {warehouses_df.iloc[point]['name']}", end="")
    print(f" -> {warehouses_df.iloc[route[0]]['name']} (возврат)")


## Эксперименты

In [ ]:
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import copy
from collections import defaultdict
import json
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Загрузка исходного графа дорожной сети
graph = ox.load_graphml("moscow_region_drive_network.graphml")
print("Исходный граф дорожной сети загружен")

# Загрузка данных о складах из JSON файла
print("Загружаем данные о складах из JSON файла...")
with open('warehouses_rc_rfc_coordinates.json', 'r', encoding='utf-8') as f:
    warehouses_json = json.load(f)

# Создаем DataFrame из JSON данных
warehouses_data = []
for warehouse in warehouses_json:
    warehouses_data.append({
        "name": warehouse['name'],
        "latitude": warehouse['latitude'],
        "longitude": warehouse['longitude']
    })

warehouses_df = pd.DataFrame(warehouses_data)
START_WAREHOUSE = 0  # ПАВЛО_СЛОБОДСКОЕ_РЦ как начальная точка

print(f"Загружено {len(warehouses_df)} складов из JSON файла")

# Привязка складов к узлам графа
print("Привязка складов к ближайшим узлам графа...")
warehouse_nodes = []
for idx, warehouse in warehouses_df.iterrows():
    point = (warehouse['latitude'], warehouse['longitude'])
    nearest_node = ox.distance.nearest_nodes(graph, point[1], point[0])
    warehouse_nodes.append(nearest_node)
    print(f"   {warehouse['name']} -> Узел {nearest_node}")

In [ ]:
warehouses_df['node_id'] = warehouse_nodes
warehouses_df

In [ ]:
# Добавлеине шума
def add_traffic_jams(graph, jam_roads, traffic_multiplier=10):
    """
    Моделирует пробки на указанных дорогах путем умножения их длины

    Принцип работы:
    - Увеличивает длину дорог в N раз, что эквивалентно увеличению времени проезда
    - Используется для моделирования заторов и снижения скорости движения
    """
    graph_jammed = graph.copy()

    print(f"Добавляем пробки на {len(jam_roads)} дорог (умножение длины в {traffic_multiplier} раз)")

    modified_edges = 0
    for u, v, key in jam_roads:
        if graph_jammed.has_edge(u, v, key):
            original_length = graph_jammed[u][v][key].get('length', 0)
            graph_jammed[u][v][key]['length'] = original_length * traffic_multiplier
            graph_jammed[u][v][key]['traffic_jam'] = True
            graph_jammed[u][v][key]['original_length'] = original_length
            modified_edges += 1

    print(f"Изменено {modified_edges} участков дорог")
    return graph_jammed

# можно считать, как блокировка/ какой-то инцидент/ непроходимая пробка
def block_roads(graph, blocked_roads, block_weight=1e9):
    """
    Полностью перекрывает дороги, делая их непроезжими

    Принцип работы:
    - Устанавливает очень большую длину для заблокированных дорог
    - Алгоритмы поиска пути будут избегать эти дороги
    - Моделирует ремонтные работы, аварии или другие препятствия
    """
    graph_blocked = graph.copy()

    print(f"Блокируем {len(blocked_roads)} дорог")

    blocked_count = 0
    for u, v, key in blocked_roads:
        if graph_blocked.has_edge(u, v, key):
            graph_blocked[u][v][key]['length'] = block_weight
            graph_blocked[u][v][key]['blocked'] = True
            blocked_count += 1

    print(f"Заблокировано {blocked_count} дорог")
    return graph_blocked

def find_important_roads(graph, road_types=['motorway', 'trunk', 'primary']):
    """
    Находит важные дороги по их типу для добавления шума
    """
    important_roads = []

    for u, v, key, data in graph.edges(keys=True, data=True):
        highway = data.get('highway', '')
        if isinstance(highway, list):
            highway = highway[0] if highway else ''

        if highway in road_types:
            important_roads.append((u, v, key))

    return important_roads

def find_roads_by_name(graph, name_patterns):
    """
    Находит дороги по названию (например, МКАД)
    """
    matched_roads = []

    for u, v, key, data in graph.edges(keys=True, data=True):
        name = data.get('name', '')
        if isinstance(name, list):
            name = ' '.join(name)

        if any(pattern.lower() in str(name).lower() for pattern in name_patterns):
            matched_roads.append((u, v, key))

    return matched_roads

def select_random_roads(roads_list, percentage):
    """
    Выбирает случайные дороги из списка для добавления шума
    """
    if not roads_list:
        return []

    n_to_select = max(1, int(len(roads_list) * percentage))
    n_to_select = min(n_to_select, len(roads_list))

    indices = np.random.choice(len(roads_list), size=n_to_select, replace=False)
    selected_roads = [roads_list[i] for i in indices]

    return selected_roads

def visualize_modified_roads(original_graph, modified_graph, scenario_name, warehouses_df, warehouse_nodes):
    """
    Визуализирует дороги, которые были изменены в сценарии
    """
    fig, ax = plt.subplots(figsize=(15, 12))

    # Рисуем исходную дорожную сеть серым цветом
    ox.plot_graph(original_graph, ax=ax, node_size=0, edge_linewidth=0.5,
                  edge_color='lightgray', bgcolor='white', show=False, close=False)

    # Находим измененные дороги
    jammed_roads = []
    blocked_roads = []

    for u, v, key, data in modified_graph.edges(keys=True, data=True):
        if data.get('traffic_jam', False):
            jammed_roads.append((u, v, key))
        if data.get('blocked', False):
            blocked_roads.append((u, v, key))

    # Рисуем дороги с пробками оранжевым
    if jammed_roads:
        for u, v, key in jammed_roads:
            if modified_graph.has_edge(u, v, key):
                edge_data = modified_graph[u][v][key]
                coords = []
                if 'geometry' in edge_data:
                    # Извлекаем координаты из LineString
                    coords = list(edge_data['geometry'].coords)
                else:
                    # Если нет геометрии, используем координаты узлов
                    u_data = modified_graph.nodes[u]
                    v_data = modified_graph.nodes[v]
                    coords = [(u_data['x'], u_data['y']), (v_data['x'], v_data['y'])]

                if coords:
                    x_vals, y_vals = zip(*coords)
                    ax.plot(x_vals, y_vals, color='orange', linewidth=3, alpha=0.8, label='Пробки' if not jammed_roads.index((u,v,key)) else "")

    # Рисуем заблокированные дороги красным
    if blocked_roads:
        for u, v, key in blocked_roads:
            if modified_graph.has_edge(u, v, key):
                edge_data = modified_graph[u][v][key]
                coords = []
                if 'geometry' in edge_data:
                    coords = list(edge_data['geometry'].coords)
                else:
                    u_data = modified_graph.nodes[u]
                    v_data = modified_graph.nodes[v]
                    coords = [(u_data['x'], u_data['y']), (v_data['x'], v_data['y'])]

                if coords:
                    x_vals, y_vals = zip(*coords)
                    ax.plot(x_vals, y_vals, color='red', linewidth=4, alpha=0.8, label='Блокировки' if not blocked_roads.index((u,v,key)) else "")

    # Рисуем склады
    for idx, warehouse in warehouses_df.iterrows():
        color = 'blue' if idx == START_WAREHOUSE else 'green'
        ax.scatter(warehouse['longitude'], warehouse['latitude'],
                  c=color, s=100, alpha=0.8, edgecolors='black', zorder=5)
        ax.annotate(warehouse['name'],
                   (warehouse['longitude'], warehouse['latitude']),
                   xytext=(5, 5), textcoords='offset points', fontsize=9)

    # Легенда
    from matplotlib.lines import Line2D
    legend_elements = [
        Line2D([0], [0], color='lightgray', lw=2, label='Обычные дороги'),
        Line2D([0], [0], color='orange', lw=3, label='Дороги с пробками'),
        Line2D([0], [0], color='red', lw=4, label='Заблокированные дороги'),
        Line2D([0], [0], marker='o', color='blue', markersize=8, label='Начальный склад'),
        Line2D([0], [0], marker='o', color='green', markersize=8, label='Остальные склады')
    ]
    ax.legend(handles=legend_elements, loc='upper right')

    ax.set_title(f'Измененные дороги в сценарии: {scenario_name}', fontsize=16)
    plt.tight_layout()
    plt.savefig(f'modified_roads_{scenario_name}.png', dpi=300, bbox_inches='tight')
    plt.show()

    return len(jammed_roads), len(blocked_roads)

In [ ]:
# Создание раздичных "сценариев" с шумом
np.random.seed(42)  # Для воспроизводимости результатов

# Находим ключевые дороги для моделирования шума
important_roads = find_important_roads(graph)
mkad_roads = find_roads_by_name(graph, ['МКАД', 'MKAD', 'Московская кольцевая'])
highway_roads = find_roads_by_name(graph, [
    'Ленинградское', 'Ярославское', 'Дмитровское', 'Рижское',
    'Волоколамское', 'Пятницкое', 'Новорижское', 'Киевское',
    'Минское', 'Можайское', 'Калужское', 'Варшавское'
])

print(f"Найдено важных дорог: {len(important_roads)}")
print(f"Найдено дорог МКАД: {len(mkad_roads)}")
print(f"Найдено основных шоссе: {len(highway_roads)}")

# СЦЕНАРИЙ 1: Только пробки на основных дорогах
print("\n1. СЦЕНАРИЙ 1: Только пробки на основных дорогах")
print("   Моделирование: Умеренные пробки на 15% основных дорог")
jam_roads_scenario1 = select_random_roads(important_roads, 0.15)
graph_traffic_jam = add_traffic_jams(graph, jam_roads_scenario1, traffic_multiplier=8)

# Визуализируем измененные дороги
jammed_count1, blocked_count1 = visualize_modified_roads(graph, graph_traffic_jam,
                                                         "Пробки на основных дорогах",
                                                         warehouses_df, warehouse_nodes)

# СЦЕНАРИЙ 2: Блокировка ключевых дорог + пробки
print("\n2. СЦЕНАРИЙ 2: Блокировка ключевых дорог + пробки")
print("   Моделирование: Блокировка 8% МКАД + 4% шоссе + пробки на других дорогах")

# Используем ДРУГИЕ дороги для пробок в этом сценарии
jam_roads_scenario2 = select_random_roads(important_roads, 0.12)
# Исключаем дороги, которые уже использовались в первом сценарии
jam_roads_scenario2 = [road for road in jam_roads_scenario2 if road not in jam_roads_scenario1]

blocked_mkad = select_random_roads(mkad_roads, 0.08)
blocked_highways = select_random_roads(highway_roads, 0.04)
all_blocked = blocked_mkad + blocked_highways

# Сначала блокируем дороги, потом добавляем пробки на ДРУГИХ дорогах
graph_blocked = block_roads(graph, all_blocked)
graph_combined = add_traffic_jams(graph_blocked, jam_roads_scenario2, traffic_multiplier=8)

# Визуализируем измененные дороги
jammed_count2, blocked_count2 = visualize_modified_roads(graph, graph_combined,
                                                         "Блокировки и пробки",
                                                         warehouses_df, warehouse_nodes)

# СЦЕНАРИЙ 3: Умеренные условия
print("\n3. СЦЕНАРИЙ 3: Умеренные условия")
print("   Моделирование: Легкие пробки и минимальные блокировки")

# УМЕРЕННЫЕ УСЛОВИЯ: меньше пробок и блокировок, меньший множитель
moderate_jam_roads = select_random_roads(important_roads, 0.08)
moderate_blocked_mkad = select_random_roads(mkad_roads, 0.03)
moderate_blocked_highways = select_random_roads(highway_roads, 0.015)
all_blocked_moderate = moderate_blocked_mkad + moderate_blocked_highways

graph_moderate_blocked = block_roads(graph, all_blocked_moderate)
graph_moderate = add_traffic_jams(graph_moderate_blocked, moderate_jam_roads, traffic_multiplier=4)

# Визуализируем измененные дороги
jammed_count3, blocked_count3 = visualize_modified_roads(graph, graph_moderate,
                                                         "Умеренные условия",
                                                         warehouses_df, warehouse_nodes)

print("\nСтатистиа измененных дорог по сценариям:")
print(f"Сценарий 1 (Пробки): {jammed_count1} дорог с пробками, {blocked_count1} заблокированных")
print(f"Сценарий 2 (Блокировки+Пробки): {jammed_count2} дорог с пробками, {blocked_count2} заблокированных")
print(f"Сценарий 3 (Умеренные): {jammed_count3} дорог с пробками, {blocked_count3} заблокированных")

In [ ]:
# Проверяем, что склады достижимы и не перекрыты
def check_warehouses_accessibility(graph, warehouses_df, warehouse_nodes):
    """
    Проверяет, что все склады достижимы в графе (существует путь между всеми парами)
    """
    n = len(warehouse_nodes)
    unreachable_pairs = []

    # Проверяем что все узлы существуют в графе
    missing_nodes = []
    for node in warehouse_nodes:
        if node not in graph.nodes:
            missing_nodes.append(node)

    if missing_nodes:
        print(f"В графе отсутствуют узлы: {missing_nodes}")
        return False

    for i in range(n):
        for j in range(i+1, n):
            try:
                nx.shortest_path_length(graph, warehouse_nodes[i], warehouse_nodes[j], weight='length')
            except (nx.NetworkXNoPath, nx.NodeNotFound):
                unreachable_pairs.append((i, j, warehouses_df.iloc[i]['name'], warehouses_df.iloc[j]['name']))

    if unreachable_pairs:
        print(f"Найдено {len(unreachable_pairs)} недостижимых пар:")
        for i, j, name1, name2 in unreachable_pairs[:3]:
            print(f"   {name1} <-> {name2}")
        if len(unreachable_pairs) > 3:
            print(f"   ... и еще {len(unreachable_pairs) - 3} пар")
        return False
    else:
        print("Все склады достижимы!")
        return True

# Создаем словарь сценариев для тестирования
scenarios = {
    "Без шума": graph,
    "Пробки": graph_traffic_jam,
    "Блокировки+Пробки": graph_combined,
    "Умеренные условия": graph_moderate
}

# Проверяем доступность для всех сценариев
accessibility_results = {}
print("Проверка доступности складов по сценариям:")
for scenario_name, scenario_graph in scenarios.items():
    print(f"\n{scenario_name}:")
    accessible = check_warehouses_accessibility(scenario_graph, warehouses_df, warehouse_nodes)
    accessibility_results[scenario_name] = accessible

In [ ]:
# Пересчёт матрицы расстояний для всех сценариев
def calculate_distance_matrix_for_scenario(graph, warehouse_nodes, scenario_name):
    """
    Вычисляет матрицу расстояний между складами для заданного сценария
    Использует алгоритм Дейкстры для нахождения кратчайших путей
    """
    print(f"Вычисляем матрицу расстояний для '{scenario_name}'...")

    n = len(warehouse_nodes)
    distance_matrix = np.zeros((n, n))

    start_time = time.time()

    for i in range(n):
        for j in range(i+1, n):
            try:
                # Проверяем что узлы существуют в графе
                if warehouse_nodes[i] not in graph.nodes or warehouse_nodes[j] not in graph.nodes:
                    distance_matrix[i][j] = 1e9
                    distance_matrix[j][i] = 1e9
                    continue

                # Вычисляем кратчайший путь по дорожной сети
                path_length = nx.shortest_path_length(graph, warehouse_nodes[i], warehouse_nodes[j], weight='length')
                distance_matrix[i][j] = path_length
                distance_matrix[j][i] = path_length

            except (nx.NetworkXNoPath, nx.NodeNotFound):
                # Если пути нет, ставим большое число
                distance_matrix[i][j] = 1e9
                distance_matrix[j][i] = 1e9

    end_time = time.time()
    computation_time = end_time - start_time

    print(f"   '{scenario_name}': {computation_time:.2f} сек")

    return distance_matrix, computation_time

In [ ]:
# Вычисляем матрицы для всех сценариев
distance_matrices = {}
computation_times = {}

for scenario_name, scenario_graph in scenarios.items():
    if accessibility_results[scenario_name]:
        dist_matrix, comp_time = calculate_distance_matrix_for_scenario(
            scenario_graph, warehouse_nodes, scenario_name
        )
        distance_matrices[scenario_name] = dist_matrix
        computation_times[scenario_name] = comp_time
    else:
        print(f"   Пропускаем '{scenario_name}' - склады недостижимы")

In [ ]:
# Вспомогательные функции для алгоритмов
def calculate_route_distance(route, distance_matrix):
    """Вычисляет общее расстояние маршрута"""
    total_distance = 0
    for i in range(len(route) - 1):
        from_node = route[i]
        to_node = route[i+1]
        total_distance += distance_matrix[from_node][to_node]
    return total_distance

In [ ]:
# Тестируем все алгоритмы на всех сценариях

all_algorithm_results = []

for scenario_name, dist_matrix in distance_matrices.items():
    print(f"\nСЦЕНАРИЙ: {scenario_name}")
    print("-" * 40)

    scenario_results = []

    # 1. Жадный алгорит
    print("1. Жадный алгоритм")
    greedy_route, greedy_distance, greedy_time = greedy_tsp(dist_matrix, START_WAREHOUSE)
    scenario_results.append({
        'algorithm': 'Жадный',
        'route': greedy_route,
        'distance_km': greedy_distance / 1000,
        'time_sec': greedy_time,
        'scenario': scenario_name
    })
    print(f"   Результат: {greedy_distance/1000:.2f} км за {greedy_time:.6f} сек")

    # 2. 2-OPT (улучшение жадного)
    print("2. 2-OPT")
    opt2_route, opt2_distance, opt2_time = tsp_2opt(greedy_route, dist_matrix)
    scenario_results.append({
        'algorithm': '2-OPT',
        'route': opt2_route,
        'distance_km': opt2_distance / 1000,
        'time_sec': opt2_time,
        'scenario': scenario_name,
        'improvement': f"{(greedy_distance - opt2_distance) / greedy_distance * 100:.1f}%" if greedy_distance > 0 else "0.0%"
    })
    print(f"   Результат: {opt2_distance/1000:.2f} км за {opt2_time:.6f} сек")
    if opt2_distance < greedy_distance:
        print(f"   Улучшение: {scenario_results[-1]['improvement']}")

    # 3. MST
    print("3. MST")
    mst_route, mst_distance, mst_time = mst_tsp(dist_matrix, START_WAREHOUSE)
    scenario_results.append({
        'algorithm': 'MST',
        'route': mst_route,
        'distance_km': mst_distance / 1000,
        'time_sec': mst_time,
        'scenario': scenario_name
    })
    print(f"   Результат: {mst_distance/1000:.2f} км за {mst_time:.6f} сек")

    # 4. Генетический алгоритм (3 запуска)
    print("4. Генетический алгоритм (3 запуска)")
    ga_runs = []
    for run in range(3):
        print(f"   Запуск {run+1}/3", end=" ")
        ga_route, ga_distance, ga_time = genetic_algorithm_tsp(
            dist_matrix, START_WAREHOUSE, pop_size=30, generations=50
        )
        ga_runs.append({
            'route': ga_route,
            'distance_km': ga_distance / 1000,
            'time_sec': ga_time
        })
        print(f"{ga_distance/1000:.2f} км")

    # Анализируем результаты генетического алгоритма
    ga_distances = [run['distance_km'] for run in ga_runs]
    best_ga_idx = np.argmin(ga_distances)
    best_ga = ga_runs[best_ga_idx]

    scenario_results.append({
        'algorithm': 'Генетический',
        'route': best_ga['route'],
        'distance_km': best_ga['distance_km'],
        'time_sec': np.mean([run['time_sec'] for run in ga_runs]),
        'scenario': scenario_name,
        'statistics': {
            'best_km': best_ga['distance_km'],
            'mean_km': np.mean(ga_distances),
            'std_km': np.std(ga_distances),
            'runs': len(ga_runs)
        }
    })
    print(f"   Лучший результат: {best_ga['distance_km']:.2f} км")
    print(f"   Статистика: среднее {np.mean(ga_distances):.2f} ± {np.std(ga_distances):.2f} км")

    all_algorithm_results.extend(scenario_results)

In [ ]:
# Анализ

# Создаем DataFrame для анализа
results_df = pd.DataFrame(all_algorithm_results)

# Группируем по алгоритмам и сценариям
algorithm_performance = results_df.groupby(['algorithm', 'scenario']).agg({
    'distance_km': 'mean',
    'time_sec': 'mean'
}).round(5)

algorithm_performance

In [ ]:
# Визуализация результатов
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# 1. Сравнение алгоритмов по сценариям (расстояния)
scenarios_list = list(distance_matrices.keys())
algorithms = ['Жадный', '2-OPT', 'MST', 'Генетический']

# Подготовка данных для группированного графика
x = np.arange(len(scenarios_list))
width = 0.2

for i, algorithm in enumerate(algorithms):
    algorithm_distances = []
    for scenario in scenarios_list:
        result = results_df[(results_df['algorithm'] == algorithm) & (results_df['scenario'] == scenario)]
        if not result.empty:
            algorithm_distances.append(result['distance_km'].values[0])
        else:
            algorithm_distances.append(0)

    ax1.bar(x + i*width, algorithm_distances, width, label=algorithm, alpha=0.8)

ax1.set_xlabel('Сценарии')
ax1.set_ylabel('Расстояние (км)')
ax1.set_title('Сравнение алгоритмов по сценариям\n(Длина маршрута)')
ax1.set_xticks(x + width*1.5)
ax1.set_xticklabels(scenarios_list, rotation=45)
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Время выполнения по алгоритмам
algorithm_times = results_df.groupby('algorithm')['time_sec'].mean()
bars2 = ax2.bar(algorithm_times.index, algorithm_times.values, color=['#ff9999', '#66b3ff', '#99ff99', '#ffcc99'])
ax2.set_ylabel('Среднее время (сек)')
ax2.set_title('Среднее время выполнения алгоритмов')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3)

for bar, time_val in zip(bars2, algorithm_times.values):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.001,
            f'{time_val:.4f} сек', ha='center', va='bottom', fontsize=9)

# 3. Увеличение расстояния по сценариям (относительно базового)
base_scenario = "Без шума"
base_results = results_df[results_df['scenario'] == base_scenario]

increase_data = []
for scenario in scenarios_list[1:]:  # Исключаем базовый сценарий
    for algorithm in algorithms:
        base_result = base_results[base_results['algorithm'] == algorithm]
        current_result = results_df[(results_df['algorithm'] == algorithm) & (results_df['scenario'] == scenario)]

        if not base_result.empty and not current_result.empty:
            base_distance = base_result['distance_km'].values[0]
            current_distance = current_result['distance_km'].values[0]
            increase = ((current_distance - base_distance) / base_distance) * 100
            increase_data.append({
                'scenario': scenario,
                'algorithm': algorithm,
                'increase_percent': increase
            })

if increase_data:
    increase_df = pd.DataFrame(increase_data)

    # Группированный график увеличения
    scenarios_increase = increase_df['scenario'].unique()
    x_increase = np.arange(len(scenarios_increase))

    for i, algorithm in enumerate(algorithms):
        algorithm_increases = []
        for scenario in scenarios_increase:
            result = increase_df[(increase_df['algorithm'] == algorithm) & (increase_df['scenario'] == scenario)]
            if not result.empty:
                algorithm_increases.append(result['increase_percent'].values[0])
            else:
                algorithm_increases.append(0)

        ax3.bar(x_increase + i*width, algorithm_increases, width, label=algorithm, alpha=0.8)

    ax3.set_xlabel('Сценарии')
    ax3.set_ylabel('Увеличение расстояния (%)')
    ax3.set_title('Увеличение длины маршрута по сценариям\n(Относительно базового сценария)')
    ax3.set_xticks(x_increase + width*1.5)
    ax3.set_xticklabels(scenarios_increase, rotation=45)
    ax3.legend()
    ax3.grid(True, alpha=0.3)

# 4. Стабильность алгоритмов (стандартное отклонение по сценариям)
stability_data = []
for algorithm in algorithms:
    algorithm_results = results_df[results_df['algorithm'] == algorithm]
    if len(algorithm_results) > 0:
        mean_distance = algorithm_results['distance_km'].mean()
        std_distance = algorithm_results['distance_km'].std()
        cv_percent = (std_distance / mean_distance) * 100 if mean_distance > 0 else 0
        stability_data.append({
            'algorithm': algorithm,
            'mean_km': mean_distance,
            'std_km': std_distance,
            'cv_percent': cv_percent
        })

if stability_data:
    stability_df = pd.DataFrame(stability_data)
    bars4 = ax4.bar(stability_df['algorithm'], stability_df['cv_percent'],
                    color=['#ff9999', '#66b3ff', '#99ff99', '#ffcc99'], alpha=0.7)
    ax4.set_ylabel('Коэффициент вариации (%)')
    ax4.set_title('Стабильность алгоритмов\n(Меньше = стабильнее)')
    ax4.tick_params(axis='x', rotation=45)
    ax4.grid(True, alpha=0.3)

    for bar, cv in zip(bars4, stability_df['cv_percent']):
        height = bar.get_height()
        ax4.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                f'{cv:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.savefig('all_algorithms_noise_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("Визуализация сохранена: 'all_algorithms_noise_comparison.png'")

## Большее количество складов

In [ ]:
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import json
import warnings
from math import radians, sin, cos, sqrt, atan2
import itertools
from collections import defaultdict, Counter
warnings.filterwarnings('ignore')

In [ ]:
# Загрузка всех складов
# Загрузка дорожной сети
graph = ox.load_graphml("moscow_region_drive_network.graphml")
print("Дорожная сеть загружена")

# Загрузка всех складов из JSON файла
print("Загружаем ВСЕ склады из all_warehouses_coordinates.json...")
with open('all_warehouses_coordinates.json', 'r', encoding='utf-8') as f:
    all_warehouses_json = json.load(f)

# Создаем DataFrame со всеми складами
all_warehouses_data = []
for warehouse in all_warehouses_json:
    all_warehouses_data.append({
        "name": warehouse['name'],
        "latitude": warehouse['latitude'],
        "longitude": warehouse['longitude'],
        "type": "РЦ/РФЦ" if "РЦ" in warehouse['name'] or "РФЦ" in warehouse['name'] else "СЦ"
    })

all_warehouses_df = pd.DataFrame(all_warehouses_data)
START_WAREHOUSE = 0  # ПАВЛО_СЛОБОДСКОЕ_РЦ как начальная точка

print(f"Загружено всего складов: {len(all_warehouses_df)}")
print(f"Из них:")
print(f"Начальная точка: {all_warehouses_df.iloc[START_WAREHOUSE]['name']}")

In [ ]:
# Привязка всех складов к узлам графа
all_warehouse_nodes = []
connection_distances = []

for idx, warehouse in all_warehouses_df.iterrows():
    point = (warehouse['latitude'], warehouse['longitude'])
    nearest_node = ox.distance.nearest_nodes(graph, point[1], point[0])
    all_warehouse_nodes.append(nearest_node)

    node_data = graph.nodes[nearest_node]
    node_point = (node_data['y'], node_data['x'])
    distance = calculate_geodesic_distance(point[0], point[1], node_point[0], node_point[1])
    connection_distances.append(distance)

    if idx < 5:  # Показываем только первые 5 для примера
        print(f"   {warehouse['name']:30} -> Узел {nearest_node} ({distance:.1f} м)")

all_warehouses_df['node_id'] = all_warehouse_nodes
all_warehouses_df['distance_to_node'] = connection_distances

# Анализ качества подключения всех складов
print(f"\nАнализ качества подключения {len(all_warehouses_df)} складов:")
good_connections = sum(1 for d in connection_distances if d <= 200)
medium_connections = sum(1 for d in connection_distances if 200 < d <= 500)
poor_connections = sum(1 for d in connection_distances if d > 500)

print(f"  - Хорошие подключения (≤200 м): {good_connections} ({good_connections/len(connection_distances)*100:.1f}%)")
print(f"  - Средние подключения (200-500 м): {medium_connections} ({medium_connections/len(connection_distances)*100:.1f}%)")
print(f"  - Плохие подключения (>500 м): {poor_connections} ({poor_connections/len(connection_distances)*100:.1f}%)")

In [ ]:
distance_matrices = {}
computation_times = {}
accessibility_results = {}

print("Проверка доступности и создание матриц расстояний для всех сценариев:")
for scenario_name, scenario_graph in scenarios.items():
    print(f"\n{scenario_name}:")
    accessible = check_warehouses_accessibility(scenario_graph, all_warehouses_df, all_warehouse_nodes)
    accessibility_results[scenario_name] = accessible

    if accessible:
        dist_matrix, comp_time = calculate_distance_matrix_for_scenario(
            scenario_graph, all_warehouse_nodes, scenario_name
        )
        distance_matrices[scenario_name] = dist_matrix
        computation_times[scenario_name] = comp_time

In [ ]:
# запуск всех алгоритмов на всех сценариях
all_results = []

for scenario_name, dist_matrix in distance_matrices.items():
    print(f"\nСЦЕНАРИЙ: {scenario_name}")
    print("-" * 40)

    scenario_results = []

    # 1. Жадный алгоритм
    print("1. Жадный алгоритм")
    greedy_route, greedy_distance, greedy_time = greedy_tsp(dist_matrix, START_WAREHOUSE)
    scenario_results.append({
        'algorithm': 'Жадный',
        'route': greedy_route,
        'distance_km': greedy_distance / 1000,
        'time_sec': greedy_time,
        'scenario': scenario_name
    })
    print(f"   Результат: {greedy_distance/1000:.2f} км за {greedy_time:.3f} сек")

    # 2. 2-OPT (улучшение жадного)
    print("2. 2-OPT")
    opt2_route, opt2_distance, opt2_time = tsp_2opt(greedy_route, dist_matrix, max_iterations=500)
    scenario_results.append({
        'algorithm': '2-OPT',
        'route': opt2_route,
        'distance_km': opt2_distance / 1000,
        'time_sec': opt2_time,
        'scenario': scenario_name,
        'improvement': f"{(greedy_distance - opt2_distance) / greedy_distance * 100:.1f}%" if greedy_distance > 0 else "0.0%"
    })
    print(f"   Результат: {opt2_distance/1000:.2f} км за {opt2_time:.3f} сек")
    if opt2_distance < greedy_distance:
        print(f"   Улучшение: {scenario_results[-1]['improvement']}")

    # 3. MST
    print("3. MST")
    mst_route, mst_distance, mst_time = mst_tsp(dist_matrix, START_WAREHOUSE)
    scenario_results.append({
        'algorithm': 'MST',
        'route': mst_route,
        'distance_km': mst_distance / 1000,
        'time_sec': mst_time,
        'scenario': scenario_name
    })
    print(f"   Результат: {mst_distance/1000:.2f} км за {mst_time:.3f} сек")

    # 4. Генетический алгоритм (2 запуска для скорости)
    print("4. Генетический алгоритм")
    print("   Запускаем 2 раза для надежности")
    ga_runs = []
    for run in range(2):
        print(f"   Запуск {run+1}/2", end=" ")
        ga_route, ga_distance, ga_time = genetic_algorithm_tsp(
            dist_matrix, START_WAREHOUSE, pop_size=30, generations=50
        )
        ga_runs.append({
            'route': ga_route,
            'distance_km': ga_distance / 1000,
            'time_sec': ga_time
        })
        print(f"{ga_distance/1000:.2f} км")

    # Анализируем результаты генетического алгоритма
    ga_distances = [run['distance_km'] for run in ga_runs]
    best_ga_idx = np.argmin(ga_distances)
    best_ga = ga_runs[best_ga_idx]

    scenario_results.append({
        'algorithm': 'Генетический',
        'route': best_ga['route'],
        'distance_km': best_ga['distance_km'],
        'time_sec': np.mean([run['time_sec'] for run in ga_runs]),
        'scenario': scenario_name,
        'statistics': {
            'best_km': best_ga['distance_km'],
            'mean_km': np.mean(ga_distances),
            'std_km': np.std(ga_distances),
            'runs': len(ga_runs)
        }
    })
    print(f"   Лучший результат: {best_ga['distance_km']:.2f} км")
    print(f"   Статистика: среднее {np.mean(ga_distances):.2f} ± {np.std(ga_distances):.2f} км")

    all_results.extend(scenario_results)

In [ ]:
# Анализ результатов

# Создаем DataFrame для анализа
results_df = pd.DataFrame(all_results)
results_df

In [ ]:
print(f"Всего тестов: {len(results_df)}")
print(f"Количество сценариев: {len(results_df['scenario'].unique())}")
print(f"Количество алгоритмов: {len(results_df['algorithm'].unique())}")

# Сводная таблица по алгоритмам и сценариям
pivot_distance = results_df.pivot_table(
    index='scenario',
    columns='algorithm',
    values='distance_km',
    aggfunc='mean'
).round(2)

pivot_time = results_df.pivot_table(
    index='scenario',
    columns='algorithm',
    values='time_sec',
    aggfunc='mean'
).round(5)

print("\nСРЕДНЯЯ ДЛИНА МАРШРУТА ПО СЦЕНАРИЯМ (км):")
print(pivot_distance)

print("\nСРЕДНЕЕ ВРЕМЯ ВЫПОЛНЕНИЯ ПО СЦЕНАРИЯМ (сек):")
print(pivot_time)

In [ ]:
# Создаем комплексные графики
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(18, 12))

# График 1: Сравнение алгоритмов по сценариям (расстояния)
scenarios_list = list(distance_matrices.keys())
algorithms = ['Жадный', '2-OPT', 'MST', 'Генетический']

# Подготовка данных для группированного графика
x = np.arange(len(scenarios_list))
width = 0.2

for i, algorithm in enumerate(algorithms):
    algorithm_distances = []
    for scenario in scenarios_list:
        result = results_df[(results_df['algorithm'] == algorithm) & (results_df['scenario'] == scenario)]
        if not result.empty:
            algorithm_distances.append(result['distance_km'].values[0])
        else:
            algorithm_distances.append(0)

    ax1.bar(x + i*width, algorithm_distances, width, label=algorithm, alpha=0.8)

ax1.set_xlabel('Сценарии')
ax1.set_ylabel('Расстояние (км)')
ax1.set_title('Сравнение алгоритмов по сценариям\n(Длина маршрута)', fontweight='bold')
ax1.set_xticks(x + width*1.5)
ax1.set_xticklabels(scenarios_list, rotation=45)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Добавляем значения на столбцы
for i, scenario in enumerate(scenarios_list):
    for j, algorithm in enumerate(algorithms):
        result = results_df[(results_df['algorithm'] == algorithm) & (results_df['scenario'] == scenario)]
        if not result.empty:
            height = result['distance_km'].values[0]
            ax1.text(x[i] + j*width, height + 5, f'{height:.1f}',
                    ha='center', va='bottom', fontsize=8)

# График 2: Время выполнения по алгоритмам
algorithm_times = results_df.groupby('algorithm')['time_sec'].mean()
bars2 = ax2.bar(algorithm_times.index, algorithm_times.values,
                color=['#ff9999', '#66b3ff', '#99ff99', '#ffcc99'], alpha=0.8)
ax2.set_ylabel('Среднее время (сек)')
ax2.set_title('Среднее время выполнения алгоритмов\n(Все сценарии)', fontweight='bold')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3)

for bar, time_val in zip(bars2, algorithm_times.values):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.01,
            f'{time_val:.3f} сек', ha='center', va='bottom', fontsize=10)

# График 3: Увеличение расстояния по сценариям (относительно базового)
base_scenario = "Без шума"
base_results = results_df[results_df['scenario'] == base_scenario]

increase_data = []
for scenario in scenarios_list[1:]:  # Исключаем базовый сценарий
    for algorithm in algorithms:
        base_result = base_results[base_results['algorithm'] == algorithm]
        current_result = results_df[(results_df['algorithm'] == algorithm) & (results_df['scenario'] == scenario)]

        if not base_result.empty and not current_result.empty:
            base_distance = base_result['distance_km'].values[0]
            current_distance = current_result['distance_km'].values[0]
            increase = ((current_distance - base_distance) / base_distance) * 100
            increase_data.append({
                'scenario': scenario,
                'algorithm': algorithm,
                'increase_percent': increase
            })

if increase_data:
    increase_df = pd.DataFrame(increase_data)

    # Группированный график увеличения
    scenarios_increase = increase_df['scenario'].unique()
    x_increase = np.arange(len(scenarios_increase))

    for i, algorithm in enumerate(algorithms):
        algorithm_increases = []
        for scenario in scenarios_increase:
            result = increase_df[(increase_df['algorithm'] == algorithm) & (increase_df['scenario'] == scenario)]
            if not result.empty:
                algorithm_increases.append(result['increase_percent'].values[0])
            else:
                algorithm_increases.append(0)

        ax3.bar(x_increase + i*width, algorithm_increases, width, label=algorithm, alpha=0.8)

    ax3.set_xlabel('Сценарии')
    ax3.set_ylabel('Увеличение расстояния (%)')
    ax3.set_title('Увеличение длины маршрута по сценариям\n(Относительно базового сценария)', fontweight='bold')
    ax3.set_xticks(x_increase + width*1.5)
    ax3.set_xticklabels(scenarios_increase, rotation=45)
    ax3.legend()
    ax3.grid(True, alpha=0.3)

# График 4: Стабильность алгоритмов (стандартное отклонение по сценариям)
stability_data = []
for algorithm in algorithms:
    algorithm_results = results_df[results_df['algorithm'] == algorithm]
    if len(algorithm_results) > 0:
        mean_distance = algorithm_results['distance_km'].mean()
        std_distance = algorithm_results['distance_km'].std()
        cv_percent = (std_distance / mean_distance) * 100 if mean_distance > 0 else 0
        stability_data.append({
            'algorithm': algorithm,
            'mean_km': mean_distance,
            'std_km': std_distance,
            'cv_percent': cv_percent
        })

if stability_data:
    stability_df = pd.DataFrame(stability_data)
    bars4 = ax4.bar(stability_df['algorithm'], stability_df['cv_percent'],
                    color=['#ff9999', '#66b3ff', '#99ff99', '#ffcc99'], alpha=0.7)
    ax4.set_ylabel('Коэффициент вариации (%)')
    ax4.set_title('Стабильность алгоритмов\n(Меньше = стабильнее)', fontweight='bold')
    ax4.tick_params(axis='x', rotation=45)
    ax4.grid(True, alpha=0.3)

    for bar, cv in zip(bars4, stability_df['cv_percent']):
        height = bar.get_height()
        ax4.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                f'{cv:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.savefig('comprehensive_algorithms_comparison_all_scenarios.png', dpi=300, bbox_inches='tight')
plt.show()

print("Комплексная визуализация сохранена: 'comprehensive_algorithms_comparison_all_scenarios.png'")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import os
import networkx as nx
from math import radians, sin, cos, sqrt, atan2

# Создаем директорию для сохранения графиков
os.makedirs('road_route_visualizations', exist_ok=True)


In [ ]:
def get_path_between_warehouses(graph, warehouse1_node, warehouse2_node):
    """
    Находит кратчайший путь по дорожной сети между двумя складами
    Возвращает список координат (lat, lon) пути
    """
    try:
        # Находим кратчайший путь по узлам графа
        path_nodes = nx.shortest_path(graph, warehouse1_node, warehouse2_node, weight='length')

        # Собираем координаты всех узлов пути
        path_coords = []
        for node in path_nodes:
            node_data = graph.nodes[node]
            path_coords.append((node_data['y'], node_data['x']))  # (lat, lon)

        return path_coords
    except (nx.NetworkXNoPath, nx.NodeNotFound):
        # Если пути нет, возвращаем прямую линию
        node1_data = graph.nodes[warehouse1_node]
        node2_data = graph.nodes[warehouse2_node]
        return [
            (node1_data['y'], node1_data['x']),
            (node2_data['y'], node2_data['x'])
        ]

def get_full_route_path(graph, route_indices, warehouse_nodes):
    """
    Получает полный маршрут по дорожной сети для заданной последовательности складов

    Args:
        graph: дорожный граф
        route_indices: список индексов складов в порядке посещения
        warehouse_nodes: список узлов графа, соответствующих складам

    Returns:
        list: список координат (lat, lon) всего маршрута
    """
    full_path = []

    for i in range(len(route_indices) - 1):
        # Получаем узлы графа для текущего и следующего склада
        from_idx = route_indices[i]
        to_idx = route_indices[i + 1]

        from_node = warehouse_nodes[from_idx]
        to_node = warehouse_nodes[to_idx]

        # Получаем путь между этими складами
        segment_path = get_path_between_warehouses(graph, from_node, to_node)

        # Добавляем сегмент к общему пути (исключая последнюю точку, чтобы не дублировать)
        if full_path:
            full_path.extend(segment_path[1:])  # исключаем первую точку (она уже есть)
        else:
            full_path.extend(segment_path)

    return full_path

def visualize_route_on_roads(graph, route_indices, warehouse_nodes, warehouses_df,
                           scenario_name, algorithm_name, distance_km=None):
    """
    Визуализирует маршрут по реальным дорогам на карте

    Args:
        graph: дорожный граф
        route_indices: список индексов складов
        warehouse_nodes: список узлов графа для складов
        warehouses_df: DataFrame со складами
        scenario_name: название сценария
        algorithm_name: название алгоритма
        distance_km: расстояние маршрута (опционально)
    """
    fig, ax = plt.subplots(figsize=(15, 12))

    # 1. Получаем полный путь по дорогам
    route_path = get_full_route_path(graph, route_indices, warehouse_nodes)

    # 2. Преобразуем координаты для отрисовки
    route_lats = [coord[0] for coord in route_path]
    route_lons = [coord[1] for coord in route_path]

    # 3. Рисуем дорожную сеть (упрощенную версию)
    # Получаем ограничивающую рамку для маршрута
    min_lat, max_lat = min(route_lats), max(route_lats)
    min_lon, max_lon = min(route_lons), max(route_lons)

    # Добавляем отступ
    lat_padding = (max_lat - min_lat) * 0.2
    lon_padding = (max_lon - min_lon) * 0.2

    bbox = (min_lat - lat_padding, max_lat + lat_padding,
            min_lon - lon_padding, max_lon + lon_padding)

    # Извлекаем подграф для этой области
    nodes_in_bbox = []
    for node, data in graph.nodes(data=True):
        if (bbox[0] <= data['y'] <= bbox[1] and
            bbox[2] <= data['x'] <= bbox[3]):
            nodes_in_bbox.append(node)

    subgraph = graph.subgraph(nodes_in_bbox)

    # Рисуем дороги (тонкие серые линии)
    for u, v, data in subgraph.edges(data=True):
        if 'geometry' in data:
            # Если есть геометрия линии, рисуем ее
            line_coords = list(data['geometry'].coords)
            x_coords = [coord[0] for coord in line_coords]
            y_coords = [coord[1] for coord in line_coords]
            ax.plot(x_coords, y_coords, 'lightgray', linewidth=0.5, alpha=0.5, zorder=1)
        else:
            # Иначе рисуем прямую линию между узлами
            u_data = subgraph.nodes[u]
            v_data = subgraph.nodes[v]
            ax.plot([u_data['x'], v_data['x']], [u_data['y'], v_data['y']],
                   'lightgray', linewidth=0.5, alpha=0.5, zorder=1)

    # 4. Рисуем маршрут (толстая цветная линия)
    ax.plot(route_lons, route_lats, 'b-', linewidth=3, alpha=0.8, zorder=3, label='Маршрут')

    # Добавляем стрелки направления каждые N точек
    arrow_step = max(1, len(route_path) // 10)
    for i in range(0, len(route_path) - 1, arrow_step):
        if i + 1 < len(route_path):
            start_lon, start_lat = route_lons[i], route_lats[i]
            end_lon, end_lat = route_lons[i + 1], route_lats[i + 1]

            # Пропускаем слишком короткие сегменты
            if calculate_geodesic_distance(start_lat, start_lon, end_lat, end_lon) > 1000:
                ax.annotate('', xy=(end_lon, end_lat), xytext=(start_lon, start_lat),
                           arrowprops=dict(arrowstyle='->', color='red', lw=2, alpha=0.7))

    # 5. Рисуем склады
    colors = plt.cm.Set1(np.linspace(0, 1, len(route_indices)))

    for i, idx in enumerate(route_indices):
        if i == 0:  # Начальный склад
            color = 'red'
            size = 200
            marker = '*'
            label = 'Начальная точка'
            zorder = 5
        elif i == len(route_indices) - 1:  # Конечный склад (возврат)
            continue  # Не рисуем, это тот же что и начальный
        else:
            color = colors[i % len(colors)]
            size = 100
            marker = 'o'
            label = None
            zorder = 4

        warehouse = warehouses_df.iloc[idx]
        ax.scatter(warehouse['longitude'], warehouse['latitude'],
                  c=color, s=size, marker=marker, edgecolors='black',
                  linewidth=2, zorder=zorder, label=label if i == 0 else None)

        # Подписываем первые 10 складов
        if i < 10:
            ax.annotate(warehouse['name'],
                       (warehouse['longitude'], warehouse['latitude']),
                       xytext=(5, 5), textcoords='offset points',
                       fontsize=9, fontweight='bold',
                       bbox=dict(boxstyle="round,pad=0.3",
                                facecolor="white", alpha=0.8))

    # 6. Настраиваем график
    title = f'{algorithm_name} - {scenario_name}'
    if distance_km:
        title += f'\nОбщее расстояние: {distance_km:.2f} км'

    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_xlabel('Долгота')
    ax.set_ylabel('Широта')
    ax.grid(True, alpha=0.3)
    ax.legend(loc='upper right')

    # Устанавливаем равные масштабы по осям
    ax.set_aspect('equal', adjustable='datalim')

    # Сохраняем график
    filename = f'road_route_{algorithm_name}_{scenario_name}'.replace(' ', '_').replace('+', '_')
    plt.tight_layout()
    plt.savefig(f'road_route_visualizations/{filename}.png', dpi=300, bbox_inches='tight')
    plt.show()

    return fig

In [ ]:
print("Визуализация всех алгоритмов на всех сценариях (4x4 сетка по дорогам)")

# Создаем большую фигуру
fig3, axes = plt.subplots(4, 4, figsize=(25, 20))

scenarios_dict = {
    'Без шума': graph,
    'Пробки': graph_traffic_jam,
    'Блокировки+Пробки': graph_combined,
    'Умеренные условия': graph_moderate
}

algorithms = ['Жадный', '2-OPT', 'MST', 'Генетический']
scenarios_list = list(scenarios_dict.keys())

# Преобразуем all_results в DataFrame для удобства поиска
all_results_df = pd.DataFrame(all_results)

# Цвета для алгоритмов
algorithm_colors = {
    'Жадный': 'blue',
    '2-OPT': 'orange',
    'MST': 'green',
    'Генетический': 'purple'
}

for algo_idx, algorithm in enumerate(algorithms):
    for scenario_idx, scenario_name in enumerate(scenarios_list):
        ax = axes[algo_idx, scenario_idx]
        scenario_graph = scenarios_dict[scenario_name]

        # Получаем результат - ИСПРАВЛЕННЫЙ СПОСОБ
        result_filtered = all_results_df[
            (all_results_df['algorithm'] == algorithm) &
            (all_results_df['scenario'] == scenario_name)
        ]

        if not result_filtered.empty:
            result_data = result_filtered.iloc[0]
            route_indices = result_data['route']
            distance_km = result_data['distance_km']

            # Определяем, какие узлы использовать (в зависимости от того, какой набор складов)
            # Проверяем, доступен ли all_warehouse_nodes
            if 'all_warehouse_nodes' in locals() or 'all_warehouse_nodes' in globals():
                current_nodes = all_warehouse_nodes
                current_df = all_warehouses_df
            else:
                current_nodes = warehouse_nodes
                current_df = warehouses_df

            # Получаем путь по дорогам
            route_path = get_full_route_path(scenario_graph, route_indices, current_nodes)

            if route_path and len(route_path) > 1:
                route_lats = [coord[0] for coord in route_path]
                route_lons = [coord[1] for coord in route_path]

                # Определяем bounding box
                min_lat, max_lat = min(route_lats), max(route_lats)
                min_lon, max_lon = min(route_lons), max(route_lons)

                # Рисуем упрощенную дорожную сеть
                lat_padding = (max_lat - min_lat) * 0.4
                lon_padding = (max_lon - min_lon) * 0.4

                bbox = (min_lat - lat_padding, max_lat + lat_padding,
                        min_lon - lon_padding, max_lon + lon_padding)

                # Отбираем и рисуем дороги в bounding box
                roads_drawn = 0
                for u, v, data in scenario_graph.edges(data=True):
                    if roads_drawn > 500:  # Ограничиваем количество дорог для производительности
                        break

                    u_data = scenario_graph.nodes[u]
                    v_data = scenario_graph.nodes[v]

                    # Проверяем, попадает ли дорога в bounding box
                    u_in_bbox = (bbox[0] <= u_data['y'] <= bbox[1] and
                                bbox[2] <= u_data['x'] <= bbox[3])
                    v_in_bbox = (bbox[0] <= v_data['y'] <= bbox[1] and
                                bbox[2] <= v_data['x'] <= bbox[3])

                    if u_in_bbox or v_in_bbox:
                        if 'geometry' in data:
                            line_coords = list(data['geometry'].coords)
                            x_coords = [coord[0] for coord in line_coords]
                            y_coords = [coord[1] for coord in line_coords]
                            ax.plot(x_coords, y_coords, 'lightgray',
                                   linewidth=0.2, alpha=0.2)
                        else:
                            ax.plot([u_data['x'], v_data['x']],
                                   [u_data['y'], v_data['y']],
                                   'lightgray', linewidth=0.2, alpha=0.2)

                        roads_drawn += 1

                # Рисуем маршрут
                ax.plot(route_lons, route_lats,
                       color=algorithm_colors.get(algorithm, 'blue'),
                       linewidth=2, alpha=0.8)

                # Рисуем начальный склад
                if route_indices and len(route_indices) > 0 and route_indices[0] < len(current_df):
                    start_idx = route_indices[0]
                    start_warehouse = current_df.iloc[start_idx]
                    ax.scatter(start_warehouse['longitude'], start_warehouse['latitude'],
                              c='red', s=60, marker='*', edgecolors='black',
                              linewidth=1.5, zorder=5)

                # Рисуем несколько промежуточных складов (первые 5)
                if len(route_indices) > 1:
                    for i in range(1, min(6, len(route_indices) - 1)):
                        idx = route_indices[i]
                        if idx < len(current_df):
                            warehouse = current_df.iloc[idx]
                            ax.scatter(warehouse['longitude'], warehouse['latitude'],
                                      c='green', s=40, marker='o', edgecolors='black',
                                      linewidth=1, zorder=4)

            ax.set_title(f'{algorithm}\n{scenario_name}\n{distance_km:.1f} км',
                        fontsize=10, fontweight='bold')
            ax.grid(True, alpha=0.1)
            ax.set_aspect('equal', adjustable='datalim')

            # Убираем подписи осей для внутренних графиков
            if algo_idx < 3:
                ax.set_xticklabels([])
                ax.set_xlabel('')
            if scenario_idx > 0:
                ax.set_yticklabels([])
                ax.set_ylabel('')
        else:
            ax.text(0.5, 0.5, 'Нет данных',
                   ha='center', va='center', transform=ax.transAxes)
            ax.set_title(f'{algorithm}: {scenario_name}', fontsize=10)

# Добавляем общие метки
fig3.text(0.5, 0.04, 'Сценарии', ha='center', va='center',
         fontsize=14, fontweight='bold')
fig3.text(0.04, 0.5, 'Алгоритмы', ha='center', va='center',
         rotation='vertical', fontsize=14, fontweight='bold')

# Добавляем легенду
legend_patches = [mpatches.Patch(color=color, label=algo)
                 for algo, color in algorithm_colors.items()]
legend_patches.append(mpatches.Patch(color='red', label='Начальная точка'))
fig3.legend(handles=legend_patches, loc='upper right',
           bbox_to_anchor=(0.98, 0.98), fontsize=11)

plt.tight_layout(rect=[0.05, 0.05, 0.95, 0.95])
plt.savefig('road_route_visualizations/all_algorithms_all_scenarios_grid_roads.png',
           dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
print("Сводная таблица результатов всех алгоритмов:")

# Создаем таблицу
summary_data = []
for scenario in scenarios_list:
    for algorithm in algorithms:
        result_filtered = all_results_df[
            (all_results_df['algorithm'] == algorithm) &
            (all_results_df['scenario'] == scenario)
        ]

        if not result_filtered.empty:
            result_data = result_filtered.iloc[0]
            summary_data.append({
                'Сценарий': scenario,
                'Алгоритм': algorithm,
                'Расстояние (км)': result_data['distance_km'],
                'Время (сек)': result_data['time_sec']
            })

summary_df = pd.DataFrame(summary_data)

# Выводим таблицу
for scenario in scenarios_list:
    print(f"\n{scenario}:")
    print("-" * 40)
    scenario_data = summary_df[summary_df['Сценарий'] == scenario]

    for _, row in scenario_data.iterrows():
        print(f"  {row['Алгоритм']:20} | {row['Расстояние (км)']:8.2f} км | {row['Время (сек)']:8.4f} сек")

# Находим лучшие результаты для каждого сценария
print("Лучшие результаты по сценариям:")
print("="*80)
for scenario in scenarios_list:
    scenario_data = summary_df[summary_df['Сценарий'] == scenario]
    if not scenario_data.empty:
        best_row = scenario_data.loc[scenario_data['Расстояние (км)'].idxmin()]
        print(f"\n{scenario}:")
        print(f"  Лучший алгоритм: {best_row['Алгоритм']}")
        print(f"  Расстояние: {best_row['Расстояние (км)']:.2f} км")
        print(f"  Время: {best_row['Время (сек)']:.4f} сек")

# Сохраняем таблицу в файл
summary_df.to_csv('road_route_visualizations/summary_results.csv', index=False)
print(f"\nТаблица сохранена в файл: 'road_route_visualizations/summary_results.csv'")

In [ ]:
print("Визуализация лучшего маршрута для каждого сценария")

# Создаем фигуру 2x2 для лучших маршрутов
fig4, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

def route_indices_to_nodes(route_indices, warehouse_nodes):
    """Преобразует индексы складов в узлы графа"""
    return [warehouse_nodes[i] for i in route_indices]

for idx, scenario_name in enumerate(scenarios_list):
    ax = axes[idx]
    scenario_graph = scenarios_dict[scenario_name]

    # Находим лучший алгоритм для этого сценария (минимальное расстояние)
    scenario_data = summary_df[summary_df['Сценарий'] == scenario_name]

    if not scenario_data.empty:
        best_row = scenario_data.loc[scenario_data['Расстояние (км)'].idxmin()]
        best_algorithm = best_row['Алгоритм']
        best_distance = best_row['Расстояние (км)']

        # Получаем маршрут
        result_filtered = all_results_df[
            (all_results_df['algorithm'] == best_algorithm) &
            (all_results_df['scenario'] == scenario_name)
        ]

        if not result_filtered.empty:
            result_data = result_filtered.iloc[0]
            route_indices = result_data['route']

            # Определяем узлы графа
            if 'all_warehouse_nodes' in locals() or 'all_warehouse_nodes' in globals():
                current_nodes = all_warehouse_nodes
                current_df = all_warehouses_df
            else:
                current_nodes = warehouse_nodes
                current_df = warehouses_df

            # Преобразуем индексы в узлы графа
            route_nodes = route_indices_to_nodes(route_indices, current_nodes)

            # Получаем полный путь по узлам графа
            full_path = []
            for i in range(len(route_nodes) - 1):
                try:
                    # Находим кратчайший путь между узлами
                    segment_path = nx.shortest_path(scenario_graph, route_nodes[i], route_nodes[i+1], weight='length')
                    # Добавляем все узлы сегмента, кроме последнего (чтобы не дублировать)
                    full_path.extend(segment_path[:-1])
                except (nx.NetworkXNoPath, nx.NodeNotFound) as e:
                    print(f"Предупреждение для {scenario_name}: не удалось найти путь между {route_nodes[i]} и {route_nodes[i+1]}: {e}")
                    # Если пути нет, добавляем только начальный и конечный узел
                    if not full_path:
                        full_path.append(route_nodes[i])
                    full_path.append(route_nodes[i+1])

            # Добавляем последний узел
            if route_nodes:
                full_path.append(route_nodes[-1])

            # Визуализируем
            if full_path:
                # Рисуем дорожную сеть
                ox.plot_graph(
                    scenario_graph,
                    ax=ax,
                    node_size=0,
                    edge_linewidth=0.3,
                    edge_color='lightgray',
                    bgcolor='white',
                    show=False,
                    close=False
                )

                # Рисуем маршрут
                ox.plot_graph_route(
                    scenario_graph,
                    full_path,
                    route_linewidth=4,
                    route_color='red',
                    route_alpha=0.8,
                    orig_dest_size=0,
                    ax=ax,
                    show=False,
                    close=False
                )

                # Рисуем склады
                for i, (idx_point, warehouse) in enumerate(current_df.iterrows()):
                    # Проверяем, что индекс в пределах маршрута
                    color = 'blue'
                    size = 150
                    alpha = 0.9

                    # Выделяем склады, которые есть в маршруте
                    if idx_point in route_indices:
                        position_in_route = route_indices.index(idx_point)
                        if position_in_route == 0:  # Начальный склад
                            color = 'red'
                            size = 200
                        else:
                            color = 'green'
                            size = 180

                    ax.scatter(
                        warehouse['longitude'],
                        warehouse['latitude'],
                        c=color,
                        s=size,
                        alpha=alpha,
                        edgecolors='black',
                        linewidth=1.5,
                        zorder=5
                    )

                # Подписываем начальный склад
                if route_indices and route_indices[0] < len(current_df):
                    start_warehouse = current_df.iloc[route_indices[0]]
                    ax.annotate(
                        start_warehouse['name'][:15] + '...' if len(start_warehouse['name']) > 15 else start_warehouse['name'],
                        (start_warehouse['longitude'], start_warehouse['latitude']),
                        xytext=(8, 8),
                        textcoords='offset points',
                        fontsize=9,
                        fontweight='bold',
                        bbox=dict(boxstyle="round,pad=0.2", facecolor='white', alpha=0.8)
                    )

        # Настраиваем график
        ax.set_title(f'{scenario_name}\nЛучший алгоритм: {best_algorithm}\nРасстояние: {best_distance:.1f} км',
                    fontsize=12, fontweight='bold')
        ax.set_xlabel('Долгота')
        ax.set_ylabel('Широта')

        # Добавляем легенду только для первого графика
        if idx == 0:
            from matplotlib.lines import Line2D
            legend_elements = [
                Line2D([0], [0], color='lightgray', lw=2, label='Дороги'),
                Line2D([0], [0], color='red', lw=4, label='Маршрут'),
                Line2D([0], [0], marker='o', color='w', label='Начальный склад',
                      markerfacecolor='red', markersize=10, markeredgecolor='black'),
                Line2D([0], [0], marker='o', color='w', label='Склады в маршруте',
                      markerfacecolor='green', markersize=10, markeredgecolor='black'),
                Line2D([0], [0], marker='o', color='w', label='Остальные склады',
                      markerfacecolor='blue', markersize=10, markeredgecolor='black')
            ]
            ax.legend(handles=legend_elements, loc='upper right', fontsize=9)

plt.suptitle('Лучшие маршруты для каждого сценария',
            fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('road_route_visualizations/best_routes_all_scenarios.png',
           dpi=300, bbox_inches='tight')
plt.show()



In [ ]:
print("\nВизуализация каждого алгоритма по отдельности")

algorithms = ['Жадный', '2-OPT', 'MST', 'Генетический']

for algorithm in algorithms:
    print(f"  Алгоритм: {algorithm}")

    # Создаем фигуру для этого алгоритма на всех сценариях
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()

    for idx, (scenario_name, scenario_graph) in enumerate(scenarios_dict.items()):
        ax = axes[idx]

        # Получаем результат
        result_filtered = all_results_df[
            (all_results_df['algorithm'] == algorithm) &
            (all_results_df['scenario'] == scenario_name)
        ]

        if not result_filtered.empty:
            result_data = result_filtered.iloc[0]
            route_indices = result_data['route']
            distance_km = result_data['distance_km']

            # Определяем узлы
            if 'all_warehouse_nodes' in locals() or 'all_warehouse_nodes' in globals():
                current_nodes = all_warehouse_nodes
                current_df = all_warehouses_df
            else:
                current_nodes = warehouse_nodes
                current_df = warehouses_df

            # Преобразуем индексы в узлы
            route_nodes = route_indices_to_nodes(route_indices, current_nodes)

            # Получаем полный путь
            full_path = []
            for i in range(len(route_nodes) - 1):
                try:
                    segment_path = nx.shortest_path(scenario_graph, route_nodes[i], route_nodes[i+1], weight='length')
                    full_path.extend(segment_path[:-1])
                except (nx.NetworkXNoPath, nx.NodeNotFound):
                    if not full_path:
                        full_path.append(route_nodes[i])
                    full_path.append(route_nodes[i+1])

            if route_nodes:
                full_path.append(route_nodes[-1])

            # Визуализируем
            if full_path:
                # Дорожная сеть
                ox.plot_graph(
                    scenario_graph,
                    ax=ax,
                    node_size=0,
                    edge_linewidth=0.3,
                    edge_color='lightgray',
                    bgcolor='white',
                    show=False,
                    close=False
                )

                # Маршрут
                ox.plot_graph_route(
                    scenario_graph,
                    full_path,
                    route_linewidth=4,
                    route_color='blue',
                    route_alpha=0.8,
                    orig_dest_size=0,
                    ax=ax,
                    show=False,
                    close=False
                )

                # Склады
                for i, (idx_point, warehouse) in enumerate(current_df.iterrows()):
                    color = 'gray'
                    size = 100

                    if idx_point in route_indices:
                        position = route_indices.index(idx_point)
                        if position == 0:
                            color = 'red'
                            size = 200
                        elif position == len(route_indices) - 1:
                            color = 'darkred'
                            size = 150
                        else:
                            color = 'green'
                            size = 120

                    ax.scatter(
                        warehouse['longitude'],
                        warehouse['latitude'],
                        c=color,
                        s=size,
                        alpha=0.9,
                        edgecolors='black',
                        linewidth=1,
                        zorder=5
                    )

                # Подпись начального склада
                if route_indices and route_indices[0] < len(current_df):
                    start_warehouse = current_df.iloc[route_indices[0]]
                    ax.annotate(
                        'Начало',
                        (start_warehouse['longitude'], start_warehouse['latitude']),
                        xytext=(5, 5),
                        textcoords='offset points',
                        fontsize=8,
                        fontweight='bold',
                        bbox=dict(boxstyle="round,pad=0.2", facecolor='white', alpha=0.8)
                    )

        ax.set_title(f'{scenario_name}\n{distance_km:.1f} км',
                    fontsize=11, fontweight='bold')
        ax.set_xlabel('')
        ax.set_ylabel('')

    plt.suptitle(f'Алгоритм: {algorithm}\nМаршруты на разных сценариях',
                fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()

    filename = f'road_route_{algorithm}_all_scenarios'.replace(' ', '_')
    plt.savefig(f'road_route_visualizations/{filename}.png',
               dpi=300, bbox_inches='tight')
    plt.show()

# RL

## Базовые методы

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from collections import deque
import matplotlib.pyplot as plt
import pandas as pd
import json
import osmnx as ox
import networkx as nx
import time

In [ ]:
# Загрузка данных
def load_data():
    # Загружаем дорожную сеть
    graph = ox.load_graphml("moscow_region_drive_network.graphml")

    # Загружаем склады из JSON
    with open('warehouses_rc_rfc_coordinates.json', 'r', encoding='utf-8') as f:
        warehouses_json = json.load(f)

    # Создаем DataFrame
    warehouses_data = []
    for warehouse in warehouses_json:
        warehouses_data.append({
            "name": warehouse['name'],
            "latitude": warehouse['latitude'],
            "longitude": warehouse['longitude']
        })

    warehouses_df = pd.DataFrame(warehouses_data)

    # Привязываем склады к узлам графа
    warehouse_nodes = []
    for idx, warehouse in warehouses_df.iterrows():
        point = (warehouse['latitude'], warehouse['longitude'])
        nearest_node = ox.distance.nearest_nodes(graph, point[1], point[0])
        warehouse_nodes.append(nearest_node)

    warehouses_df['node_id'] = warehouse_nodes

    # Создаем матрицу расстояний между складами
    n_warehouses = len(warehouse_nodes)
    distance_matrix = np.zeros((n_warehouses, n_warehouses))

    for i in range(n_warehouses):
        for j in range(i+1, n_warehouses):
            try:
                distance = nx.shortest_path_length(graph, warehouse_nodes[i], warehouse_nodes[j], weight='length')
                distance_matrix[i][j] = distance
                distance_matrix[j][i] = distance
            except nx.NetworkXNoPath:
                distance_matrix[i][j] = 1e9
                distance_matrix[j][i] = 1e9

    # Конвертируем в float32 для PyTorch
    distance_matrix = distance_matrix.astype(np.float32)

    return distance_matrix, warehouses_df

In [ ]:
# Базовая инфраструктура
class TSPEnvironment:
    """
    Класс среды
    """

    def __init__(self, distance_matrix, warehouses_df):
        self.distance_matrix = distance_matrix # матрица попарных расстояний между складами (в метрах)
        self.warehouses_df = warehouses_df     # DataFrame с информацией о складах (координаты, названия)
        self.n_warehouses = len(warehouses_df)
        self.reset()

    def reset(self):
        """
        Сброс среды в начальное состояние
        """
        self.current_warehouse = 0  # Начальный склад
        self.visited = set([0])     # Уже посетили начальный склад
        self.route = [0]            # Маршрут начинается с начального склада
        self.total_distance = 0     # Общая пройденная дистанция
        self.done = False           # Флаг завершения эпизода
        return self._get_state()

    def _get_state(self):
        """
        Кодирование состояния в формат для нейросети
        """
        visited_mask = np.zeros(self.n_warehouses, dtype=np.float32)
        for i in range(self.n_warehouses):
            visited_mask[i] = 1.0 if i in self.visited else 0.0

        return {
            'current': self.current_warehouse,  # индекс текущего склада
            'visited_mask': visited_mask        #бинарный вектор длины n_warehouses (1 = склад посещен, 0 = не посещен)
        }

    def step(self, action):
        """
        Выполнение одного шага в среде

        Правила:
        1. Если действие - посещение уже посещенного склада (кроме случая завершения) -> штраф -1000
        2. Иначе: награда = отрицательное расстояние до следующего склада
           (чем дальше едем, тем хуже)
        3. При завершении маршрута (все склады посещены):
           - добавляем расстояние возврата в начальную точку
           - даем бонус обратно пропорциональный общей дистанции
             1000/(total_distance+1e-6) - поощряем короткие маршруты
        """
        if self.done:
            return self._get_state(), 0.0, True, {}

        if action in self.visited and len(self.visited) < self.n_warehouses:
            # Штраф за повторное посещение незавершенного маршрута
            reward = -1000.0
            self.done = True
        else:
            # Основная награда: отрицательное пройденное расстояние
            reward = -float(self.distance_matrix[self.current_warehouse][action])
            self.total_distance += float(self.distance_matrix[self.current_warehouse][action])

            # Обновление состояния
            self.current_warehouse = action
            self.visited.add(action)
            self.route.append(action)

            # Проверка завершения (все склады посещены)
            self.done = len(self.visited) == self.n_warehouses

            if self.done:
                # Добавляем расстояние возврата в начальную точку
                return_distance = float(self.distance_matrix[action][0])
                reward -= return_distance
                self.total_distance += return_distance
                self.route.append(0)

                # Бонус за завершение: поощряем короткие маршруты
                # 1e-6 для избежания деления на 0
                reward += 1000.0 / (self.total_distance + 1e-6)

        return self._get_state(), float(reward), self.done, {}

    def get_route_info(self):
        """Возвращает информацию о текущем маршруте"""
        return {
            'route': self.route,
            'total_distance': self.total_distance,
            'distance_km': self.total_distance / 1000.0  # Конвертация в км
        }

In [ ]:
# Нейросетевые архитектуры
class PolicyNetwork(nn.Module):
    """
    сеть для Policy Gradient методов

    Архитектура: простая полносвязная сеть с 2 скрытыми слоями
    Размеры: вход = n_warehouses * 2, выход = n_warehouses

    Входные данные:
    - One-hot encoding текущего склада (n_warehouses)
    - Маска посещенных складов (n_warehouses)
    Всего: 2 * n_warehouses признаков

    Параметры:
    - hidden_size=128: стандартный размер для простых задач
    """

    def __init__(self, n_warehouses, hidden_size=128):
        super(PolicyNetwork, self).__init__()
        self.n_warehouses = n_warehouses

        self.network = nn.Sequential(
            nn.Linear(n_warehouses * 2, hidden_size),  # Входной слой
            nn.ReLU(),  # Активация для нелинейности
            nn.Linear(hidden_size, hidden_size),  # Скрытый слой
            nn.ReLU(),
            nn.Linear(hidden_size, n_warehouses)  # Выходной слой
        )

    def forward(self, state):
        """Прямой проход через сеть"""
        current = state['current']
        visited_mask = state['visited_mask']

        # One-hot кодирование текущего положения
        current_onehot = torch.zeros(current.shape[0], self.n_warehouses, dtype=torch.float32)
        for i, c in enumerate(current):
            current_onehot[i, c] = 1.0

        # Конкатенация one-hot вектора и маски посещенных
        x = torch.cat([current_onehot, visited_mask], dim=1)

        # Проход через сеть
        logits = self.network(x)

        # Маскирование уже посещенных складов (большое отрицательное число)
        mask = visited_mask.bool()
        logits[mask] = -1e9  # Делаем их невыбираемыми

        # Softmax для получения вероятностей действий
        return torch.softmax(logits, dim=-1)


class QNetwork(nn.Module):
    """
    Q-сеть для DQN методов
    - Выход: Q-значения для каждого действия
    - Используется для оценки "ценности" действий
    """

    def __init__(self, n_warehouses, hidden_size=128):
        super(QNetwork, self).__init__()
        self.n_warehouses = n_warehouses

        self.network = nn.Sequential(
            nn.Linear(n_warehouses * 2, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_warehouses)  # Q-значения для каждого склада
        )

    def forward(self, state):
        """Возвращает Q-значения для всех возможных действий"""
        current = state['current']
        visited_mask = state['visited_mask']

        current_onehot = torch.zeros(current.shape[0], self.n_warehouses, dtype=torch.float32)
        for i, c in enumerate(current):
            current_onehot[i, c] = 1.0

        x = torch.cat([current_onehot, visited_mask], dim=1)

        q_values = self.network(x)
        q_values[visited_mask.bool()] = -1e9  # Маскирование посещенных

        return q_values


class ActorCriticNetwork(nn.Module):
    """
    Сеть для Actor-Critic методов (A2C, PPO)
    Содержит:
    - Общий энкодер для извлечения признаков
    - Два "головы":
      1. Actor: вероятности действий
      2. Critic: оценка ценности состояния

    Архитектура разделена для стабильности обучения
    """

    def __init__(self, n_warehouses, hidden_size=128):
        super(ActorCriticNetwork, self).__init__()
        self.n_warehouses = n_warehouses

        # Общий энкодер (shared feature extractor)
        self.encoder = nn.Sequential(
            nn.Linear(n_warehouses * 2, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU()
        )

        # Actor head: политика (распределение действий)
        self.actor = nn.Linear(hidden_size, n_warehouses)

        # Critic head: ценность состояния (скаляр)
        self.critic = nn.Linear(hidden_size, 1)

    def forward(self, state):
        """Возвращает и политику, и ценность состояния"""
        current = state['current']
        visited_mask = state['visited_mask']

        # Подготовка входных данных
        current_onehot = torch.zeros(current.shape[0], self.n_warehouses, dtype=torch.float32)
        for i, c in enumerate(current):
            current_onehot[i, c] = 1.0

        x = torch.cat([current_onehot, visited_mask], dim=1)
        features = self.encoder(x)  # Общие признаки

        # Actor: вероятности действий
        logits = self.actor(features)
        logits[visited_mask.bool()] = -1e9  # Маскирование
        action_probs = torch.softmax(logits, dim=-1)

        # Critic: ценность состояния
        state_value = self.critic(features)

        return action_probs, state_value

Агенты RL

In [ ]:
class PolicyGradientAgent:
    """
    Агент Policy Gradient

    Алгоритм:
    1. Собирает траектории (действия, логи вероятностей, награды)
    2. Вычисляет return (сумму наград с discount factor)
    3. Обновляет политику в направлении увеличения награды

    Гиперпараметры:
    - learning_rate=0.001: стандартное значение для Adam
    - discount_factor=0.99: учитывает будущие награды
    """

    def __init__(self, n_warehouses, learning_rate=0.001):
        self.policy_network = PolicyNetwork(n_warehouses)
        self.optimizer = optim.Adam(self.policy_network.parameters(), lr=learning_rate)
        self.saved_log_probs = []  # Логи вероятностей выбранных действий
        self.rewards = []  # Награды за шаги

    def select_action(self, state):
        """Выбор действия на основе текущей политики"""
        # Конвертация состояния в тензоры
        state_tensor = {
            'current': torch.tensor([state['current']], dtype=torch.long),
            'visited_mask': torch.tensor([state['visited_mask']], dtype=torch.float32)
        }

        # Получение вероятностей действий
        probs = self.policy_network(state_tensor)

        # Сэмплирование действия из распределения
        m = torch.distributions.Categorical(probs)
        action = m.sample()

        # Сохранение лога вероятности для обновления
        self.saved_log_probs.append(m.log_prob(action))
        return action.item()

    def update(self):
        """Обновление политики после завершения эпизода"""
        if not self.rewards:
            return

        # Вычисление дисконтированных returns (G_t)
        R = 0
        policy_loss = []
        returns = []

        # Обратный проход по наградам для расчета returns
        for r in self.rewards[::-1]:
            R = r + 0.99 * R  # discount factor γ=0.99
            returns.insert(0, R)

        # Нормализация returns для стабильности
        returns = torch.tensor(returns, dtype=torch.float32)
        returns = (returns - returns.mean()) / (returns.std() + 1e-9)

        # Вычисление потерь: -log(π(a|s)) * G_t
        for log_prob, R in zip(self.saved_log_probs, returns):
            policy_loss.append(-log_prob * R)

        # Градиентный спуск
        self.optimizer.zero_grad()
        policy_loss = torch.stack(policy_loss).sum()
        policy_loss.backward()
        self.optimizer.step()

        # Очистка буферов для следующего эпизода
        self.saved_log_probs = []
        self.rewards = []

In [ ]:
class A2CAgent:
    """
    Advantage Actor-Critic агент

    Использует Critic для оценки advantage (A = Q - V)
    Одновременно обучение Actor и Critic

    Гиперпараметры:
    - learning_rate=0.001
    - gamma=0.99: коэффициент дисконтирования
    """

    def __init__(self, n_warehouses, learning_rate=0.001, gamma=0.99):
        self.actor_critic = ActorCriticNetwork(n_warehouses)
        self.optimizer = optim.Adam(self.actor_critic.parameters(), lr=learning_rate)
        self.gamma = gamma
        self.saved_log_probs = []  # Логи вероятностей действий
        self.rewards = []  # Награды
        self.state_values = []  # Оценки Critic

    def select_action(self, state):
        """Выбор действия с использованием Actor-Critic"""
        state_tensor = {
            'current': torch.tensor([state['current']], dtype=torch.long),
            'visited_mask': torch.tensor([state['visited_mask']], dtype=torch.float32)
        }

        # Получение вероятностей действий и оценки состояния
        probs, state_value = self.actor_critic(state_tensor)
        m = torch.distributions.Categorical(probs)
        action = m.sample()

        # Сохранение для обновления
        self.saved_log_probs.append(m.log_prob(action))
        self.state_values.append(state_value)
        return action.item()

    def update(self):
        """Обновление Actor и Critic"""
        if not self.rewards:
            return

        # Вычисление дисконтированных returns
        R = 0
        policy_loss = []
        returns = []

        for r in self.rewards[::-1]:
            R = r + self.gamma * R
            returns.insert(0, R)

        # Нормализация returns
        returns = torch.tensor(returns, dtype=torch.float32)
        returns = (returns - returns.mean()) / (returns.std() + 1e-9)

        # Объединение оценок состояний
        state_values = torch.cat(self.state_values)

        # Advantage = Returns - State Values
        advantages = returns - state_values.squeeze()

        # Потери Actor: -log(π(a|s)) * A
        for log_prob, advantage in zip(self.saved_log_probs, advantages):
            policy_loss.append(-log_prob * advantage.detach())

        # Потери Critic: MSE между предсказаниями и returns
        value_loss = nn.MSELoss()(state_values.squeeze(), returns)

        # Совместное обновление
        self.optimizer.zero_grad()
        loss = torch.stack(policy_loss).sum() + value_loss
        loss.backward()
        self.optimizer.step()

        # Очистка буферов
        self.saved_log_probs = []
        self.rewards = []
        self.state_values = []

In [ ]:
class PPOAgent:
    """
    Proximal Policy Optimization агент

    Особенности:
    - Clipping для стабильности обучения
    - Несколько эпох обновления на одном наборе данных
    - Энтропийный бонус для exploration

    Гиперпараметры:
    - learning_rate=0.001
    - gamma=0.99: дисконтирование
    - clip_epsilon=0.2: параметр clipping
    """

    def __init__(self, n_warehouses, learning_rate=0.001, gamma=0.99, clip_epsilon=0.2):
        self.actor_critic = ActorCriticNetwork(n_warehouses)
        self.optimizer = optim.Adam(self.actor_critic.parameters(), lr=learning_rate)
        self.gamma = gamma
        self.clip_epsilon = clip_epsilon
        self.memory = []  # Буфер для хранения траектории

    def select_action(self, state):
        """Выбор действия с сохранением в память"""
        state_tensor = {
            'current': torch.tensor([state['current']], dtype=torch.long),
            'visited_mask': torch.tensor([state['visited_mask']], dtype=torch.float32)
        }

        with torch.no_grad():
            probs, state_value = self.actor_critic(state_tensor)
            m = torch.distributions.Categorical(probs)
            action = m.sample()

        # Сохранение всей информации для PPO updates
        self.memory.append({
            'state': state_tensor,
            'action': action,
            'log_prob': m.log_prob(action),
            'value': state_value,
            'reward': 0.0,  # Заполняется позже
            'done': False
        })

        return action.item()

    def update(self, final_reward=0.0):
        """Обновление политики с PPO clipping"""
        if not self.memory:
            return

        # Распределение финальной награды по всем шагам
        for i in range(len(self.memory)):
            self.memory[i]['reward'] = final_reward / len(self.memory)

        # Извлечение данных из памяти
        states = [m['state'] for m in self.memory]
        actions = torch.stack([m['action'] for m in self.memory])
        old_log_probs = torch.stack([m['log_prob'] for m in self.memory])
        old_values = torch.cat([m['value'] for m in self.memory])
        rewards = torch.tensor([m['reward'] for m in self.memory], dtype=torch.float32)

        # Вычисление дисконтированных returns
        returns = []
        R = 0.0
        for r in rewards.flip(0):
            R = r + self.gamma * R
            returns.insert(0, R)
        returns = torch.tensor(returns, dtype=torch.float32)

        # Advantage с нормализацией
        advantages = returns - old_values.squeeze()
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)

        # Несколько эпох PPO
        for _ in range(3):
            new_log_probs = []
            new_values = []
            new_entropy = []

            # Пересчет для новых параметров
            for i, state in enumerate(states):
                probs, value = self.actor_critic(state)
                dist = torch.distributions.Categorical(probs)
                new_log_probs.append(dist.log_prob(actions[i]))
                new_values.append(value)
                new_entropy.append(dist.entropy())

            new_log_probs = torch.stack(new_log_probs)
            new_values = torch.cat(new_values)
            new_entropy = torch.stack(new_entropy)

            # PPO ratio и clipping
            ratio = (new_log_probs - old_log_probs).exp()
            surr1 = ratio * advantages
            surr2 = torch.clamp(ratio, 1 - self.clip_epsilon, 1 + self.clip_epsilon) * advantages

            # Потери
            value_loss = nn.MSELoss()(new_values.squeeze(), returns)
            policy_loss = -torch.min(surr1, surr2).mean()  # Берем минимум для clipping
            entropy_bonus = -0.01 * new_entropy.mean()  # Поощрение exploration

            total_loss = policy_loss + 0.5 * value_loss + entropy_bonus

            # Обновление
            self.optimizer.zero_grad()
            total_loss.backward()
            self.optimizer.step()

        # Очистка памяти
        self.memory = []

In [ ]:
class DQNAgent:
    """
    Deep Q-Network агент

    Особенности:
    - Experience replay для декорреляции данных
    - Target network для стабильности
    - ε-greedy exploration

    Гиперпараметры:
    - learning_rate=0.001
    - gamma=0.99
    - epsilon=1.0: начальная вероятность случайного действия
    - epsilon_decay=0.995: скорость уменьшения ε
    - memory_size=10000: размер replay buffer
    - batch_size=32: размер мини-батча
    """

    def __init__(self, n_warehouses, learning_rate=0.001, gamma=0.99,
                 epsilon=1.0, epsilon_decay=0.995):
        self.q_network = QNetwork(n_warehouses)
        self.target_network = QNetwork(n_warehouses)
        self.target_network.load_state_dict(self.q_network.state_dict())

        self.optimizer = optim.Adam(self.q_network.parameters(), lr=learning_rate)
        self.gamma = gamma
        self.epsilon = epsilon  # Для ε-greedy
        self.epsilon_decay = epsilon_decay
        self.memory = deque(maxlen=10000)  # Replay buffer
        self.batch_size = 32

    def select_action(self, state):
        """ε-greedy выбор действия"""
        # Случайное действие с вероятностью ε
        if random.random() < self.epsilon:
            visited_mask = state['visited_mask']
            valid_actions = [i for i in range(len(visited_mask)) if visited_mask[i] == 0.0]
            return random.choice(valid_actions) if valid_actions else 0

        # Жадное действие на основе Q-значений
        state_tensor = {
            'current': torch.tensor([state['current']], dtype=torch.long),
            'visited_mask': torch.tensor([state['visited_mask']], dtype=torch.float32)
        }

        with torch.no_grad():
            q_values = self.q_network(state_tensor)
            return q_values.argmax().item()

    def store_experience(self, state, action, reward, next_state, done):
        """Сохранение опыта в replay buffer"""
        self.memory.append((state, action, reward, next_state, done))

    def update(self):
        """Обновление Q-сети на мини-батче из replay buffer"""
        if len(self.memory) < self.batch_size:
            return

        # Сэмплирование случайного мини-батча
        batch = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        # Подготовка тензоров
        state_current = torch.tensor([s['current'] for s in states], dtype=torch.long)
        state_visited = torch.tensor([s['visited_mask'] for s in states], dtype=torch.float32)

        next_state_current = torch.tensor([s['current'] for s in next_states], dtype=torch.long)
        next_state_visited = torch.tensor([s['visited_mask'] for s in next_states], dtype=torch.float32)

        state_tensors = {'current': state_current, 'visited_mask': state_visited}
        next_state_tensors = {'current': next_state_current, 'visited_mask': next_state_visited}

        # Вычисление target Q-значений с target network
        with torch.no_grad():
            next_q_values = self.target_network(next_state_tensors).max(1)[0]

        # Функция Беллмана: Q-target = r + γ * max Q(s', a')
        targets = torch.tensor(rewards, dtype=torch.float32) + \
                  self.gamma * next_q_values * (1 - torch.tensor(dones, dtype=torch.float32))

        # Текущие Q-значения
        current_q_values = self.q_network(state_tensors)
        current_q_values = current_q_values[torch.arange(self.batch_size), actions]

        # MSE loss между current и target Q-значениями
        loss = nn.MSELoss()(current_q_values, targets)

        # Градиентный спуск
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # Decay ε для уменьшения exploration со временем
        self.epsilon = max(0.01, self.epsilon * self.epsilon_decay)

        # Периодическое обновление target network
        if random.random() < 0.01:  # С вероятностью 1%
            self.target_network.load_state_dict(self.q_network.state_dict())

In [ ]:
class DoubleDQNAgent(DQNAgent):
    """
    Double DQN агент - улучшение DQN

    Решает проблему переоценки (overestimation) Q-значений:
    - Использует online network для выбора действий
    - Использует target network для их оценки

    Наследует все от DQNAgent, переопределяет только update()
    """

    def update(self):
        """Обновление с Double DQN логикой"""
        if len(self.memory) < self.batch_size:
            return

        batch = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        # Подготовка тензоров
        state_current = torch.tensor([s['current'] for s in states], dtype=torch.long)
        state_visited = torch.tensor([s['visited_mask'] for s in states], dtype=torch.float32)

        next_state_current = torch.tensor([s['current'] for s in next_states], dtype=torch.long)
        next_state_visited = torch.tensor([s['visited_mask'] for s in next_states], dtype=torch.float32)

        state_tensors = {'current': state_current, 'visited_mask': state_visited}
        next_state_tensors = {'current': next_state_current, 'visited_mask': next_state_visited}

        # Double DQN: online network выбирает, target network оценивает
        with torch.no_grad():
            next_actions = self.q_network(next_state_tensors).argmax(1)  # Online network
            next_q_values = self.target_network(next_state_tensors)      # Target network
            next_q_values = next_q_values[torch.arange(self.batch_size), next_actions]

        # Функция Беллмана
        targets = torch.tensor(rewards, dtype=torch.float32) + \
                  self.gamma * next_q_values * (1 - torch.tensor(dones, dtype=torch.float32))

        # Текущие Q-значения
        current_q_values = self.q_network(state_tensors)
        current_q_values = current_q_values[torch.arange(self.batch_size), actions]

        # Loss и обновление
        loss = nn.MSELoss()(current_q_values, targets)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # Decay ε
        self.epsilon = max(0.01, self.epsilon * self.epsilon_decay)

        # Периодическое обновление target network
        if random.random() < 0.01:
            self.target_network.load_state_dict(self.q_network.state_dict())

In [ ]:
class SACAgent:
    """
    Soft Actor-Critic агент (для дискретных действий)

    Особенности:
    - Maximum entropy RL: баланс exploration и exploitation
    - Два критика для уменьшения переоценки
    - Автоматическая настройка temperature parameter (α)

    Гиперпараметры:
    - learning_rate=0.001
    - gamma=0.99
    - alpha=0.2: параметр энтропии (temperature)
    """

    def __init__(self, n_warehouses, learning_rate=0.001, gamma=0.99, alpha=0.2):
        self.actor = PolicyNetwork(n_warehouses)
        self.critic1 = QNetwork(n_warehouses)  # Первый критик
        self.critic2 = QNetwork(n_warehouses)  # Второй критик

        # Отдельные оптимизаторы для каждой сети
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=learning_rate)
        self.critic1_optimizer = optim.Adam(self.critic1.parameters(), lr=learning_rate)
        self.critic2_optimizer = optim.Adam(self.critic2.parameters(), lr=learning_rate)

        self.gamma = gamma
        self.alpha = alpha  # Temperature parameter
        self.memory = deque(maxlen=10000)
        self.batch_size = 32

    def select_action(self, state):
        """Выбор действия с учетом энтропии"""
        state_tensor = {
            'current': torch.tensor([state['current']], dtype=torch.long),
            'visited_mask': torch.tensor([state['visited_mask']], dtype=torch.float32)
        }

        with torch.no_grad():
            probs = self.actor(state_tensor)
            m = torch.distributions.Categorical(probs)
            action = m.sample()

        return action.item()

    def store_experience(self, state, action, reward, next_state, done):
        """Сохранение опыта в replay buffer"""
        self.memory.append((state, action, reward, next_state, done))

    def update(self):
        """Обновление актора и критиков"""
        if len(self.memory) < self.batch_size:
            return

        # Сэмплирование мини-батча
        batch = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        # Подготовка тензоров
        state_current = torch.tensor([s['current'] for s in states], dtype=torch.long)
        state_visited = torch.tensor([s['visited_mask'] for s in states], dtype=torch.float32)

        next_state_current = torch.tensor([s['current'] for s in next_states], dtype=torch.long)
        next_state_visited = torch.tensor([s['visited_mask'] for s in next_states], dtype=torch.float32)

        state_tensors = {'current': state_current, 'visited_mask': state_visited}
        next_state_tensors = {'current': next_state_current, 'visited_mask': next_state_visited}

        actions_tensor = torch.tensor(actions)
        rewards_tensor = torch.tensor(rewards, dtype=torch.float32)
        dones_tensor = torch.tensor(dones, dtype=torch.float32)

        # Обновление критиков
        with torch.no_grad():
            # Распределение политики для следующего состояния
            next_probs = self.actor(next_state_tensors)
            next_log_probs = torch.log(next_probs + 1e-8)

            # Q-значения от двух критиков
            next_q1 = self.critic1(next_state_tensors)
            next_q2 = self.critic2(next_state_tensors)
            next_q = torch.min(next_q1, next_q2)  # Берем минимум для консервативности

            # Soft value: ∑π(a|s) * (Q(s,a) - α * logπ(a|s))
            next_value = (next_probs * (next_q - self.alpha * next_log_probs)).sum(dim=1)
            targets = rewards_tensor + self.gamma * next_value * (1 - dones_tensor)

        # Потери для критиков
        current_q1 = self.critic1(state_tensors)
        current_q2 = self.critic2(state_tensors)

        current_q1 = current_q1[torch.arange(self.batch_size), actions_tensor]
        current_q2 = current_q2[torch.arange(self.batch_size), actions_tensor]

        critic1_loss = nn.MSELoss()(current_q1, targets)
        critic2_loss = nn.MSELoss()(current_q2, targets)

        # Обновление критиков
        self.critic1_optimizer.zero_grad()
        critic1_loss.backward()
        self.critic1_optimizer.step()

        self.critic2_optimizer.zero_grad()
        critic2_loss.backward()
        self.critic2_optimizer.step()

        # Обновление актора
        probs = self.actor(state_tensors)
        log_probs = torch.log(probs + 1e-8)

        with torch.no_grad():
            q1 = self.critic1(state_tensors)
            q2 = self.critic2(state_tensors)
            q = torch.min(q1, q2)

        # Потери актора: максимизация (Q(s,a) - α * logπ(a|s))
        actor_loss = (probs * (self.alpha * log_probs - q)).sum(dim=1).mean()

        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

In [ ]:
# Обучение и сравнение
def train_rl_agents(distance_matrix, warehouses_df, num_episodes=500):
    """
    Обучение всех RL агентов

    Параметры:
    - distance_matrix: матрица расстояний
    - warehouses_df: информация о складах
    - num_episodes=500: количество эпизодов обучения
      Источник: эмпирически выбран для начального обучения
    """
    env = TSPEnvironment(distance_matrix, warehouses_df)
    n_warehouses = len(warehouses_df)

    # Инициализация всех агентов
    agents = {
        'Policy Gradient': PolicyGradientAgent(n_warehouses),
        'A2C': A2CAgent(n_warehouses),
        'PPO': PPOAgent(n_warehouses),
        'DQN': DQNAgent(n_warehouses),
        'Double DQN': DoubleDQNAgent(n_warehouses),
        'SAC': SACAgent(n_warehouses)
    }

    results = {}

    print("Обучение")
    print(f"Складов: {n_warehouses}")
    print(f"Эпизодов: {num_episodes}")
    print(f"{'-'*80}")

    for agent_name, agent in agents.items():
        print(f"\n{agent_name}")
        print("-" * 60)

        start_time = time.time()

        episode_rewards = []
        episode_distances = []
        best_route = None
        best_distance = float('inf')

        # Основной цикл обучения
        for episode in range(num_episodes):
            state = env.reset()
            total_reward = 0.0
            done = False
            step_count = 0
            max_steps = n_warehouses * 2 # Лимит шагов для предотвращения бесконечных циклов

            # Эпизод
            while not done and step_count < max_steps:
                action = agent.select_action(state)
                next_state, reward, done, _ = env.step(action)
                total_reward += reward

                 # Сохранение опыта для off-policy методов
                if hasattr(agent, 'store_experience'):
                    agent.store_experience(state, action, reward, next_state, done)

                # Сохранение наград для on-policy методов
                if hasattr(agent, 'rewards'):
                    agent.rewards.append(reward)

                state = next_state
                step_count += 1

            # Периодическое обновление
            if episode % 10 == 0 or done:
                if agent_name == 'PPO':        # PPO использует финальную награду
                    agent.update(total_reward)
                elif hasattr(agent, 'update'):
                    agent.update()

            # Информация о маршруте
            route_info = env.get_route_info()
            current_distance = route_info['distance_km']

            # Сохранение статистики
            episode_rewards.append(total_reward)
            episode_distances.append(current_distance)

            # Обновление лучшего результата
            if current_distance < best_distance and len(env.visited) == n_warehouses:
                best_distance = current_distance
                best_route = route_info['route']

            # Логирование прогресса
            if episode % 100 == 0:
                visited_count = len(env.visited)
                print(f"  Эпизод {episode:4d}: {current_distance:7.2f} км, "
                      f"Посещено: {visited_count:2d}/{n_warehouses}")

        end_time = time.time()
        total_time = end_time - start_time
        avg_time_per_episode = total_time / num_episodes

        # Сохраняем результаты
        results[agent_name] = {
            'rewards': episode_rewards,
            'distances': episode_distances,
            'best_route': best_route,
            'best_distance': best_distance,
            'avg_last_100': np.mean(episode_distances[-100:]),  # Среднее последних 100
            'std_last_100': np.std(episode_distances[-100:]),   # Стандартное отклонение
            'total_time': total_time,                           # Общее время
            'avg_time_per_episode': avg_time_per_episode        # Среднее время на эпизод
        }

        print(f"   Лучшее: {best_distance:.2f} км")
        print(f"   Среднее (последние 100): {results[agent_name]['avg_last_100']:.2f} км")
        print(f"   Время: {total_time:.1f} сек ({avg_time_per_episode:.3f} сек/эпизод)")

    return results, agents


def print_rl_results_table(results):
    """
    Выводит таблицу сравнения RL агентов

    Что считаем:
    1. Лучший результат (км) - минимальная длина маршрута за все эпизоды
    2. Среднее последние 100 (км) - средняя длина последних 100 маршрутов
    3. Станд. отклонение - станд. отклонение последних 100 маршрутов
    4. Время обучения (сек) - общее время обучения агента
    5. Время на эпизод (сек) - среднее время на один эпизод
    """
    print(f"\n{'='*80}")
    print("Свобдная таблицы")
    print(f"{'='*80}")

    # Создаем список для данных таблицы
    table_data = []

    for agent_name, result in results.items():
        table_data.append([
            agent_name,
            f"{result['best_distance']:.2f}",
            f"{result['avg_last_100']:.2f}",
            f"{result['std_last_100']:.2f}",
            f"{result['total_time']:.1f}",
            f"{result['avg_time_per_episode']:.3f}"
        ])

    # Сортируем по лучшему результату (по возрастанию)
    table_data.sort(key=lambda x: float(x[1]))

    # Заголовки таблицы
    headers = [
        "Метод",
        "Лучший результат (км)",
        "Среднее последние 100 (км)",
        "Станд. отклонение",
        "Время обучения (сек)",
        "Время на эпизод (сек)"
    ]

    # Выводим таблицу
    print(f"{'Метод':<20} | {'Лучший':>10} | {'Среднее':>10} | {'Ст.откл':>8} | {'Время':>8} | {'Время/эп':>8}")
    print("-" * 80)

    for row in table_data:
        print(f"{row[0]:<20} | {row[1]:>10} | {row[2]:>10} | {row[3]:>8} | {row[4]:>8} | {row[5]:>8}")

    return table_data

def plot_results(results):
    """
    Визуализация результатов обучения RL агентов

    Выводит 4 графика:
    1. Сглаженные награды по эпизодам
    2. Сглаженные расстояния маршрутов
    3. Лучшие результаты каждого алгоритма
    4. Общее время обучения каждого алгоритма
    """
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

    # 1. График наград (сглаженный)
    for agent_name, result in results.items():
        rewards_smooth = pd.Series(result['rewards']).rolling(20, min_periods=1).mean()
        ax1.plot(rewards_smooth, label=agent_name, alpha=0.7)
    ax1.set_title('Episode Rewards (Smoothed)')
    ax1.set_xlabel('Episode')
    ax1.set_ylabel('Total Reward')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # 2. График расстояний (сглаженный)
    for agent_name, result in results.items():
        distances_smooth = pd.Series(result['distances']).rolling(20, min_periods=1).mean()
        ax2.plot(distances_smooth, label=agent_name, alpha=0.7)
    ax2.set_title('Route Distances (Smoothed)')
    ax2.set_xlabel('Episode')
    ax2.set_ylabel('Distance (km)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    # 3. Лучшие результаты (столбчатая диаграмма)
    best_distances = {name: result['best_distance'] for name, result in results.items()}
    names = list(best_distances.keys())
    distances = list(best_distances.values())

    colors = ['#ff9999', '#66b3ff', '#99ff99', '#ffcc99', '#c2c2f0', '#ffb3e6']
    bars1 = ax3.bar(names, distances, color=colors[:len(names)])
    ax3.set_title('Best Route Distances by Algorithm')
    ax3.set_ylabel('Distance (km)')
    ax3.tick_params(axis='x', rotation=45)

    for bar, distance in zip(bars1, distances):
        height = bar.get_height()
        ax3.text(bar.get_x() + bar.get_width()/2., height + max(distances)*0.02,
                f'{distance:.1f} км', ha='center', va='bottom', fontsize=9)

    # 4. Общее время обучения (сек)
    total_times = {name: result['total_time'] for name, result in results.items()}
    names_time = list(total_times.keys())
    times = list(total_times.values())

    bars2 = ax4.bar(names_time, times, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7', '#DDA0DD'])
    ax4.set_title('Total Training Time by Algorithm')
    ax4.set_ylabel('Time (seconds)')
    ax4.tick_params(axis='x', rotation=45)
    ax4.grid(True, alpha=0.3, axis='y')

    # Добавляем значения времени на столбцы
    for bar, time_val in zip(bars2, times):
        height = bar.get_height()
        ax4.text(bar.get_x() + bar.get_width()/2., height + max(times)*0.02,
                f'{time_val:.1f} сек', ha='center', va='bottom', fontsize=9)

    plt.tight_layout()
    plt.savefig('rl_algorithms_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
distance_matrix, warehouses_df = load_data()

In [ ]:
print(f"Загружено: {len(warehouses_df)} складов")
print(f"Размер матрицы: {distance_matrix.shape}")
print(f"Тип данных: {distance_matrix.dtype}")

In [ ]:
print(f"Загружено: {len(warehouses_df)} складов")
print(f"Размер матрицы: {distance_matrix.shape}")
print(f"Тип данных: {distance_matrix.dtype}")

# Запускаем обучение RL агентов
print("\nОбучение RL агентов")
results, agents = train_rl_agents(distance_matrix, warehouses_df, num_episodes=500)

# Выводим таблицу сравнения
print_rl_results_table(results)

# Визуализируем результаты
plot_results(results)

# Сохраняем результаты
results_summary = {}
for agent_name, result in results.items():
    results_summary[agent_name] = {
        'best_distance_km': result['best_distance'],
        'best_route': result['best_route'],
        'avg_last_100_km': result['avg_last_100'],
        'std_last_100_km': result['std_last_100'],
        'total_time_sec': result['total_time'],
        'avg_time_per_episode_sec': result['avg_time_per_episode']
    }

with open('rl_results.json', 'w') as f:
    json.dump(results_summary, f, indent=2)

print("\nРезультаты сохранены в 'rl_results.json'")

## RL оптимизация

In [ ]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from collections import deque
import matplotlib.pyplot as plt
import pandas as pd
import json

In [ ]:
# Оптимизируем архитектуры
class EnhancedPolicyNetwork(nn.Module):
    """
    Улучшения для Policy Gradient методов:
    1. Добавляем LayerNorm - нормализуем выходы слоев для стабильности обучения
    2. Используем Dropout - предотвращаем переобучение (отключаем часть нейронов при обучении)
    3. Xavier инициализация - инициализируем веса для быстрой сходимости
    4. Больше скрытых слоев - увеличиваем емкость сети для сложных паттернов
    """
    def __init__(self, n_warehouses, hidden_size=256):
        super(EnhancedPolicyNetwork, self).__init__()
        self.n_warehouses = n_warehouses

        # Строим последовательность слоев с регуляризацией
        self.network = nn.Sequential(
            # Входной слой: one-hot текущего + маска посещенных (2*n_warehouses признаков)
            nn.Linear(n_warehouses * 2, hidden_size),
            nn.LayerNorm(hidden_size),  # нормализация: стабилизирует обучение
            nn.ReLU(),  # Нелинейность для сложных зависимостей
            nn.Dropout(0.1),  # регуляризация: отключаем 10% нейронов при обучении

            # Скрытый слой
            nn.Linear(hidden_size, hidden_size),
            nn.LayerNorm(hidden_size),  # Еще одна нормализация
            nn.ReLU(),
            nn.Dropout(0.1),  # Еще немного регуляризации

            # Выходной слой: вероятности выбора каждого склада
            nn.Linear(hidden_size, n_warehouses)
        )

        # инициализация Xavier для равномерного распределения градиентов
        self._initialize_weights()

    def _initialize_weights(self):
        """Инициализируем веса по методу Xavier для быстрой сходимости"""
        for layer in self.network:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)  # Xavier инициализация весов
                nn.init.constant_(layer.bias, 0.01)  # Маленькое положительное смещение

    def forward(self, state):
        """Прямой проход через сеть"""
        # Получаем текущий склад и маску посещенных
        current = state['current']
        visited_mask = state['visited_mask']

        # One-hot кодирование текущего склада
        current_onehot = torch.zeros(current.shape[0], self.n_warehouses, dtype=torch.float32)
        for i, c in enumerate(current):
            current_onehot[i, c] = 1.0

        # Объединяем one-hot вектора и маску посещенных
        x = torch.cat([current_onehot, visited_mask], dim=1)

        # Проходим через все слои сети
        logits = self.network(x)

        # маскируем уже посещенные склады
        mask = visited_mask.bool()
        logits[mask] = -1e8  # Очень большое отрицательное число = очень низкая вероятность

        # Превращаем logits в вероятности с помощью softmax
        return torch.softmax(logits, dim=-1)

class EnhancedActorCriticNetwork(nn.Module):
    """
    Улучшения Actor-Critic сети:
    - LayerNorm в энкодере
    - Раздельные головы Actor и Critic
    - Xavier инициализация

    Actor-Critic:
    1. Actor (Актор) - предсказывает вероятности действий
    2. Critic (Критик) - оценивает "ценность" текущего состояния
    3. Общий энкодер - извлекает общие признаки для обоих голов
    Преимущество: Critic помогает Actor понимать, какие действия действительно хороши
    """
    def __init__(self, n_warehouses, hidden_size=256):
        super(EnhancedActorCriticNetwork, self).__init__()
        self.n_warehouses = n_warehouses

        # Общий энкодер: извлекает признаки для Actor и Critic
        self.encoder = nn.Sequential(
            nn.Linear(n_warehouses * 2, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.ReLU()
        )

        # Actor: предсказывает вероятности действий
        self.actor = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Linear(hidden_size // 2, n_warehouses)
        )

        # Critic: оценивает ценность состояния (скалярное значение)
        self.critic = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Linear(hidden_size // 2, 1)
        )

        # Инициализируем веса
        self._initialize_weights()

    def _initialize_weights(self):
        """Инициализация весов для всех слоев"""
        for module in [self.encoder, self.actor, self.critic]:
            for layer in module:
                if isinstance(layer, nn.Linear):
                    nn.init.xavier_uniform_(layer.weight)
                    nn.init.constant_(layer.bias, 0.01)

    def forward(self, state):
        """Возвращает и политику Actor, и оценку Critic"""
        current = state['current']
        visited_mask = state['visited_mask']

        # One-hot кодирование текущего склада
        current_onehot = torch.zeros(current.shape[0], self.n_warehouses, dtype=torch.float32)
        for i, c in enumerate(current):
            current_onehot[i, c] = 1.0

        # Объединяем входные данные
        x = torch.cat([current_onehot, visited_mask], dim=1)

        # Извлекаем общие признаки через энкодер
        features = self.encoder(x)

        # Actor: вероятности действий
        logits = self.actor(features)
        logits[visited_mask.bool()] = -1e8  # Маскируем посещенные
        action_probs = torch.softmax(logits, dim=-1)

        # Critic: оценка ценности состояния
        state_value = self.critic(features)

        return action_probs, state_value

class EnhancedSACNetwork(nn.Module):
    """
    Улучшение сети для SAC (Soft Actor-Critic):
    1. Dueling Architecture для лучшей оценки value и advantage
    2. LayerNorm для стабильности обучения
    3. Dropout для регуляризации
    4. Xavier инициализация весов

    Потенциальные плюсы:
    - Value stream оценивает общую ценность состояния
    - Advantage stream оценивает преимущество каждого действия
    - Это помогает в задачах с большим пространством действий
    """
    def __init__(self, n_warehouses, hidden_size=256):
        super(EnhancedSACNetwork, self).__init__()
        self.n_warehouses = n_warehouses

        # Общий энкодер
        self.encoder = nn.Sequential(
            nn.Linear(n_warehouses * 2, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_size, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.ReLU(),
            nn.Dropout(0.1)
        )

        # Value stream: оценивает ценность состояния
        self.value_stream = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Linear(hidden_size // 2, 1)
        )

        # Advantage stream: оценивает преимущество действий
        self.advantage_stream = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Linear(hidden_size // 2, n_warehouses)
        )

        self._initialize_weights()

    def _initialize_weights(self):
        """Инициализация весов по методу Xavier"""
        for module in [self.encoder, self.value_stream, self.advantage_stream]:
            for layer in module:
                if isinstance(layer, nn.Linear):
                    nn.init.xavier_uniform_(layer.weight)
                    nn.init.constant_(layer.bias, 0.01)

    def forward(self, state):
        """Возвращает Q-значения с использованием Dueling Architecture"""
        current = state['current']
        visited_mask = state['visited_mask']

        # One-hot кодирование текущего склада
        current_onehot = torch.zeros(current.shape[0], self.n_warehouses, dtype=torch.float32)
        for i, c in enumerate(current):
            current_onehot[i, c] = 1.0

        # Объединяем входные данные
        x = torch.cat([current_onehot, visited_mask], dim=1)

        # Извлекаем признаки через энкодер
        features = self.encoder(x)

        # Вычисляем value и advantage
        value = self.value_stream(features)
        advantage = self.advantage_stream(features)

        # Dueling формула: Q(s,a) = V(s) + A(s,a) - mean(A(s,a))
        q_values = value + advantage - advantage.mean(dim=1, keepdim=True)

        # Маскируем посещенные склады
        q_values[visited_mask.bool()] = -1e8

        return q_values

In [ ]:
#Оптимизированные RL aгенты
class EnhancedPolicyGradientAgent:
    """
    Улучшение базового Policy Gradient агента

    Улучшения:
    1. Энтропийная регуляризация: поощряем exploration
    2. AdamW оптимизатор: более стабильный Adam с L2 регуляризацией
    3. Cosine Annealing: автоматически уменьшаем learning rate
    4. Gradient Clipping: предотвращаем взрыв градиентов

    Policy Gradient:
    1. Собираем траекторию (действия, награды)
    2. Вычисляем advantage (насколько действия лучше среднего)
    3. Увеличиваем вероятность хороших действий, уменьшаем плохих
    """
    def __init__(self, n_warehouses, learning_rate=0.001, gamma=0.99, entropy_coef=0.01):
        # Инициализируем нейросеть
        self.policy_network = EnhancedPolicyNetwork(n_warehouses)

        # AdamW (Adam с weight decay)
        self.optimizer = optim.AdamW(self.policy_network.parameters(),
                                     lr=learning_rate, weight_decay=1e-4)

        # Cosine Annealing: автоматически настраивает learning rate
        self.scheduler = optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max=1500)

        # Гиперпараметры
        self.gamma = gamma  # Коэффициент дисконтирования будущих наград
        self.entropy_coef = entropy_coef  # Сила энтропийной регуляризации

        # Буферы для хранения данных эпизода
        self.saved_log_probs = []  # Логарифмы вероятностей выбранных действий
        self.rewards = []  # Полученные награды
        self.entropies = []  # Энтропия распределения (мера неопределенности)

    def select_action(self, state):
        """Выбор действия на основе текущей политики"""
        # Преобразуем состояние в тензор
        state_tensor = {
            'current': torch.tensor([state['current']], dtype=torch.long),
            'visited_mask': torch.tensor([state['visited_mask']], dtype=torch.float32)
        }

        # Получаем вероятности действий из сети
        probs = self.policy_network(state_tensor)

        # Создаем категориальное распределение
        m = torch.distributions.Categorical(probs)
        action = m.sample()  # Выбираем действие случайно согласно распределению
        entropy = m.entropy()  # Вычисляем энтропию (понадобится для регуляризации)

        # Сохраняем для обучения
        self.saved_log_probs.append(m.log_prob(action))
        self.entropies.append(entropy)

        return action.item()  # Возвращаем индекс склада

    def update(self):
        """Обновление политики после завершения эпизода"""
        if not self.rewards:  # Если нет данных, ничего не делаем
            return

        # Вычисляем дисконтированные returns (G_t)
        R = 0  # Накопленная дисконтированная награда
        returns = []  # Список returns для каждого шага

        # Идем с конца эпизода к началу
        for r in self.rewards[::-1]:
            R = r + self.gamma * R  # Дисконтируем и суммируем
            returns.insert(0, R)  # Вставляем в начало (чтобы сохранить порядок)

        # Преобразуем в тензор и нормализуем для стабильности
        returns = torch.tensor(returns, dtype=torch.float32)
        returns = (returns - returns.mean()) / (returns.std() + 1e-8)

        # Вычисляем loss для Policy Gradient
        policy_loss = []
        for log_prob, R in zip(self.saved_log_probs, returns):
            # Основная формула Policy Gradient: -log(π(a|s)) * G_t
            # Минимизируем это = максимизируем вероятность хороших действий
            policy_loss.append(-log_prob * R)

        # Энтропийная регуляризация
        # Суммируем энтропии за все шаги эпизода
        entropy = torch.stack(self.entropies).sum()

        # Общий loss: policy_loss - коэффициент * энтропия
        # Энтропийная регуляризация поощряет exploration
        loss = torch.stack(policy_loss).sum() - self.entropy_coef * entropy

        # Градиентный спуск
        self.optimizer.zero_grad()  # Обнуляем градиенты
        loss.backward()  # Вычисляем градиенты

        # Gradient clipping: обрезаем большие градиенты для стабильности
        torch.nn.utils.clip_grad_norm_(self.policy_network.parameters(), max_norm=1.0)

        self.optimizer.step()  # Обновляем веса
        self.scheduler.step()  # Обновляем learning rate

        # Очищаем буферы для следующего эпизода
        self.saved_log_probs = []
        self.rewards = []
        self.entropies = []

class EnhancedA2CAgent:
    """
    Улучшенный A2C (Advantage Actor-Critic) агент.

    Улучшения:
    - Энтропийная регуляризация для Actor
    - Gradient clipping
    - Cosine annealing scheduler

    A2C:
    1. Actor выбирает действия
    2. Critic оценивает состояния
    3. Advantage = Returns - Value (насколько действие лучше среднего)
    Critic помогает Actor быстрее учиться, оценивая состояния
    """
    def __init__(self, n_warehouses, learning_rate=0.001, gamma=0.99, entropy_coef=0.01):
        # Инициализируем Actor-Critic сеть
        self.actor_critic = EnhancedActorCriticNetwork(n_warehouses)

        # Оптимизатор и scheduler
        self.optimizer = optim.AdamW(self.actor_critic.parameters(),
                                     lr=learning_rate, weight_decay=1e-4)
        self.scheduler = optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max=1500)

        # Гиперпараметры
        self.gamma = gamma
        self.entropy_coef = entropy_coef

        # Буферы
        self.saved_log_probs = []  # Логи вероятностей Actor
        self.rewards = []  # Награды
        self.state_values = []  # Оценки Critic
        self.entropies = []  # Энтропия

    def select_action(self, state):
        """Выбор действия с использованием Actor-Critic"""
        state_tensor = {
            'current': torch.tensor([state['current']], dtype=torch.long),
            'visited_mask': torch.tensor([state['visited_mask']], dtype=torch.float32)
        }

        # Получаем вероятности действий (Actor) и оценку состояния (Critic)
        probs, state_value = self.actor_critic(state_tensor)

        # Создаем распределение и выбираем действие
        m = torch.distributions.Categorical(probs)
        action = m.sample()
        entropy = m.entropy()

        # Сохраняем для обучения
        self.saved_log_probs.append(m.log_prob(action))
        self.state_values.append(state_value)
        self.entropies.append(entropy)

        return action.item()

    def update(self):
        """Обновление Actor и Critic после эпизода"""
        if not self.rewards:
            return

        # Вычисляем дисконтированные returns
        R = 0
        returns = []

        for r in self.rewards[::-1]:
            R = r + self.gamma * R
            returns.insert(0, R)

        returns = torch.tensor(returns, dtype=torch.float32)
        returns = (returns - returns.mean()) / (returns.std() + 1e-8)

        # Вычисляем Advantage
        state_values = torch.cat(self.state_values)  # Объединяем оценки Critic
        # Advantage = Returns - Value
        # Показывает, насколько returns лучше оценки Critic
        advantages = returns - state_values.squeeze()

        # Actor loss (Policy Gradient с advantage)
        policy_loss = []
        for log_prob, advantage in zip(self.saved_log_probs, advantages):
            # Учитываем advantage вместо простого return
            policy_loss.append(-log_prob * advantage.detach())

        # Critic loss (MSE между предсказаниями и реальными returns)
        value_loss = nn.MSELoss()(state_values.squeeze(), returns)

        # Энтропийная регуляризация
        entropy = torch.stack(self.entropies).sum()

        # Общий loss: Actor loss + Critic loss + Энтропия
        loss = torch.stack(policy_loss).sum() + 0.5 * value_loss - self.entropy_coef * entropy

        # Градиентный спуск
        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.actor_critic.parameters(), max_norm=1.0)
        self.optimizer.step()
        self.scheduler.step()

        # Очищаем буферы
        self.saved_log_probs = []
        self.rewards = []
        self.state_values = []
        self.entropies = []

class EnhancedPPOAgent:
    """
    Улучшенный PPO (Proximal Policy Optimization) агент.

    Улучшения:
    - Меньший clip epsilon (0.1 вместо 0.2)
    - Больше эпох (4 вместо 3)
    - AdamW с weight decay
    - Cosine annealing

    PPO:
    1. Clipped surrogate objective - предотвращает слишком большие изменения политики
    2. Несколько эпох обучения на одних данных - эффективное использование данных
    3. Энтропийная регуляризация
    Стабильное обучение, минимизирует риск испортить политику
    """
    def __init__(self, n_warehouses, learning_rate=0.0003, gamma=0.99,
                 clip_epsilon=0.1, entropy_coef=0.02):
        # Инициализируем сеть
        self.actor_critic = EnhancedActorCriticNetwork(n_warehouses)

        # Оптимизатор и scheduler
        self.optimizer = optim.AdamW(self.actor_critic.parameters(),
                                     lr=learning_rate, weight_decay=1e-4)
        self.scheduler = optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max=1500)

        # Гиперпараметры PPO
        self.gamma = gamma
        self.clip_epsilon = clip_epsilon  # Параметр clipping (меньше = осторожнее)
        self.entropy_coef = entropy_coef  # Сила энтропийной регуляризации

        # Буфер для хранения траектории
        self.memory = []

    def select_action(self, state):
        """Выбор действия с сохранением в memory для PPO"""
        state_tensor = {
            'current': torch.tensor([state['current']], dtype=torch.long),
            'visited_mask': torch.tensor([state['visited_mask']], dtype=torch.float32)
        }

        # В режиме без градиентов (для скорости)
        with torch.no_grad():
            probs, state_value = self.actor_critic(state_tensor)
            m = torch.distributions.Categorical(probs)
            action = m.sample()
            entropy = m.entropy()

        # Сохраняем все данные шага для PPO updates
        self.memory.append({
            'state': state_tensor,
            'action': action,
            'log_prob': m.log_prob(action),
            'value': state_value,
            'entropy': entropy
        })

        return action.item()

    def update(self, episode_reward=0):
        """
        Обновление политики PPO

        PPO:
        - Использует данные всего эпизода
        - Делает несколько эпох обновления
        - Применяет clipping для стабильности
        """
        if not self.memory:
            return

        # Извлекаем данные из memory
        states = [m['state'] for m in self.memory]
        actions = torch.stack([m['action'] for m in self.memory])
        old_log_probs = torch.stack([m['log_prob'] for m in self.memory])
        old_values = torch.cat([m['value'] for m in self.memory])
        old_entropies = torch.stack([m['entropy'] for m in self.memory])

        # используем одну награду за эпизод
        # Распределяем финальную награду по всем шагам
        rewards = torch.tensor([episode_reward / len(self.memory)] * len(self.memory),
                              dtype=torch.float32)

        # Вычисляем дисконтированные returns
        returns = []
        R = 0
        for r in rewards.flip(0):
            R = r + self.gamma * R
            returns.insert(0, R)
        returns = torch.tensor(returns, dtype=torch.float32)

        # Вычисляем advantage
        advantages = returns - old_values.squeeze()
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)

        # несколько эпох обучения на одних данных
        for _ in range(4):  # 4 эпохи вместо обычных 3
            new_log_probs = []
            new_values = []
            new_entropies = []

            # Пересчитываем для новых параметров
            for i, state in enumerate(states):
                probs, value = self.actor_critic(state)
                dist = torch.distributions.Categorical(probs)
                new_log_probs.append(dist.log_prob(actions[i]))
                new_values.append(value)
                new_entropies.append(dist.entropy())

            new_log_probs = torch.stack(new_log_probs)
            new_values = torch.cat(new_values)
            new_entropies = torch.stack(new_entropies)

            # Формула PPO: Clipped Surrogate Objective
            ratio = (new_log_probs - old_log_probs).exp()  # Отношение новых и старых вероятностей
            surr1 = ratio * advantages  # Обычный Policy Gradient
            # Clipped вариант: ограничиваем изменение вероятностей
            surr2 = torch.clamp(ratio, 1 - self.clip_epsilon, 1 + self.clip_epsilon) * advantages

            # Берем минимум для предотвращения слишком больших шагов
            policy_loss = -torch.min(surr1, surr2).mean()

            # Critic loss (MSE)
            value_loss = nn.MSELoss()(new_values.squeeze(), returns)

            # Энтропийная регуляризация
            entropy_bonus = -self.entropy_coef * new_entropies.mean()

            # Общий loss
            total_loss = policy_loss + 0.5 * value_loss + entropy_bonus

            # Градиентный спуск
            self.optimizer.zero_grad()
            total_loss.backward()
            torch.nn.utils.clip_grad_norm_(self.actor_critic.parameters(), max_norm=1.0)
            self.optimizer.step()

        # Обновляем learning rate
        self.scheduler.step()

        # Очищаем memory
        self.memory = []

In [ ]:
class EnhancedSACAgent:
    """
    Улучшение SAC (Soft Actor-Critic) агента

    Улучшения:
    1. Automatic entropy tuning (автоматическая настройка α):
       - α адаптируется во время обучения
       - Балансирует exploration и exploitation

    2. Double critics (два критика):
       - Две Q-сети для уменьшения переоценки
       - Берем минимум для консервативных оценок

    3. Target networks (таргет сети):
       - Плавное обновление для стабильности
       - Polyak averaging

    4. Prioratized experience replay:
       - Учимся быстрее на важных примерах
       - Исправляем смещение sampling

    5. Cosine anneling LR scheduler:
       - Автоматическая настройка learning rate

    6. Gradient clipping:
       - Предотвращаем взрыв градиентов
    """

    def __init__(self, n_warehouses, learning_rate=0.0003, gamma=0.99,
                 tau=0.005, alpha=0.2, target_entropy=None,
                 use_prioritized_replay=True):

        # инициализация сетей
        # Актор (политика)
        self.actor = EnhancedPolicyNetwork(n_warehouses)

        # Два критика (Q-сети)
        self.critic1 = EnhancedSACNetwork(n_warehouses)
        self.critic2 = EnhancedSACNetwork(n_warehouses)

        # Target сети для стабильности
        self.critic1_target = EnhancedSACNetwork(n_warehouses)
        self.critic2_target = EnhancedSACNetwork(n_warehouses)

        # Копируем веса в target сети
        self.critic1_target.load_state_dict(self.critic1.state_dict())
        self.critic2_target.load_state_dict(self.critic2.state_dict())

        # оптимизаторы
        # AdamW с weight decay для лучшего обобщения
        self.actor_optimizer = optim.AdamW(self.actor.parameters(),
                                          lr=learning_rate, weight_decay=1e-4)
        self.critic1_optimizer = optim.AdamW(self.critic1.parameters(),
                                            lr=learning_rate, weight_decay=1e-4)
        self.critic2_optimizer = optim.AdamW(self.critic2.parameters(),
                                            lr=learning_rate, weight_decay=1e-4)

        # Cosine anneling scheduler
        self.actor_scheduler = optim.lr_scheduler.CosineAnnealingLR(
            self.actor_optimizer, T_max=1500
        )
        self.critic1_scheduler = optim.lr_scheduler.CosineAnnealingLR(
            self.critic1_optimizer, T_max=1500
        )
        self.critic2_scheduler = optim.lr_scheduler.CosineAnnealingLR(
            self.critic2_optimizer, T_max=1500
        )

        # Automatic entropy tuning
        # Инициализируем log_alpha для автоматической настройки
        self.target_entropy = target_entropy or -np.log(1.0 / n_warehouses) * 0.98
        self.log_alpha = torch.tensor(np.log(alpha), requires_grad=True)
        self.alpha_optimizer = optim.AdamW([self.log_alpha], lr=learning_rate)

        # гиперпараметры
        self.gamma = gamma  # Коэффициент дисконтирования
        self.tau = tau      # Параметр плавного обновления target сетей
        self.n_warehouses = n_warehouses

        # replay buffer
        self.use_prioritized_replay = use_prioritized_replay

        if use_prioritized_replay:
            # Приоритизированный replay buffer
            self.memory = PrioritizedReplayBuffer(capacity=10000, alpha=0.6)
            self.beta = 0.4  # Параметр для корректировки смещения
            self.beta_increment = 0.001  # Увеличение beta со временем
        else:
            # Обычный replay buffer
            self.memory = deque(maxlen=10000)

        self.batch_size = 256  # Больше batch size для стабильности

        # статистика
        self.update_step = 0
        self.actor_losses = []
        self.critic_losses = []
        self.alpha_values = []

    def select_action(self, state):
        """
        Выбор действия с использованием стохастической политики:
        1. Получаем вероятности действий из актора
        2. Добавляем шум через энтропийную регуляризацию
        3. Выбираем действие согласно распределению
        """
        state_tensor = {
            'current': torch.tensor([state['current']], dtype=torch.long),
            'visited_mask': torch.tensor([state['visited_mask']], dtype=torch.float32)
        }

        # Получаем вероятности действий
        probs = self.actor(state_tensor)

        # Создаем категориальное распределение
        m = torch.distributions.Categorical(probs)
        action = m.sample()

        # Также вычисляем log вероятности для обучения
        log_prob = m.log_prob(action)

        return action.item(), log_prob

    def store_experience(self, state, action, reward, next_state, done):
        """
        Сохранение опыта в Replay buffer
        Для приоритизированного replay buffer храним также TD ошибку как приоритет
        """
        experience = (state, action, reward, next_state, done)

        if self.use_prioritized_replay:
            # Инициализируем с максимальным приоритетом
            max_priority = 1.0 if len(self.memory) == 0 else self.memory.max_priority()
            self.memory.add(experience, max_priority)
        else:
            self.memory.append(experience)

    def _soft_update(self, target, source):
        """
        Плавное обновление target сетей

        Формула: θ_target = τ * θ_source + (1 - τ) * θ_target

        Почему это важно:
        - Предотвращает резкие изменения в target значениях
        - Стабилизирует обучение Q-сетей
        """
        for target_param, param in zip(target.parameters(), source.parameters()):
            target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)

    def update(self):
        """
        Обновление SAC
        """

        # Проверяем, достаточно ли данных в buffer
        if len(self.memory) < self.batch_size:
            return

        # Sampling из Replay Buffer
        if self.use_prioritized_replay:
            batch, indices, weights = self.memory.sample(self.batch_size, self.beta)
            states, actions, rewards, next_states, dones = zip(*batch)
            weights = torch.tensor(weights, dtype=torch.float32)

            # Увеличиваем beta
            self.beta = min(1.0, self.beta + self.beta_increment)
        else:
            batch = random.sample(self.memory, self.batch_size)
            states, actions, rewards, next_states, dones = zip(*batch)
            weights = torch.ones(self.batch_size)

        # Подготовка тензоров
        state_current = torch.tensor([s['current'] for s in states], dtype=torch.long)
        state_visited = torch.tensor([s['visited_mask'] for s in states], dtype=torch.float32)

        next_state_current = torch.tensor([s['current'] for s in next_states], dtype=torch.long)
        next_state_visited = torch.tensor([s['visited_mask'] for s in next_states], dtype=torch.float32)

        state_tensors = {'current': state_current, 'visited_mask': state_visited}
        next_state_tensors = {'current': next_state_current, 'visited_mask': next_state_visited}

        actions_tensor = torch.tensor(actions)
        rewards_tensor = torch.tensor(rewards, dtype=torch.float32)
        dones_tensor = torch.tensor(dones, dtype=torch.float32)

        # Обновлеине критика
        with torch.no_grad():
            # Получаем политику для следующего состояния
            next_probs = self.actor(next_state_tensors)
            next_log_probs = torch.log(next_probs + 1e-8)  # Добавляем epsilon для стабильности

            # Получаем Q-значения от target критиков
            next_q1_target = self.critic1_target(next_state_tensors)
            next_q2_target = self.critic2_target(next_state_tensors)
            next_q_target = torch.min(next_q1_target, next_q2_target)

            # Вычисляем значение следующего состояния с учетом энтропии
            alpha = self.log_alpha.exp()
            next_value = (next_probs * (next_q_target - alpha * next_log_probs)).sum(dim=1)

            # Target Q-значения: r + γ * (1 - done) * V(s')
            target_q = rewards_tensor + self.gamma * next_value * (1 - dones_tensor)

        # Текущие Q-значения
        current_q1 = self.critic1(state_tensors)
        current_q2 = self.critic2(state_tensors)

        current_q1 = current_q1[torch.arange(self.batch_size), actions_tensor]
        current_q2 = current_q2[torch.arange(self.batch_size), actions_tensor]

        # Потери для критиков с учетом весов
        critic1_loss = (weights * (current_q1 - target_q).pow(2)).mean()
        critic2_loss = (weights * (current_q2 - target_q).pow(2)).mean()

        # Обновляем критиков
        self.critic1_optimizer.zero_grad()
        critic1_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.critic1.parameters(), max_norm=1.0)
        self.critic1_optimizer.step()

        self.critic2_optimizer.zero_grad()
        critic2_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.critic2.parameters(), max_norm=1.0)
        self.critic2_optimizer.step()

        # Обновление актора
        # Пересчитываем Q-значения с обновленными критикаим
        with torch.no_grad():
            q1 = self.critic1(state_tensors)
            q2 = self.critic2(state_tensors)
            q = torch.min(q1, q2)  # Берем минимум для консервативности

        # Получаем текущие вероятности действий
        probs = self.actor(state_tensors)
        log_probs = torch.log(probs + 1e-8)

        # Вычисляем потерю актора
        alpha = self.log_alpha.exp().detach()
        actor_loss = (probs * (alpha * log_probs - q)).sum(dim=1).mean()

        # Обновляем актора
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.actor.parameters(), max_norm=1.0)
        self.actor_optimizer.step()

        # Обновление температуры параметра α
        # Вычисляем потерю для α
        with torch.no_grad():
            # Текущая энтропия политики
            current_entropy = -(probs * log_probs).sum(dim=1).mean()

        # Целевая функция для α
        alpha_loss = -(self.log_alpha * (current_entropy - self.target_entropy).detach()).mean()

        # Обновляем α
        self.alpha_optimizer.zero_grad()
        alpha_loss.backward()
        self.alpha_optimizer.step()

        # Обновление target сетей
        self._soft_update(self.critic1_target, self.critic1)
        self._soft_update(self.critic2_target, self.critic2)

        # Обновление приоритетов (если используется PER)
        if self.use_prioritized_replay:
            # Вычисляем TD ошибки для обновления приоритетов
            with torch.no_grad():
                td_errors = torch.abs(target_q - current_q1).detach().numpy()
            self.memory.update_priorities(indices, td_errors)

        # Обновлеине learning rate schedulre
        self.actor_scheduler.step()
        self.critic1_scheduler.step()
        self.critic2_scheduler.step()

        # Сохранение статистики
        self.update_step += 1
        self.actor_losses.append(actor_loss.item())
        self.critic_losses.append((critic1_loss.item() + critic2_loss.item()) / 2)
        self.alpha_values.append(self.log_alpha.exp().item())

        return {
            'actor_loss': actor_loss.item(),
            'critic_loss': (critic1_loss.item() + critic2_loss.item()) / 2,
            'alpha': self.log_alpha.exp().item(),
            'entropy': current_entropy.item()
        }

class PrioritizedReplayBuffer:
    """
    Приоритизированный Replay Buffer:
    1. Хранит опыт с приоритетами (TD ошибками)
    2. Выбирает важные примеры для обучения чаще
    3. Корректирует смещение через importance sampling weights
    """

    def __init__(self, capacity=10000, alpha=0.6, beta=0.4, beta_increment=0.001):
        self.capacity = capacity
        self.alpha = alpha  # Параметр приоритета (0 = равномерный, 1 = только приоритеты)
        self.beta = beta    # Параметр importance sampling
        self.beta_increment = beta_increment

        self.buffer = []
        self.priorities = np.zeros(capacity, dtype=np.float32)
        self.position = 0
        self.size = 0

        # Минимальный приоритет для новых примеров
        self.min_priority = 1.0

    def __len__(self):
        """Возвращает текущий размер buffer"""
        return self.size

    def add(self, experience, priority=None):
        """
        Добавление опыта в buffer
        """
        if priority is None:
            priority = self.max_priority()

        if len(self.buffer) < self.capacity:
            self.buffer.append(experience)
        else:
            self.buffer[self.position] = experience

        # Сохраняем приоритет
        self.priorities[self.position] = priority ** self.alpha
        self.position = (self.position + 1) % self.capacity
        self.size = min(self.size + 1, self.capacity)

    def sample(self, batch_size, beta=None):
        """
        Выборка batch с учетом приоритетов
        """
        if beta is None:
            beta = self.beta

        if self.size == 0:
            return [], [], []

        # Нормализуем приоритеты для получения вероятностей
        probs = self.priorities[:self.size] / (self.priorities[:self.size].sum() + 1e-8)

        # Выбираем индексы согласно вероятностям
        indices = np.random.choice(self.size, min(batch_size, self.size), p=probs, replace=False)

        # Вычисляем importance sampling weights
        weights = (self.size * probs[indices]) ** (-beta)
        weights = weights / (weights.max() + 1e-8)  # Нормализуем

        # Собираем batch
        batch = [self.buffer[i] for i in indices]

        return batch, indices, weights

    def update_priorities(self, indices, priorities):
        """
        Обновление приоритетов для выбранных примеров
        """
        for idx, priority in zip(indices, priorities):
            self.priorities[idx] = (priority + 1e-5) ** self.alpha  # Добавляем epsilon

    def max_priority(self):
        """
        Возвращает максимальный приоритет в buffer
        """
        if self.size == 0:
            return 1.0
        return self.priorities[:self.size].max()

In [ ]:
class DuelingQNetwork(nn.Module):
    """
    Dueling DQN сеть:
    1. Разделяет оценку на Value и Advantage потоки
    2. Value stream: оценивает общую ценность состояния
    3. Advantage stream: оценивает преимущество каждого действия

    Формула: Q(s,a) = V(s) + A(s,a) - mean(A(s,a))
    """

    def __init__(self, n_warehouses, hidden_size=256):
        super(DuelingQNetwork, self).__init__()
        self.n_warehouses = n_warehouses

        # Общий энкодер
        self.encoder = nn.Sequential(
            nn.Linear(n_warehouses * 2, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU()
        )

        # Value stream: оценивает общую ценность состояния
        self.value_stream = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Linear(hidden_size // 2, 1)  # Одно значение на состояние
        )

        # Advantage stream: оценивает преимущество каждого действия
        self.advantage_stream = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Linear(hidden_size // 2, n_warehouses)  # Одно значение на каждое действие
        )

        self._initialize_weights()

    def _initialize_weights(self):
        """Инициализация весов"""
        for module in [self.encoder, self.value_stream, self.advantage_stream]:
            for layer in module:
                if isinstance(layer, nn.Linear):
                    nn.init.xavier_uniform_(layer.weight)
                    nn.init.constant_(layer.bias, 0.01)

    def forward(self, state):
        """Прямой проход через Dueling сеть"""
        current = state['current']
        visited_mask = state['visited_mask']

        # One-hot кодирование текущего склада
        current_onehot = torch.zeros(current.shape[0], self.n_warehouses, dtype=torch.float32)
        for i, c in enumerate(current):
            current_onehot[i, c] = 1.0

        # Объединяем входные данные
        x = torch.cat([current_onehot, visited_mask], dim=1)

        # Извлекаем признаки через энкодер
        features = self.encoder(x)

        # Вычисляем value и advantage
        value = self.value_stream(features)  # [batch_size, 1]
        advantage = self.advantage_stream(features)  # [batch_size, n_actions]

        #  Dueling формула: Q(s,a) = V(s) + A(s,a) - mean(A(s,a))
        # Это помогает стабилизировать обучение
        q_values = value + advantage - advantage.mean(dim=1, keepdim=True)

        # Маскируем посещенные склады
        q_values[visited_mask.bool()] = -1e8

        return q_values

class NoisyLinear(nn.Module):
    """
    Noisy linear слой:
    1. Добавляет параметризованный шум к весам и смещениям
    2. Заменяет ε-greedy exploration
    3. Позволяет сети исследовать самостоятельно

    Используется в Noisy DQN для улучшенного exploration
    """

    def __init__(self, in_features, out_features, std_init=0.4):
        super(NoisyLinear, self).__init__()

        self.in_features = in_features
        self.out_features = out_features
        self.std_init = std_init

        # Весовые параметры
        self.weight_mu = nn.Parameter(torch.Tensor(out_features, in_features))
        self.weight_sigma = nn.Parameter(torch.Tensor(out_features, in_features))
        self.register_buffer('weight_epsilon', torch.Tensor(out_features, in_features))

        # Смещения
        self.bias_mu = nn.Parameter(torch.Tensor(out_features))
        self.bias_sigma = nn.Parameter(torch.Tensor(out_features))
        self.register_buffer('bias_epsilon', torch.Tensor(out_features))

        self.reset_parameters()
        self.reset_noise()

    def reset_parameters(self):
        """Инициализация параметров"""
        mu_range = 1 / np.sqrt(self.in_features)

        self.weight_mu.data.uniform_(-mu_range, mu_range)
        self.weight_sigma.data.fill_(self.std_init / np.sqrt(self.in_features))

        self.bias_mu.data.uniform_(-mu_range, mu_range)
        self.bias_sigma.data.fill_(self.std_init / np.sqrt(self.out_features))

    def reset_noise(self):
        """Сброс шума"""
        epsilon_in = self._scale_noise(self.in_features)
        epsilon_out = self._scale_noise(self.out_features)

        self.weight_epsilon.copy_(epsilon_out.ger(epsilon_in))
        self.bias_epsilon.copy_(epsilon_out)

    def _scale_noise(self, size):
        """Генерация шума"""
        x = torch.randn(size)
        return x.sign().mul(x.abs().sqrt())

    def forward(self, x):
        """Прямой проход с шумом"""
        if self.training:
            weight = self.weight_mu + self.weight_sigma * self.weight_epsilon
            bias = self.bias_mu + self.bias_sigma * self.bias_epsilon
        else:
            weight = self.weight_mu
            bias = self.bias_mu

        return nn.functional.linear(x, weight, bias)

class NoisyDuelingQNetwork(nn.Module):
    """
    NoisyDuealing DQN сеть

    Особенности:
    1. Dueling Architecture (Value + Advantage streams)
    2. Noisy Layers для exploration
    3. Не требует ε-greedy
    """

    def __init__(self, n_warehouses, hidden_size=256):
        super(NoisyDuelingQNetwork, self).__init__()
        self.n_warehouses = n_warehouses

        # Энкодер С noisy layers
        self.encoder = nn.Sequential(
            NoisyLinear(n_warehouses * 2, hidden_size),
            nn.ReLU(),
            NoisyLinear(hidden_size, hidden_size),
            nn.ReLU()
        )

        # Value stream
        self.value_stream = nn.Sequential(
            NoisyLinear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            NoisyLinear(hidden_size // 2, 1)
        )

        # Advantage stream
        self.advantage_stream = nn.Sequential(
            NoisyLinear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            NoisyLinear(hidden_size // 2, n_warehouses)
        )

    def forward(self, state):
        current = state['current']
        visited_mask = state['visited_mask']

        current_onehot = torch.zeros(current.shape[0], self.n_warehouses, dtype=torch.float32)
        for i, c in enumerate(current):
            current_onehot[i, c] = 1.0

        x = torch.cat([current_onehot, visited_mask], dim=1)
        features = self.encoder(x)

        value = self.value_stream(features)
        advantage = self.advantage_stream(features)

        # Dueling формула
        q_values = value + advantage - advantage.mean(dim=1, keepdim=True)

        # Маскируем посещенные
        q_values[visited_mask.bool()] = -1e8

        return q_values

    def reset_noise(self):
        """Сброс шума во всех Noisy слоях"""
        for module in self.modules():
            if isinstance(module, NoisyLinear):
                module.reset_noise()

In [ ]:
class ImprovedDQNAgent:
    """
    Улучшенный DQN агент
    """

    def __init__(self, n_warehouses, learning_rate=0.0003, gamma=0.99,
                 tau=0.005, use_per=True, use_noisy=True, n_step=3):
        """
        Инициализация улучшенного DQN
        """

        # Выбираем тип сети
        if use_noisy:
            self.q_network = NoisyDuelingQNetwork(n_warehouses)
            self.target_network = NoisyDuelingQNetwork(n_warehouses)
        else:
            self.q_network = DuelingQNetwork(n_warehouses)
            self.target_network = DuelingQNetwork(n_warehouses)

        # Копируем веса в target сеть
        self.target_network.load_state_dict(self.q_network.state_dict())

        # Оптимизатор И Scheduler
        self.optimizer = optim.AdamW(self.q_network.parameters(),
                                    lr=learning_rate, weight_decay=1e-4)
        self.scheduler = optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max=1500)

        # Гиперпараметры
        self.gamma = gamma
        self.tau = tau
        self.n_step = n_step
        self.use_noisy = use_noisy
        self.n_warehouses = n_warehouses

        # Replay buffer
        self.use_per = use_per
        if use_per:
            self.memory = PrioritizedReplayBuffer(capacity=10000, alpha=0.6)
            self.beta = 0.4
            self.beta_increment = 0.001
        else:
            self.memory = deque(maxlen=10000)
            self.n_step_buffer = deque(maxlen=n_step)  # Для N-step returns

        self.batch_size = 128

        # Статистика
        self.update_step = 0
        self.losses = []

        # Для Noisy Networks не нужен ε-greedy
        if use_noisy:
            self.epsilon = 0.0
        else:
            self.epsilon = 1.0
            self.epsilon_decay = 0.995
            self.epsilon_min = 0.01

    def select_action(self, state):
        """
        Выбор действия

        Если используем Noisy Networks:
        - Шум уже встроен в веса сети
        - Не нужен ε-greedy

        Если используем обычный DQN:
        - Используем ε-greedy для exploration
        """

        if not self.use_noisy and random.random() < self.epsilon:
            # ε-greedy: случайное действие
            visited_mask = state['visited_mask']
            valid_actions = [i for i in range(len(visited_mask)) if visited_mask[i] == 0.0]
            return random.choice(valid_actions) if valid_actions else 0

        # Жадное действие на основе Q-значений
        state_tensor = {
            'current': torch.tensor([state['current']], dtype=torch.long),
            'visited_mask': torch.tensor([state['visited_mask']], dtype=torch.float32)
        }

        with torch.no_grad():
            q_values = self.q_network(state_tensor)
            return q_values.argmax().item()

    def store_experience(self, state, action, reward, next_state, done):
        """
        Сохранение опыта в Replay buffer
        """
        experience = (state, action, reward, next_state, done)

        if self.use_per:
            # Для PER вычисляем TD ошибку для приоритета
            state_tensor = {
                'current': torch.tensor([state['current']], dtype=torch.long),
                'visited_mask': torch.tensor([state['visited_mask']], dtype=torch.float32)
            }

            with torch.no_grad():
                current_q = self.q_network(state_tensor)[0, action]

                next_state_tensor = {
                    'current': torch.tensor([next_state['current']], dtype=torch.long),
                    'visited_mask': torch.tensor([next_state['visited_mask']], dtype=torch.float32)
                }

                # Double DQN: online сеть выбирает, target сеть оценивает
                next_action = self.q_network(next_state_tensor).argmax().item()
                next_q = self.target_network(next_state_tensor)[0, next_action]

                target_q = reward + self.gamma * next_q * (1 - done)
                td_error = abs(target_q - current_q).item()

            # Сохраняем с приоритетом
            self.memory.add(experience, td_error)
        else:
            # Обычное сохранение
            self.memory.append(experience)

    def update(self):
        """
        Обновление сети
        """

        # Проверяем, достаточно ли данных в buffer
        if len(self.memory) < self.batch_size:
            return

        # Sampling
        if self.use_per:
            batch, indices, weights = self.memory.sample(self.batch_size, self.beta)

            # Проверяем, что получили достаточное количество примеров
            if len(batch) < self.batch_size:
                return

            states, actions, rewards, next_states, dones = zip(*batch)
            weights = torch.tensor(weights, dtype=torch.float32)

            # Увеличиваем beta
            self.beta = min(1.0, self.beta + self.beta_increment)
        else:
            batch = random.sample(list(self.memory), min(self.batch_size, len(self.memory)))

            if len(batch) < self.batch_size:
                return

            states, actions, rewards, next_states, dones = zip(*batch)
            weights = torch.ones(len(batch))

        # Подготовка тензоров
        state_current = torch.tensor([s['current'] for s in states], dtype=torch.long)
        state_visited = torch.tensor([s['visited_mask'] for s in states], dtype=torch.float32)

        next_state_current = torch.tensor([s['current'] for s in next_states], dtype=torch.long)
        next_state_visited = torch.tensor([s['visited_mask'] for s in next_states], dtype=torch.float32)

        state_tensors = {'current': state_current, 'visited_mask': state_visited}
        next_state_tensors = {'current': next_state_current, 'visited_mask': next_state_visited}

        actions_tensor = torch.tensor(actions)
        rewards_tensor = torch.tensor(rewards, dtype=torch.float32)
        dones_tensor = torch.tensor(dones, dtype=torch.float32)

        # Вычисление target Q-значений (Double DQN)
        with torch.no_grad():
            # Online сеть выбирает действия
            next_actions = self.q_network(next_state_tensors).argmax(1)

            # Target сеть оценивает выбранные действия
            next_q_values = self.target_network(next_state_tensors)
            next_q_values = next_q_values[torch.arange(len(batch)), next_actions]

            # Вычисляем target Q-значения
            target_q = rewards_tensor + self.gamma * next_q_values * (1 - dones_tensor)

        # Вычисление текущих Q-значений
        current_q_values = self.q_network(state_tensors)
        current_q_values = current_q_values[torch.arange(len(batch)), actions_tensor]

        # Вычисление потери
        td_errors = target_q - current_q_values

        if self.use_per:
            # Для PER: взвешенная MSE
            loss = (weights * td_errors.pow(2)).mean()
        else:
            # Обычная MSE
            loss = td_errors.pow(2).mean()

        # Градиентный спуск
        self.optimizer.zero_grad()
        loss.backward()

        # Gradient clipping для стабильности
        torch.nn.utils.clip_grad_norm_(self.q_network.parameters(), max_norm=1.0)

        self.optimizer.step()
        self.scheduler.step()

        # Обновление Target Сети (Soft update)
        self._soft_update_target_network()

        # Обновление приоритетов (для PER)
        if self.use_per:
            td_errors_np = td_errors.detach().abs().numpy()
            self.memory.update_priorities(indices, td_errors_np)

        # Сброс шума (для noisy network)
        if self.use_noisy and hasattr(self.q_network, 'reset_noise'):
            self.q_network.reset_noise()
            self.target_network.reset_noise()

        # Decay ε (для обыченого DQN)
        if not self.use_noisy:
            self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

        # Сохраниение статистики
        self.update_step += 1
        self.losses.append(loss.item())

        return loss.item()

    def _soft_update_target_network(self):
        """
        Плавное обновление target сети
        θ_target = τ * θ_online + (1 - τ) * θ_target
        """
        for target_param, param in zip(self.target_network.parameters(),
                                      self.q_network.parameters()):
            target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)

In [ ]:
class ImprovedDoubleDQNAgent(ImprovedDQNAgent):
    """
    Улучшенный DOUBLE DQN aгент

    Наследует от ImprovedDQNAgent, но переопределяет:
    1. Более консервативный target update
    2. Дополнительные техники для стабильности

    Отличие от ImprovedDQNAgent:
    - Более частая инициализация target сети
    - Больший batch size
    - Дополнительная регуляризация
    """

    def __init__(self, n_warehouses, learning_rate=0.0003, gamma=0.99,
                 tau=0.001, use_per=True, use_noisy=True, n_step=3):
        super().__init__(n_warehouses, learning_rate, gamma, tau, use_per, use_noisy, n_step)

        # Больше regularization для DOUBLE DQN
        self.batch_size = 256  # Больше batch size для стабильности
        self.tau = tau  # Меньше tau для более медленного обновления

        # дополнительная regularization
        self.gradient_clip = 0.5  # Меньше clipping

    def update(self):
        """
        Обновление DOUBLE DQN С дополнительными мерами:
        1. Более частый hard update target сети
        2. Дополнительная регуляризация
        3. Более консервативное обучение
        """

        loss = super().update()

        # Переодически hard update
        # Каждые 1000 шагов полностью копируем веса
        if self.update_step % 1000 == 0:
            self.target_network.load_state_dict(self.q_network.state_dict())

        return loss

In [ ]:
def train_all_rl_methods(distance_matrix, warehouses_df, num_episodes=1500):
    """
    Обучение RL методов
    """

    env = TSPEnvironment(distance_matrix, warehouses_df)
    n_warehouses = len(warehouses_df)

    agents = {
        # Policy Gradient методы
        'Policy Gradient (базовый)': PolicyGradientAgent(n_warehouses),
        'Policy Gradient (улучшенный)': EnhancedPolicyGradientAgent(n_warehouses),

        # Actor-Critic методы
        'A2C (базовый)': A2CAgent(n_warehouses),
        'A2C (улучшенный)': EnhancedA2CAgent(n_warehouses),

        # PPO методы
        'PPO (базовый)': PPOAgent(n_warehouses),
        'PPO (улучшенный)': EnhancedPPOAgent(n_warehouses),

        # DQN методы
        'DQN (базовый)': DQNAgent(n_warehouses),
        'Double DQN (базовый)': DoubleDQNAgent(n_warehouses),
        'DQN (улучшенный)': ImprovedDQNAgent(n_warehouses, use_per=True, use_noisy=False),
        'Double DQN (улучшенный)': ImprovedDoubleDQNAgent(n_warehouses, use_per=True, use_noisy=False),

        # SAC методы
        'SAC (базовый)': SACAgent(n_warehouses),
        'SAC (улучшенный)': EnhancedSACAgent(n_warehouses),
    }

    results = {}

    print(f"Начинаем обучение {len(agents)} RL методов")
    print(f"Количество складов: {n_warehouses}")
    print(f"Количество эпизодов на метод: {num_episodes}")
    print("=" * 80)

    # Обучаем каждого агента
    for agent_name, agent in agents.items():
        print(f"\nОбучение: {agent_name}")
        print("-" * 60)

        start_time = time.time()

        episode_rewards = []
        episode_distances = []
        best_route = None
        best_distance = float('inf')

        for episode in range(num_episodes):
            state = env.reset()
            episode_reward = 0.0
            done = False
            step_count = 0
            max_steps = n_warehouses * 2

            while not done and step_count < max_steps:
                # Выбор действия
                if 'SAC' in agent_name:
                    # SAC возвращает и действие, и log_prob
                    action_result = agent.select_action(state)
                    if isinstance(action_result, tuple):
                        action = action_result[0]  # Берем только действие
                    else:
                        action = action_result
                else:
                    action = agent.select_action(state)

                # Шаг в среде
                next_state, reward, done, _ = env.step(action)
                episode_reward += reward

                # Сохраняем опыт (для методов с replay buffer)
                if hasattr(agent, 'store_experience'):
                    agent.store_experience(state, action, reward, next_state, done)

                # Сохраняем награды (для on-policy методов)
                if hasattr(agent, 'rewards'):
                    agent.rewards.append(reward)

                state = next_state
                step_count += 1

            # Обновление политики
            # Для PPO обновляем с финальной наградой
            if 'PPO' in agent_name:
                agent.update(episode_reward)
            # Для остальных on-policy методов обновляем каждые 10 эпизодов
            elif hasattr(agent, 'update') and not hasattr(agent, 'store_experience'):
                if episode % 10 == 0 or done:
                    agent.update()
            # Для off-policy методов обновляем всегда
            elif hasattr(agent, 'update'):
                agent.update()

            # Статистика
            route_info = env.get_route_info()
            current_distance = route_info['distance_km']

            episode_rewards.append(episode_reward)
            episode_distances.append(current_distance)

            if current_distance < best_distance and len(env.visited) == n_warehouses:
                best_distance = current_distance
                best_route = route_info['route']

            # Прогресс
            if episode % 100 == 0:
                visited_count = len(env.visited)
                epsilon = getattr(agent, 'epsilon', 0)
                print(f"  Эпизод {episode:4d}: {current_distance:7.2f} км, "
                      f"ε={epsilon:.3f}, Посещено: {visited_count}/{n_warehouses}")

        # Результаты
        end_time = time.time()
        training_time = end_time - start_time

        results[agent_name] = {
            'best_distance': best_distance,
            'best_route': best_route,
            'avg_last_100': np.mean(episode_distances[-100:]) if episode_distances else float('inf'),
            'std_last_100': np.std(episode_distances[-100:]) if len(episode_distances) >= 100 else 0,
            'training_time_seconds': training_time,
            'training_time_minutes': training_time / 60,
            'episode_distances': episode_distances,
            'episode_rewards': episode_rewards
        }

        print(f"\n{agent_name} завершен")
        print(f"   Лучший результат: {best_distance:.2f} км")
        print(f"   Среднее (последние 100): {results[agent_name]['avg_last_100']:.2f} км")
        print(f"   Время обучения: {training_time:.2f} секунд")

    return results

In [ ]:
distance_matrix, warehouses_df = load_data()

In [ ]:
print(f"Загружено складов: {len(warehouses_df)}")
print(f"Размер матрицы расстояний: {distance_matrix.shape}")

print("Количество эпизодов на метод: 1500")
print("=" * 80)

# Запускаем обучение всех методов
all_results = train_all_rl_methods(distance_matrix, warehouses_df, num_episodes=1500)

In [ ]:
summary_data = []
for agent_name, result in all_results.items():
    summary_data.append({
        'Метод': agent_name,
        'Лучший результат (км)': f"{result['best_distance']:.2f}",
        'Среднее последние 100 (км)': f"{result['avg_last_100']:.2f}",
        'Станд. отклонение': f"{result['std_last_100']:.2f}",
        'Время обучения (мин)': f"{result['training_time_minutes']:.2f}",
        'Время на эпизод (сек)': f"{result['training_time_seconds']/1500:.3f}"
    })

summary_df = pd.DataFrame(summary_data)

# Сортируем по лучшему результату
summary_df = summary_df.sort_values(by='Лучший результат (км)', key=lambda x: x.str.replace(' км', '').astype(float))

summary_df

In [ ]:
def plot_comprehensive_results(results):
    """Комплексная визуализация всех результатов"""
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(18, 12))

    # График 1: Лучшие результаты (горизонтальные столбцы)
    methods = list(results.keys())
    best_distances = [results[name]['best_distance'] for name in methods]

    colors = []
    for name in methods:
        if 'улучшенный' in name:
            colors.append('green')
        elif 'базовый' in name:
            colors.append('red')
        else:
            colors.append('blue')

    bars = ax1.barh(methods, best_distances, color=colors, alpha=0.7)
    ax1.set_xlabel('Лучшее расстояние (км)')
    ax1.set_title('Лучшие результаты всех методов\n(чем меньше, тем лучше)', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3, axis='x')

    for bar, distance in zip(bars, best_distances):
        width = bar.get_width()
        ax1.text(width + max(best_distances)*0.01, bar.get_y() + bar.get_height()/2,
                f'{distance:.1f} км', ha='left', va='center', fontsize=9)

    # График 2: Время обучения в секундах
    training_times = [results[name]['training_time_seconds'] for name in methods]
    bars2 = ax2.bar(methods, training_times, color='orange', alpha=0.7)
    ax2.set_ylabel('Время обучения (секунды)', fontsize=12)
    ax2.set_title('Время обучения методов', fontsize=14, fontweight='bold')
    ax2.tick_params(axis='x', rotation=90, labelsize=9)
    ax2.grid(True, alpha=0.3, axis='y')

    # Добавляем значения на столбцы
    for bar, time_val in zip(bars2, training_times):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height + max(training_times)*0.01,
                f'{time_val:.1f} сек', ha='center', va='bottom', fontsize=8)

    # График 3: Скорость сходимости (время до лучшего результата)
    # Считаем эпизод, на котором был достигнут лучший результат
    convergence_data = []
    for name in methods:
        episode_distances = results[name]['episode_distances']
        if episode_distances:
            best_dist = results[name]['best_distance']
            # Находим первый эпизод с таким результатом
            convergence_episode = None
            for i, dist in enumerate(episode_distances):
                if abs(dist - best_dist) < 0.1:  # Учитываем погрешность
                    convergence_episode = i
                    break
            if convergence_episode is not None:
                convergence_data.append(convergence_episode)
            else:
                convergence_data.append(len(episode_distances))
        else:
            convergence_data.append(0)

    bars3 = ax3.bar(methods, convergence_data, color='purple', alpha=0.7)
    ax3.set_ylabel('Эпизод сходимости', fontsize=12)
    ax3.set_title('Скорость сходимости методов\n(чем меньше, тем быстрее)', fontsize=14, fontweight='bold')
    ax3.tick_params(axis='x', rotation=45, labelsize=9)
    ax3.grid(True, alpha=0.3, axis='y')

    # Добавляем значения
    for bar, conv_ep in zip(bars3, convergence_data):
        height = bar.get_height()
        ax3.text(bar.get_x() + bar.get_width()/2., height + max(convergence_data)*0.01,
                f'{conv_ep}', ha='center', va='bottom', fontsize=8)

    # График 4: Сравнение базовых и улучшенных версий
    base_methods = [name for name in methods if 'базовый' in name]

    base_distances = []
    enhanced_distances = []
    comparison_labels = []

    for base_name in base_methods:
        enhanced_name = base_name.replace(' (базовый)', ' (улучшенный)')
        if enhanced_name in results:
            base_distances.append(results[base_name]['best_distance'])
            enhanced_distances.append(results[enhanced_name]['best_distance'])
            comparison_labels.append(base_name.replace(' (базовый)', ''))

    if base_distances:
        x = np.arange(len(comparison_labels))
        width = 0.35

        bars_base = ax4.bar(x - width/2, base_distances, width,
                           label='Базовая версия', alpha=0.7, color='red')
        bars_enhanced = ax4.bar(x + width/2, enhanced_distances, width,
                               label='Улучшенная версия', alpha=0.7, color='green')

        ax4.set_ylabel('Лучший результат (км)', fontsize=12)
        ax4.set_title('Сравнение базовых и улучшенных версий', fontsize=14, fontweight='bold')
        ax4.set_xticks(x)
        ax4.set_xticklabels(comparison_labels, rotation=45, ha='right', fontsize=9)
        ax4.legend()
        ax4.grid(True, alpha=0.3, axis='y')

        # Добавляем процент улучшения
        for i, (base_dist, enhanced_dist) in enumerate(zip(base_distances, enhanced_distances)):
            improvement = ((base_dist - enhanced_dist) / base_dist) * 100
            color = 'green' if improvement > 0 else 'red'
            ax4.text(i, max(base_dist, enhanced_dist) + max(base_distances)*0.05,
                    f'{improvement:+.1f}%', ha='center', va='bottom',
                    color=color, fontweight='bold', fontsize=9)
    else:
        ax4.text(0.5, 0.5, 'Нет данных для сравнения\nбазовых и улучшенных версий',
                ha='center', va='center', transform=ax4.transAxes, fontsize=12)
        ax4.set_title('Сравнение недоступно', fontsize=14, fontweight='bold')

    plt.tight_layout()
    plt.savefig('rl_methods_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()

    print("\nВизуализация сохранена: 'rl_methods_comparison.png'")

# Визуализация
plot_comprehensive_results(all_results)

In [ ]:
improvement_data = []

for agent_name in all_results.keys():
    if ' (улучшенный)' in agent_name:
        base_name = agent_name.replace(' (улучшенный)', ' (базовый)')

        if base_name in all_results:
            base_result = all_results[base_name]['best_distance']
            enhanced_result = all_results[agent_name]['best_distance']

            improvement = ((base_result - enhanced_result) / base_result) * 100

            base_time = all_results[base_name]['training_time_seconds']
            enhanced_time = all_results[agent_name]['training_time_seconds']
            time_change = ((enhanced_time - base_time) / base_time) * 100

            improvement_data.append({
                'Метод': agent_name.replace(' (улучшенный)', ''),
                'Базовый (км)': f"{base_result:.2f}",
                'Улучшенный (км)': f"{enhanced_result:.2f}",
                'Улучшение (%)': f"{improvement:+.2f}",
                'Время базовый (сек)': f"{base_time:.1f}",
                'Время улучшенный (сек)': f"{enhanced_time:.1f}",
                'Изменение времени (%)': f"{time_change:+.1f}",
            })

if improvement_data:
    improvement_df = pd.DataFrame(improvement_data)

    # Сортируем по улучшению результата (от большего к меньшему)
    improvement_df['Улучшение числ'] = improvement_df['Улучшение (%)'].str.replace('%', '').astype(float)
    improvement_df = improvement_df.sort_values('Улучшение числ', ascending=False)
    improvement_df = improvement_df.drop('Улучшение числ', axis=1)

    print("\nСравнение базовых и улучшенных версий:")
    print("-" * 80)
    print(improvement_df.to_string(index=False))

else:
    print("Нет данных для сравнения базовых и улучшенных версий")

## RL c шумом

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import time
from collections import defaultdict

In [ ]:
def train_and_test_on_noise_scenarios(num_episodes=1500):
    """
    Обучение и тестирование RL агентов на сценариях с шумом

    Что делаем:
    1. Загружаем 4 сценария: без шума, пробки, блокировки+пробки, умеренные условия
    2. Для каждого сценария вычисляем матрицу расстояний между складами
    3. Обучаем выбранные RL агенты на каждом сценарии (1500 эпизодов)
    4. Анализируем устойчивость методов к разным типам шума
    5. Сравниваем результаты и выявляем лучшие методы для каждого сценария
    """

    print(f"Количество эпизодов на метод: {num_episodes}")
    print("=" * 80)

    # Загружаем сценарии с шумом (такие же как и в классическом)
    try:
        scenarios = {
            "Без шума": graph,
            "Пробки": graph_traffic_jam,
            "Блокировки+Пробки": graph_combined,
            "Умеренные условия": graph_moderate
        }
        print("Сценарии с шумом успешно загружены")
    except NameError:
        print("Ошибка: сценарии с шумом не определены.")
        return None, None

    # Загружаем данные складов
    print("\nЗагружаем данные складов")
    with open('warehouses_rc_rfc_coordinates.json', 'r', encoding='utf-8') as f:
        warehouses_json = json.load(f)

    warehouses_data = []
    for warehouse in warehouses_json:
        warehouses_data.append({
            "name": warehouse['name'],
            "latitude": warehouse['latitude'],
            "longitude": warehouse['longitude']
        })

    warehouses_df = pd.DataFrame(warehouses_data)
    n_warehouses = len(warehouses_df)

    # Привязываем склады к узлам графа
    print(f"Привязываем {n_warehouses} складов к дорожной сети")
    warehouse_nodes = []
    for idx, warehouse in warehouses_df.iterrows():
        point = (warehouse['latitude'], warehouse['longitude'])
        nearest_node = ox.distance.nearest_nodes(graph, point[1], point[0])
        warehouse_nodes.append(nearest_node)

    warehouses_df['node_id'] = warehouse_nodes

    # Создаем матрицы расстояний для каждого сценария
    print("\nВычисляем матрицы расстояний для всех сценариев")
    scenario_matrices = {}

    for scenario_name, scenario_graph in scenarios.items():
        print(f"  Сценарий: {scenario_name}")

        n_warehouses = len(warehouse_nodes)
        distance_matrix = np.zeros((n_warehouses, n_warehouses))

        for i in range(n_warehouses):
            for j in range(i+1, n_warehouses):
                try:
                    # Проверяем, что узлы существуют в графе
                    if warehouse_nodes[i] not in scenario_graph.nodes or warehouse_nodes[j] not in scenario_graph.nodes:
                        distance_matrix[i][j] = 1e9
                        distance_matrix[j][i] = 1e9
                        continue

                    distance = nx.shortest_path_length(scenario_graph, warehouse_nodes[i], warehouse_nodes[j], weight='length')
                    distance_matrix[i][j] = distance
                    distance_matrix[j][i] = distance
                except nx.NetworkXNoPath:
                    distance_matrix[i][j] = 1e9
                    distance_matrix[j][i] = 1e9

        scenario_matrices[scenario_name] = distance_matrix.astype(np.float32)
        print(f"    Завершено")

    # Оставленные методы
    selected_algorithms = {
        'SAC (улучшенный)': lambda n: EnhancedSACAgent(n),
        'DQN (улучшенный)': lambda n: ImprovedDQNAgent(n, use_per=True, use_noisy=False),
        'Double DQN (улучшенный)': lambda n: ImprovedDoubleDQNAgent(n, use_per=True, use_noisy=False),
        'PPO (базовый)': lambda n: PPOAgent(n),
        'A2C (улучшенный)': lambda n: EnhancedA2CAgent(n),
        'Policy Gradient (базовый)': lambda n: PolicyGradientAgent(n)
    }

    # Словарь для хранения результатов
    all_scenario_results = {}
    training_times = {}

    # Обучаем и тестируем каждый алгоритм на каждом сценарии
    for scenario_name, distance_matrix in scenario_matrices.items():
        print(f"\n{'='*80}")
        print(f"Сценарий: {scenario_name}")
        print(f"{'='*80}")

        scenario_results = {}

        for agent_name, agent_constructor in selected_algorithms.items():
            print(f"\n  Обучаем: {agent_name}")
            print(f"  {'-'*60}")

            start_time = time.time()

            # Создаем агента и среду
            agent = agent_constructor(n_warehouses)
            env = TSPEnvironment(distance_matrix, warehouses_df)

            # Обучение
            episode_distances = []
            episode_rewards = []
            best_distance = float('inf')
            best_route = None

            for episode in range(num_episodes):
                state = env.reset()
                episode_reward = 0.0
                done = False

                step_count = 0
                max_steps = n_warehouses * 2

                while not done and step_count < max_steps:
                    # Выбор действия (особенность SAC)
                    if 'SAC' in agent_name:
                        action_result = agent.select_action(state)
                        if isinstance(action_result, tuple):
                            action = action_result[0]  # SAC возвращает (action, log_prob)
                        else:
                            action = action_result
                    else:
                        action = agent.select_action(state)

                    # Шаг в среде
                    next_state, reward, done, _ = env.step(action)
                    episode_reward += reward

                    # Сохраняем опыт для off-policy методов
                    if hasattr(agent, 'store_experience'):
                        agent.store_experience(state, action, reward, next_state, done)

                    # Сохраняем награды для on-policy методов
                    if hasattr(agent, 'rewards'):
                        agent.rewards.append(reward)

                    state = next_state
                    step_count += 1

                # Обновление политики
                # Для PPO передаем финальную награду
                if 'PPO' in agent_name:
                    agent.update(episode_reward)
                # Для остальных on-policy методов обновляем каждые 10 эпизодов
                elif hasattr(agent, 'update') and not hasattr(agent, 'store_experience'):
                    if episode % 10 == 0 or done:
                        agent.update()
                # Для off-policy методов обновляем всегда
                elif hasattr(agent, 'update'):
                    agent.update()

                # Сохраняем результаты
                route_info = env.get_route_info()
                current_distance = route_info['distance_km']

                episode_distances.append(current_distance)
                episode_rewards.append(episode_reward)

                if current_distance < best_distance and len(env.visited) == n_warehouses:
                    best_distance = current_distance
                    best_route = route_info['route']

                # Прогресс каждые 150 эпизодов
                if episode % 150 == 0:
                    visited_count = len(env.visited)
                    epsilon = getattr(agent, 'epsilon', 0)
                    print(f"    Эпизод {episode:4d}: {current_distance:7.2f} км, "
                          f"ε={epsilon:.3f}, Посещено: {visited_count}/{n_warehouses}")

            end_time = time.time()
            training_time = end_time - start_time

            # Сохраняем результаты для этого агента
            scenario_results[agent_name] = {
                'best_distance': best_distance,
                'best_route': best_route,
                'avg_last_100': np.mean(episode_distances[-100:]) if len(episode_distances) >= 100 else float('inf'),
                'std_last_100': np.std(episode_distances[-100:]) if len(episode_distances) >= 100 else 0,
                'avg_all': np.mean(episode_distances),
                'std_all': np.std(episode_distances),
                'training_time_seconds': training_time,
                'episode_distances': episode_distances,
                'episode_rewards': episode_rewards
            }

            training_times[(scenario_name, agent_name)] = training_time

            print(f"    Завершено: {best_distance:.2f} км за {training_time:.1f} сек")

        all_scenario_results[scenario_name] = scenario_results

    return all_scenario_results, scenario_matrices, training_times

In [ ]:
def create_summary_tables(all_scenario_results):
    """
    Создает сводные таблицы результатов по всем сценариям

    Возвращает:
    - summary_df: полная таблица
    - pivot_best: сводная таблица лучших результатов
    - pivot_avg: сводная таблица средних результатов
    - pivot_time: сводная таблица времени обучения
    """

    scenario_names = list(all_scenario_results.keys())
    algorithm_names = list(all_scenario_results[scenario_names[0]].keys())

    print("\nСводные таблицы")
    print("-" * 120)

    # Создаем полную таблицу
    summary_data = []
    for scenario_name in scenario_names:
        for algorithm_name in algorithm_names:
            result = all_scenario_results[scenario_name][algorithm_name]
            summary_data.append({
                'Сценарий': scenario_name,
                'Метод': algorithm_name,
                'Лучший результат (км)': f"{result['best_distance']:.2f}",
                'Среднее последние 100 (км)': f"{result['avg_last_100']:.2f}",
                'Станд. отклонение': f"{result['std_last_100']:.2f}",
                'Время обучения (сек)': f"{result['training_time_seconds']:.2f}",
                'Время на эпизод (мс)': f"{result['training_time_seconds']*1000/1500:.1f}"
            })

    summary_df = pd.DataFrame(summary_data)

    # Создаем pivot таблицы
    pivot_best = summary_df.pivot(index='Метод', columns='Сценарий', values='Лучший результат (км)')
    pivot_avg = summary_df.pivot(index='Метод', columns='Сценарий', values='Среднее последние 100 (км)')
    pivot_time = summary_df.pivot(index='Метод', columns='Сценарий', values='Время обучения (сек)')

    # Выводим таблицы
    print("\nЛучшие результаты:")
    print("-" * 80)
    print(pivot_best)

    print("\nСредние результаты (последние 100 эпизодов):")
    print("-" * 80)
    print(pivot_avg)

    print("\nВремя обучения (секунды):")
    print("-" * 80)
    print(pivot_time)

    return summary_df, pivot_best, pivot_avg, pivot_time

In [ ]:
def plot_results_by_scenario(all_scenario_results):
    """
    Создает графики с результатами по сценариям
    Ось X: сценарии
    Столбцы: методы
    """

    scenario_names = list(all_scenario_results.keys())
    algorithm_names = list(all_scenario_results[scenario_names[0]].keys())

    # Сокращенные имена методов для подписей
    short_names = []
    for name in algorithm_names:
        short = name.replace(' (базовый)', ' (б.)').replace(' (улучшенный)', ' (у.)')
        short_names.append(short)

    # Создаем комплексные графики
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(18, 14))

    # 1. Лучшие результаты по сценариям
    # Ось X: сценарии, столбцы: методы

    # Подготовка данных
    best_results = {}
    for scenario_name in scenario_names:
        best_results[scenario_name] = [
            all_scenario_results[scenario_name][algo]['best_distance']
            for algo in algorithm_names
        ]

    # Создаем данные для группированных столбцов
    x = np.arange(len(scenario_names))  # позиции по оси X для сценариев
    width = 0.1  # ширина столбца для каждого метода

    # Разные цвета для разных категорий методов
    color_palette = plt.cm.Set3(np.linspace(0, 1, len(algorithm_names)))

    for i, (algo_name, short_name) in enumerate(zip(algorithm_names, short_names)):
        algo_values = [best_results[scenario][i] for scenario in scenario_names]
        ax1.bar(x + i*width - width*(len(algorithm_names)-1)/2,
               algo_values, width, label=short_name, color=color_palette[i], alpha=0.8)

    ax1.set_xlabel('Сценарии', fontsize=12)
    ax1.set_ylabel('Лучшее расстояние (км)', fontsize=12)
    ax1.set_title('Лучшие результаты методов RL по сценариям\n(чем меньше, тем лучше)',
                  fontsize=14, fontweight='bold')
    ax1.set_xticks(x)
    ax1.set_xticklabels(scenario_names, rotation=90, fontsize=11)
    ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    ax1.grid(True, alpha=0.3)
    ax1.tick_params(axis='x', rotation=90)

    # Добавляем значения на столбцы (только минимальные для каждого сценария)
    for j, scenario in enumerate(scenario_names):
        min_value = min(best_results[scenario])
        min_index = best_results[scenario].index(min_value)
        ax1.text(j + min_index*width - width*(len(algorithm_names)-1)/2,
                min_value - max(min_value*0.02, 1),
                f'{min_value:.1f}', ha='center', va='top', fontsize=8,
                fontweight='bold', color='red')

    # 2. Средние результаты по сценариям
    avg_results = {}
    for scenario_name in scenario_names:
        avg_results[scenario_name] = [
            all_scenario_results[scenario_name][algo]['avg_last_100']
            for algo in algorithm_names
        ]

    for i, (algo_name, short_name) in enumerate(zip(algorithm_names, short_names)):
        algo_values = [avg_results[scenario][i] for scenario in scenario_names]
        ax2.bar(x + i*width - width*(len(algorithm_names)-1)/2,
               algo_values, width, label=short_name, color=color_palette[i], alpha=0.8)

    ax2.set_xlabel('Сценарии', fontsize=12)
    ax2.set_ylabel('Среднее расстояние (км)', fontsize=12)
    ax2.set_title('Средние результаты методов RL (последние 100 эпизодов)',
                  fontsize=14, fontweight='bold')
    ax2.set_xticks(x)
    ax2.set_xticklabels(scenario_names, rotation=90, fontsize=11)
    ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    ax2.grid(True, alpha=0.3)
    ax2.tick_params(axis='x', rotation=90)

    # График 3. Ухудшение для сценариев
    base_scenario = "Без шума"
    performance_degradation = {}

    # Рассчитываем ухудшение для каждого сценария с шумом
    for scenario_name in scenario_names:
        if scenario_name == base_scenario:
            continue

        degradation = []
        for algo_name in algorithm_names:
            base_result = all_scenario_results[base_scenario][algo_name]['best_distance']
            current_result = all_scenario_results[scenario_name][algo_name]['best_distance']
            degradation_pct = ((current_result - base_result) / base_result) * 100
            degradation.append(degradation_pct)

        performance_degradation[scenario_name] = degradation

    # Создаем подграфик для ухудшения
    noise_scenarios = [s for s in scenario_names if s != base_scenario]
    x_degradation = np.arange(len(noise_scenarios))

    if performance_degradation:
        # Разные цвета для ухудшения (красные оттенки)
        degradation_colors = plt.cm.Reds(np.linspace(0.3, 0.9, len(algorithm_names)))

        for i, (algo_name, short_name) in enumerate(zip(algorithm_names, short_names)):
            algo_values = [performance_degradation[scenario][i] for scenario in noise_scenarios]
            bars = ax3.bar(x_degradation + i*width - width*(len(algorithm_names)-1)/2,
                          algo_values, width, label=short_name,
                          color=degradation_colors[i], alpha=0.8)

            # Добавляем значения на столбцы
            for j, value in enumerate(algo_values):
                ax3.text(x_degradation[j] + i*width - width*(len(algorithm_names)-1)/2,
                        value + (1 if value >= 0 else -3),
                        f'{value:.1f}%', ha='center',
                        va='bottom' if value >= 0 else 'top', fontsize=7)

    ax3.set_xlabel('Сценарии с шумом', fontsize=12)
    ax3.set_ylabel('Ухудшение производительности (%)', fontsize=12)
    ax3.set_title('Ухудшение относительно сценария без шума\n(положительные значения = ухудшение)',
                  fontsize=14, fontweight='bold')
    ax3.set_xticks(x_degradation)
    ax3.set_xticklabels(noise_scenarios, rotation=90, fontsize=11)
    ax3.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    ax3.grid(True, alpha=0.3)
    ax3.axhline(y=0, color='black', linestyle='-', alpha=0.5, linewidth=0.8)

    # График 4. Устойчивость к шуму
    stability_scores = {}

    for algo_name in algorithm_names:
        scores = []
        base_perf = all_scenario_results[base_scenario][algo_name]['best_distance']

        for scenario_name in scenario_names:
            if scenario_name == base_scenario:
                continue

            scenario_perf = all_scenario_results[scenario_name][algo_name]['best_distance']
            degradation = ((scenario_perf - base_perf) / base_perf) * 100
            stability_score = 100 - abs(degradation)
            scores.append(stability_score)

        stability_scores[algo_name] = np.mean(scores) if scores else 100

    # Сортируем методы по устойчивости
    sorted_algorithms = sorted(stability_scores.items(), key=lambda x: x[1], reverse=True)
    algorithms_sorted = [x[0].replace(' (базовый)', ' (б.)').replace(' (улучшенный)', ' (у.)')
                        for x in sorted_algorithms]
    scores_sorted = [x[1] for x in sorted_algorithms]

    # Цвета для столбцов
    bar_colors = []
    for algo in algorithms_sorted:
        if 'у.' in algo:
            bar_colors.append('green')
        elif 'б.' in algo:
            bar_colors.append('blue')
        else:
            bar_colors.append('gray')

    bars = ax4.barh(algorithms_sorted, scores_sorted, color=bar_colors, alpha=0.7)
    ax4.set_xlabel('Оценка устойчивости к шуму', fontsize=12)
    ax4.set_ylabel('Методы RL', fontsize=12)
    ax4.set_title('Рейтинг методов по устойчивости к шуму\n(выше = лучше)',
                  fontsize=14, fontweight='bold')
    ax4.grid(True, alpha=0.3, axis='x')
    ax4.set_xlim([0, 105])

    # Добавляем значения на столбцы
    for bar, score in zip(bars, scores_sorted):
        width = bar.get_width()
        ax4.text(width + 1, bar.get_y() + bar.get_height()/2,
                f'{score:.1f}', ha='left', va='center', fontsize=9)

    plt.tight_layout()
    plt.savefig('rl_noise_scenarios_comprehensive_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()

    print("Визуализация сохранена: 'rl_noise_scenarios_comprehensive_analysis.png'")

    return stability_scores

In [ ]:
def plot_learning_curves(all_scenario_results, max_methods_per_plot=6):
    """
    Визуализация кривых обучения для всех сценариев и методов

    Аргументы:
    - all_scenario_results: результаты всех сценариев
    - max_methods_per_plot: максимальное количество методов на одном графике
    """

    scenario_names = list(all_scenario_results.keys())

    print("\n" + "="*80)
    print("Визуализация кривых обучения")
    print("="*80)

    # Создаем графики кривых обучения для каждого сценария
    for scenario_name in scenario_names:
        results = all_scenario_results[scenario_name]
        method_names = list(results.keys())

        # Разбиваем методы на группы для лучшей читаемости
        num_groups = (len(method_names) + max_methods_per_plot - 1) // max_methods_per_plot

        for group_idx in range(num_groups):
            start_idx = group_idx * max_methods_per_plot
            end_idx = min(start_idx + max_methods_per_plot, len(method_names))
            group_methods = method_names[start_idx:end_idx]

            fig, axes = plt.subplots(2, 3, figsize=(18, 10))
            axes = axes.flatten()

            for idx, method_name in enumerate(group_methods):
                if idx >= len(axes):
                    break

                ax = axes[idx]
                result = results[method_name]
                episode_distances = result['episode_distances']

                if episode_distances:
                    # Сглаживаем кривую обучения
                    smoothed = pd.Series(episode_distances).rolling(50, min_periods=1).mean()

                    # Основная кривая
                    ax.plot(smoothed, color='blue', alpha=0.7, linewidth=2, label='Сглаженная')

                    # Лучший результат
                    best_dist = result['best_distance']
                    ax.axhline(y=best_dist, color='red', linestyle='--', alpha=0.7,
                              label=f'Лучшее: {best_dist:.1f} км')

                    # Средний результат
                    avg_last_100 = result['avg_last_100']
                    ax.axhline(y=avg_last_100, color='green', linestyle=':', alpha=0.7,
                              label=f'Среднее: {avg_last_100:.1f} км')

                    # Эпизод сходимости
                    convergence_episode = np.argmin(episode_distances)
                    ax.axvline(x=convergence_episode, color='orange', linestyle='-.', alpha=0.5,
                              label=f'Сходимость: {convergence_episode}')

                    ax.set_xlabel('Эпизод', fontsize=10)
                    ax.set_ylabel('Расстояние (км)', fontsize=10)

                    # Сокращаем название метода для заголовка
                    short_name = method_name.replace(' (базовый)', '\n(б.)').replace(' (улучшенный)', '\n(у.)')
                    ax.set_title(short_name, fontsize=11, fontweight='bold')

                    ax.legend(fontsize=8)
                    ax.grid(True, alpha=0.3)

                    # Добавляем аннотацию с информацией
                    info_text = f"Сценарий: {scenario_name}\n"
                    info_text += f"Лучшее: {best_dist:.1f} км\n"
                    info_text += f"Среднее: {avg_last_100:.1f} км\n"
                    info_text += f"Сходимость: {convergence_episode} эп."

                    ax.text(0.02, 0.98, info_text, transform=ax.transAxes,
                           fontsize=8, verticalalignment='top',
                           bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

            # Скрываем пустые оси
            for idx in range(len(group_methods), len(axes)):
                axes[idx].axis('off')

            group_title = f"Кривые обучения: {scenario_name}"
            if num_groups > 1:
                group_title += f" (Группа {group_idx + 1}/{num_groups})"

            plt.suptitle(group_title, fontsize=14, fontweight='bold')
            plt.tight_layout()

            filename = f'learning_curves_{scenario_name.replace(" ", "_").replace("+", "_")}'
            if num_groups > 1:
                filename += f'_group_{group_idx + 1}'
            filename += '.png'

            plt.savefig(filename, dpi=300, bbox_inches='tight')
            plt.show()
            print(f"  Сохранено: {filename}")


def analyze_noise_scenario_results(all_scenario_results, training_times):
    """
    Основная функция анализа результатов по сценариям с шумом

    Выполняет:
    1. Создание сводных таблиц
    2. Построение графиков
    3. Визуализацию кривых обучения
    """

    print("\n" + "="*80)
    print("Анализ результатов")
    print("="*80)

    # 1. Сводные таблицы
    print("\n Шаг 1: Создание сводных таблиц")
    summary_df, pivot_best, pivot_avg, pivot_time = create_summary_tables(all_scenario_results)

    # 2. Графики
    print("\n Шаг 2: Построение графиков")
    stability_scores = plot_results_by_scenario(all_scenario_results)

    # 3. Кривые обучения
    print("\n Шаг 3: Визуализация кривых обучения")
    plot_learning_curves(all_scenario_results)

In [ ]:
# Запускаем обучение и тестирование
all_scenario_results, scenario_matrices, training_times = train_and_test_on_noise_scenarios(num_episodes=1500)

In [ ]:
analysis_results = analyze_noise_scenario_results(all_scenario_results, training_times)
analysis_results

## RL на большем количестве складов

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
from collections import defaultdict
import time

In [ ]:
def train_rl_all_warehouses_noise_scenarios():
    """
    Обучение RL методов на всех складах со всеми сценариями шума
    """
    # Загрузка всех складов
    print("\nЗагружаем все склады")
    with open('all_warehouses_coordinates.json', 'r', encoding='utf-8') as f:
        all_warehouses_json = json.load(f)

    all_warehouses_data = []
    for warehouse in all_warehouses_json:
        all_warehouses_data.append({
            "name": warehouse['name'],
            "latitude": warehouse['latitude'],
            "longitude": warehouse['longitude'],
            "type": "РЦ/РФЦ" if "РЦ" in warehouse['name'] or "РФЦ" in warehouse['name'] else "СЦ"
        })

    all_warehouses_df = pd.DataFrame(all_warehouses_data)

    # Привязка складов к узлам графа
    all_warehouse_nodes = []
    for idx, warehouse in all_warehouses_df.iterrows():
        point = (warehouse['latitude'], warehouse['longitude'])
        nearest_node = ox.distance.nearest_nodes(graph, point[1], point[0])
        all_warehouse_nodes.append(nearest_node)

    all_warehouses_df['node_id'] = all_warehouse_nodes
    print(f"Загружено складов: {len(all_warehouses_df)}")
    print(f"Распределение по типам: {dict(all_warehouses_df['type'].value_counts())}")

    # Создание матриц расстояний для всех сценариев
    print("\nСоздание матриц расстояний для всех сценариев")
    scenario_matrices_all = {}

    scenarios_all = {
        "Без шума": graph,
        "Пробки": graph_traffic_jam,
        "Блокировки+Пробки": graph_combined,
        "Умеренные условия": graph_moderate
    }

    for scenario_name, scenario_graph in scenarios_all.items():
        print(f"  Вычисление: {scenario_name}...")

        n_warehouses = len(all_warehouse_nodes)
        distance_matrix = np.zeros((n_warehouses, n_warehouses))

        for i in range(n_warehouses):
            for j in range(i+1, n_warehouses):
                try:
                    distance = nx.shortest_path_length(scenario_graph, all_warehouse_nodes[i], all_warehouse_nodes[j], weight='length')
                    distance_matrix[i][j] = distance
                    distance_matrix[j][i] = distance
                except nx.NetworkXNoPath:
                    distance_matrix[i][j] = 1e9
                    distance_matrix[j][i] = 1e9

        scenario_matrices_all[scenario_name] = distance_matrix.astype(np.float32)
        print(f"    Завершено: {scenario_name}")

    # Выбранные RL методы для тестирования
    algorithms = {
        'SAC (улучшенный)': lambda n: EnhancedSACAgent(n),
        'DQN (улучшенный)': lambda n: ImprovedDQNAgent(n, use_per=True, use_noisy=False),
        'Double DQN (улучшенный)': lambda n: ImprovedDoubleDQNAgent(n, use_per=True, use_noisy=False),
        'PPO (базовый)': lambda n: PPOAgent(n),
        'A2C (улучшенный)': lambda n: EnhancedA2CAgent(n),
        'Policy Gradient (базовый)': lambda n: PolicyGradientAgent(n)
    }

    n_warehouses_all = len(all_warehouses_df)
    num_episodes = 1500
    comprehensive_results = {}

    print(f"\nНачинаем обучение на {n_warehouses_all} складах")
    print(f"Тестируемые методы: {', '.join(algorithms.keys())}")
    print(f"Количество эпизодов: {num_episodes}")

    # Обучение каждого алгоритма на каждом сценарии
    for scenario_name, distance_matrix in scenario_matrices_all.items():
        print(f"\nСценарий: {scenario_name}")
        print("-" * 50)

        scenario_results = {}

        for agent_name, agent_constructor in algorithms.items():
            print(f"  Обучение: {agent_name}")
            start_time = time.time()

            # Создаем агента и среду
            agent = agent_constructor(n_warehouses_all)
            env = TSPEnvironment(distance_matrix, all_warehouses_df)

            # Обучение на 1500 эпизодов
            episode_distances = []
            episode_rewards = []
            best_route = None
            best_distance = float('inf')

            for episode in range(num_episodes):
                state = env.reset()
                episode_reward = 0.0
                done = False
                step_count = 0
                max_steps = n_warehouses_all * 2

                step_rewards = []

                while not done and step_count < max_steps:
                    # Выбор действия (особенность SAC)
                    if 'SAC' in agent_name:
                        action_result = agent.select_action(state)
                        if isinstance(action_result, tuple):
                            action = action_result[0]  # SAC возвращает (action, log_prob)
                        else:
                            action = action_result
                    else:
                        action = agent.select_action(state)

                    # Шаг в среде
                    next_state, reward, done, _ = env.step(action)
                    episode_reward += reward
                    step_rewards.append(reward)

                    # Сохраняем опыт для off-policy методов
                    if hasattr(agent, 'store_experience'):
                        agent.store_experience(state, action, reward, next_state, done)

                    # Сохраняем награды для on-policy методов
                    if hasattr(agent, 'rewards'):
                        agent.rewards.append(reward)

                    state = next_state
                    step_count += 1

                # Обновление политики
                # Для PPO передаем финальную награду
                if 'PPO' in agent_name:
                    agent.update(episode_reward)
                # Для остальных on-policy методов обновляем каждые 10 эпизодов
                elif hasattr(agent, 'update') and not hasattr(agent, 'store_experience'):
                    if episode % 10 == 0 or done:
                        agent.update()
                # Для off-policy методов обновляем всегда
                elif hasattr(agent, 'update'):
                    agent.update()

                # Сохраняем результаты
                route_info = env.get_route_info()
                current_distance = route_info['distance_km']

                episode_distances.append(current_distance)
                episode_rewards.append(episode_reward)

                if current_distance < best_distance and len(env.visited) == n_warehouses_all:
                    best_distance = current_distance
                    best_route = route_info['route']

                # Прогресс каждые 150 эпизодов
                if episode % 150 == 0:
                    visited_count = len(env.visited)
                    epsilon = getattr(agent, 'epsilon', 0)
                    print(f"    Эпизод {episode:4d}: {current_distance:7.2f} км, "
                          f"ε={epsilon:.3f}, Посещено: {visited_count}/{n_warehouses_all}")

            end_time = time.time()
            training_time = end_time - start_time

            # Анализ результатов
            avg_distance = np.mean(episode_distances)
            std_distance = np.std(episode_distances)
            avg_reward = np.mean(episode_rewards)

            scenario_results[agent_name] = {
                'best_distance': best_distance,
                'avg_distance': avg_distance,
                'std_distance': std_distance,
                'avg_reward': avg_reward,
                'training_time': training_time,
                'episode_distances': episode_distances,
                'episode_rewards': episode_rewards,
                'best_route': best_route
            }

            print(f"    Завершено: {agent_name}")
            print(f"      Лучший результат: {best_distance:.2f} км")
            print(f"      Средний результат: {avg_distance:.2f} км")
            print(f"      Время обучения: {training_time:.1f} секунд")

        comprehensive_results[scenario_name] = scenario_results

    return comprehensive_results, all_warehouses_df

In [ ]:
def create_rl_summary_tables(all_scenario_results):
    """
    Создание сводных таблиц результатов RL методов
    """
    scenario_names = list(all_scenario_results.keys())
    algorithm_names = list(all_scenario_results[scenario_names[0]].keys())

    # Создание полной таблицы
    summary_data = []
    for scenario_name in scenario_names:
        for algorithm_name in algorithm_names:
            result = all_scenario_results[scenario_name][algorithm_name]
            summary_data.append({
                'Сценарий': scenario_name,
                'Метод': algorithm_name,
                'Лучший результат (км)': f"{result['best_distance']:.2f}",
                'Среднее расстояние (км)': f"{result['avg_distance']:.2f}",
                'Станд. отклонение': f"{result['std_distance']:.2f}",
                'Средняя награда': f"{result['avg_reward']:.2f}",
                'Время обучения (сек)': f"{result['training_time']:.2f}",
                'Время на эпизод (сек)': f"{result['training_time']/1500:.3f}"
            })

    summary_df = pd.DataFrame(summary_data)

    # Создание pivot таблиц
    print("\nЛучшие результаты по сценариям:")
    pivot_best = summary_df.pivot(index='Метод', columns='Сценарий', values='Лучший результат (км)')
    print(pivot_best)

    print("\nСредние расстояния по сценариям:")
    pivot_avg = summary_df.pivot(index='Метод', columns='Сценарий', values='Среднее расстояние (км)')
    print(pivot_avg)

    print("\nВремя обучения по сценариям:")
    pivot_time = summary_df.pivot(index='Метод', columns='Сценарий', values='Время обучения (сек)')
    print(pivot_time)

    return summary_df, pivot_best, pivot_avg, pivot_time

In [ ]:
def plot_rl_comprehensive_results(all_scenario_results):
    """
    Визуализация комплексных результатов RL методов
    """
    scenario_names = list(all_scenario_results.keys())
    algorithm_names = list(all_scenario_results[scenario_names[0]].keys())

    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(18, 14))

    # График 1: Лучшие результаты по сценариям
    best_data = {algo: [] for algo in algorithm_names}
    for scenario_name in scenario_names:
        for algo_name in algorithm_names:
            result = all_scenario_results[scenario_name][algo_name]
            best_data[algo_name].append(result['best_distance'])

    x = np.arange(len(scenario_names))
    width = 0.12

    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']

    for i, algo_name in enumerate(algorithm_names):
        ax1.bar(x + i * width, best_data[algo_name], width, label=algo_name, color=colors[i], alpha=0.8)

    ax1.set_xlabel('Сценарии')
    ax1.set_ylabel('Лучшее расстояние (км)')
    ax1.set_title('Лучшие результаты RL методов по сценариям\n(Все склады, 1500 эпизодов)')
    ax1.set_xticks(x + width * (len(algorithm_names) - 1) / 2)
    ax1.set_xticklabels(scenario_names, rotation=45)
    ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    ax1.grid(True, alpha=0.3)

    # Добавление значений на столбцы (только для лучших в каждом сценарии)
    for j, scenario in enumerate(scenario_names):
        min_value = min([all_scenario_results[scenario][algo]['best_distance'] for algo in algorithm_names])
        ax1.text(j + width * (len(algorithm_names) - 1) / 2, min_value - max(min_value*0.02, 1),
                f'{min_value:.1f}', ha='center', va='top', fontsize=8, fontweight='bold', color='red')

    # График 2: Средние результаты по сценариям
    avg_data = {algo: [] for algo in algorithm_names}
    for scenario_name in scenario_names:
        for algo_name in algorithm_names:
            result = all_scenario_results[scenario_name][algo_name]
            avg_data[algo_name].append(result['avg_distance'])

    for i, algo_name in enumerate(algorithm_names):
        ax2.bar(x + i * width, avg_data[algo_name], width, label=algo_name, color=colors[i], alpha=0.8)

    ax2.set_xlabel('Сценарии')
    ax2.set_ylabel('Среднее расстояние (км)')
    ax2.set_title('Средние результаты RL методов по сценариям\n(Все склады, 1500 эпизодов)')
    ax2.set_xticks(x + width * (len(algorithm_names) - 1) / 2)
    ax2.set_xticklabels(scenario_names, rotation=45)
    ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    ax2.grid(True, alpha=0.3)

    # График 3: Устойчивость к шуму (ухудшение производительности)
    base_scenario = "Без шума"
    degradation_data = {algo: [] for algo in algorithm_names}

    for algo_name in algorithm_names:
        base_avg = all_scenario_results[base_scenario][algo_name]['avg_distance']
        for scenario_name in scenario_names:
            if scenario_name != base_scenario:
                current_avg = all_scenario_results[scenario_name][algo_name]['avg_distance']
                degradation_pct = ((current_avg - base_avg) / base_avg) * 100
                degradation_data[algo_name].append(degradation_pct)

    x_degrade = np.arange(len(scenario_names) - 1)
    for i, algo_name in enumerate(algorithm_names):
        ax3.bar(x_degrade + i * width, degradation_data[algo_name], width,
                label=algo_name, color=colors[i], alpha=0.8)

    ax3.set_xlabel('Сценарии с шумом')
    ax3.set_ylabel('Ухудшение производительности (%)')
    ax3.set_title('Устойчивость методов к шуму\n(Относительно базового сценария)')
    ax3.set_xticks(x_degrade + width * (len(algorithm_names) - 1) / 2)
    ax3.set_xticklabels([name for name in scenario_names if name != base_scenario], rotation=45)
    ax3.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    ax3.grid(True, alpha=0.3)
    ax3.axhline(y=0, color='red', linestyle='--', alpha=0.5)

    # График 4: Рейтинг методов по комплексной оценке
    algorithm_scores = {}

    for algo_name in algorithm_names:
        # Оценка качества
        quality_score = 0
        for scenario_name in scenario_names:
            avg_dist = all_scenario_results[scenario_name][algo_name]['avg_distance']
            quality_score += 1000 / avg_dist

        # Оценка устойчивости
        stability_score = 0
        base_avg = all_scenario_results[base_scenario][algo_name]['avg_distance']
        for scenario_name in scenario_names:
            if scenario_name != base_scenario:
                current_avg = all_scenario_results[scenario_name][algo_name]['avg_distance']
                degradation = abs((current_avg - base_avg) / base_avg)
                stability_score += (1 - degradation) * 100

        total_score = 0.5 * (quality_score / len(scenario_names)) + 0.5 * (stability_score / (len(scenario_names) - 1))
        algorithm_scores[algo_name] = total_score

    sorted_algorithms = sorted(algorithm_scores.items(), key=lambda x: x[1], reverse=True)
    algo_names_sorted = [x[0] for x in sorted_algorithms]
    scores_sorted = [x[1] for x in sorted_algorithms]

    # Разные цвета для базовых и улучшенных методов
    bar_colors = []
    for algo in algo_names_sorted:
        if 'улучшенный' in algo:
            bar_colors.append('#2E8B57')  # зеленый
        elif 'базовый' in algo:
            bar_colors.append('#4169E1')  # синий
        else:
            bar_colors.append('#DC143C')  # красный

    bars = ax4.barh(algo_names_sorted, scores_sorted, color=bar_colors, alpha=0.7)
    ax4.set_xlabel('Комплексная оценка')
    ax4.set_ylabel('Методы')
    ax4.set_title('Рейтинг RL методов\n(Качество + Устойчивость к шуму)')
    ax4.grid(True, alpha=0.3, axis='x')
    ax4.set_xlim([0, 105])

    for bar, score in zip(bars, scores_sorted):
        width = bar.get_width()
        ax4.text(width + 1, bar.get_y() + bar.get_height()/2,
                f'{score:.1f}', ha='left', va='center', fontsize=9)

    plt.tight_layout()
    plt.savefig('rl_comprehensive_results_all_warehouses_1500_episodes.png', dpi=300, bbox_inches='tight')
    plt.show()

    return sorted_algorithms

In [ ]:
def plot_rl_learning_curves(all_scenario_results, max_methods_per_plot=4):
    """
    Визуализация кривых обучения RL методов
    """
    print("\nВизуализация кривых обучения")
    print("=" * 60)

    scenario_names = list(all_scenario_results.keys())

    for scenario_name in scenario_names:
        results = all_scenario_results[scenario_name]
        method_names = list(results.keys())

        num_groups = (len(method_names) + max_methods_per_plot - 1) // max_methods_per_plot

        for group_idx in range(num_groups):
            start_idx = group_idx * max_methods_per_plot
            end_idx = min(start_idx + max_methods_per_plot, len(method_names))
            group_methods = method_names[start_idx:end_idx]

            fig, axes = plt.subplots(2, 2, figsize=(16, 10))
            axes = axes.flatten()

            for idx, method_name in enumerate(group_methods):
                if idx >= len(axes):
                    break

                ax = axes[idx]
                result = results[method_name]
                episode_distances = result['episode_distances']

                if episode_distances:
                    # Сглаживание кривой обучения
                    smoothed = pd.Series(episode_distances).rolling(50, min_periods=1).mean()

                    # Основная кривая
                    ax.plot(smoothed, color='blue', alpha=0.7, linewidth=2, label='Сглаженная')

                    # Лучший результат
                    best_dist = result['best_distance']
                    ax.axhline(y=best_dist, color='red', linestyle='--', alpha=0.7,
                              label=f'Лучшее: {best_dist:.1f} км')

                    # Средний результат
                    avg_distance = result['avg_distance']
                    ax.axhline(y=avg_distance, color='green', linestyle=':', alpha=0.7,
                              label=f'Среднее: {avg_distance:.1f} км')

                    ax.set_xlabel('Эпизод')
                    ax.set_ylabel('Расстояние (км)')
                    ax.set_title(method_name, fontsize=11, fontweight='bold')
                    ax.legend(fontsize=8)
                    ax.grid(True, alpha=0.3)

            for idx in range(len(group_methods), len(axes)):
                axes[idx].axis('off')

            group_title = f"Кривые обучения: {scenario_name}"
            if num_groups > 1:
                group_title += f" (Группа {group_idx + 1}/{num_groups})"

            plt.suptitle(group_title, fontsize=14, fontweight='bold')
            plt.tight_layout()

            filename = f'learning_curves_{scenario_name.replace(" ", "_").replace("+", "_")}'
            if num_groups > 1:
                filename += f'_group_{group_idx + 1}'
            filename += '.png'

            plt.savefig(filename, dpi=300, bbox_inches='tight')
            plt.show()
            print(f"  Сохранено: {filename}")

In [ ]:
def analyze_rl_results(all_scenario_results):
    """
    Детальный анализ результатов RL методов
    """
    print("\nДетальный анализ результатов")
    print("=" * 80)

    scenario_names = list(all_scenario_results.keys())
    algorithm_names = list(all_scenario_results[scenario_names[0]].keys())

    print("\nЛучшие методы по сценариям:")
    print("-" * 60)

    best_methods_by_scenario = {}
    for scenario_name in scenario_names:
        best_method = min(algorithm_names,
                         key=lambda algo: all_scenario_results[scenario_name][algo]['best_distance'])
        best_distance = all_scenario_results[scenario_name][best_method]['best_distance']
        best_methods_by_scenario[scenario_name] = (best_method, best_distance)
        print(f"{scenario_name:25}: {best_method:25} ({best_distance:.2f} км)")

    print("\nНаиболее устойчивые методы:")
    print("-" * 60)

    base_scenario = "Без шума"
    stability_scores = {}

    for algo_name in algorithm_names:
        base_avg = all_scenario_results[base_scenario][algo_name]['avg_distance']
        degradation_scores = []

        for scenario_name in scenario_names:
            if scenario_name != base_scenario:
                current_avg = all_scenario_results[scenario_name][algo_name]['avg_distance']
                degradation = abs((current_avg - base_avg) / base_avg)
                degradation_scores.append(degradation)

        avg_degradation = np.mean(degradation_scores) if degradation_scores else 0
        stability_scores[algo_name] = 100 * (1 - avg_degradation)

    sorted_stability = sorted(stability_scores.items(), key=lambda x: x[1], reverse=True)
    for algo_name, stability in sorted_stability:
        print(f"{algo_name:30}: {stability:.1f}%")

    return best_methods_by_scenario, sorted_stability

In [ ]:
# Запуск обучения
all_scenario_results, warehouses_df = train_rl_all_warehouses_noise_scenarios()

In [ ]:
# Создание сводных таблиц
summary_df, pivot_best, pivot_avg, pivot_time = create_rl_summary_tables(all_scenario_results)

In [ ]:
# Визуализация результатов
sorted_algorithms = plot_rl_comprehensive_results(all_scenario_results)

In [ ]:
# Анализ результатов
best_methods_by_scenario, sorted_stability = analyze_rl_results(all_scenario_results)

In [ ]:
# Визуализация кривых обучения (опционально)
plot_rl_learning_curves(all_scenario_results)

In [ ]:
def save_rl_routes_to_json(all_scenario_results, warehouses_df, filename='rl_routes.json'):
    """
    Сохраняет найденные RL маршруты в JSON файл

    Args:
        all_scenario_results: результаты RL методов
        warehouses_df: DataFrame со складами
        filename: имя файла для сохранения
    """

    # Создаем структуру данных
    routes_dict = {
        'metadata': {
            'total_scenarios': len(all_scenario_results),
            'total_routes': sum(len(scenario) for scenario in all_scenario_results.values()),
            'saved_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
        },
        'warehouses': [],
        'scenarios': {}
    }

    # Сохраняем информацию о складах
    for idx, warehouse in warehouses_df.iterrows():
        routes_dict['warehouses'].append({
            'id': idx,
            'name': warehouse['name'],
            'latitude': float(warehouse['latitude']),
            'longitude': float(warehouse['longitude']),
            'type': warehouse.get('type', 'unknown')
        })

    # Сохраняем маршруты по сценариям
    for scenario_name, scenario_results in all_scenario_results.items():
        routes_dict['scenarios'][scenario_name] = {}

        for algorithm_name, result in scenario_results.items():
            route_indices = result['best_route']

            # Детальная информация о маршруте
            route_details = []
            for i, idx in enumerate(route_indices[:-1]):  # Исключаем последний дубликат
                if idx < len(warehouses_df):
                    warehouse = warehouses_df.iloc[idx]
                    route_details.append({
                        'order': i + 1,
                        'warehouse_id': idx,
                        'warehouse_name': warehouse['name'],
                        'latitude': float(warehouse['latitude']),
                        'longitude': float(warehouse['longitude'])
                    })

            routes_dict['scenarios'][scenario_name][algorithm_name] = {
                'distance_km': float(result['best_distance']),
                'avg_distance_km': float(result.get('avg_distance', 0)),
                'training_time_sec': float(result.get('training_time', 0)),
                'route_indices': [int(idx) for idx in route_indices],
                'route_details': route_details
            }

    # Сохраняем в JSON файл
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(routes_dict, f, ensure_ascii=False, indent=2)

    print(f"Маршруты сохранены в JSON файл: {filename}")
    return routes_dict

# Сохраняем в JSON
rl_routes_json = save_rl_routes_to_json(all_scenario_results, warehouses_df, 'rl_routes_detailed.json')

In [ ]:
# Создаем словарь с графами для каждого сценария
scenarios_graphs = {
    "Без шума": graph,
    "Пробки": graph_traffic_jam,
    "Блокировки+Пробки": graph_combined,
    "Умеренные условия": graph_moderate
}

In [ ]:
def get_full_route_path_on_roads(graph, route_indices, warehouse_nodes):
    """
    Получает полный путь по дорожной сети для заданного маршрута

    Args:
        graph: дорожный граф
        route_indices: список индексов складов
        warehouse_nodes: список узлов графа для складов

    Returns:
        list: список координат (lat, lon) пути по дорогам
    """
    full_path = []

    for i in range(len(route_indices) - 1):
        from_idx = route_indices[i]
        to_idx = route_indices[i + 1]

        from_node = warehouse_nodes[from_idx]
        to_node = warehouse_nodes[to_idx]

        try:
            # Находим кратчайший путь по дорожной сети
            path_nodes = nx.shortest_path(graph, from_node, to_node, weight='length')

            # Получаем координаты всех узлов пути
            for node in path_nodes:
                node_data = graph.nodes[node]
                full_path.append((node_data['y'], node_data['x']))  # (lat, lon)
        except (nx.NetworkXNoPath, nx.NodeNotFound):
            # Если пути нет, используем прямую линию между складами
            from_data = graph.nodes[from_node]
            to_data = graph.nodes[to_node]
            full_path.append((from_data['y'], from_data['x']))
            full_path.append((to_data['y'], to_data['x']))

    return full_path


In [ ]:
print("\n1. Визуализация лучших RL маршрутов по сценариям (по дорогам)...")

# Создаем сетку 2x2 для лучших маршрутов
fig_best, axes_best = plt.subplots(2, 2, figsize=(16, 12))
axes_best = axes_best.flatten()

scenario_names = list(all_scenario_results.keys())

# Получаем узлы графа для всех складов
# (должны быть определены в коде RL)
if 'all_warehouse_nodes' in locals() or 'all_warehouse_nodes' in globals():
    warehouse_nodes = all_warehouse_nodes
else:
    # Если не найдено, создаем заново
    print("Создаем привязку складов к узлам графа...")
    warehouse_nodes = []
    for idx, warehouse in warehouses_df.iterrows():
        point = (warehouse['latitude'], warehouse['longitude'])
        nearest_node = ox.distance.nearest_nodes(graph, point[1], point[0])
        warehouse_nodes.append(nearest_node)

for idx, scenario_name in enumerate(scenario_names):
    ax = axes_best[idx]
    scenario_graph = scenarios_graphs[scenario_name]

    # Находим лучший метод для этого сценария
    scenario_results = all_scenario_results[scenario_name]
    best_method = min(scenario_results.keys(),
                     key=lambda x: scenario_results[x]['best_distance'])

    best_result = scenario_results[best_method]
    best_route = best_result['best_route']
    best_distance = best_result['best_distance']

    # Получаем полный путь по дорогам
    road_path = get_full_route_path_on_roads(scenario_graph, best_route, warehouse_nodes)

    # Преобразуем координаты
    if road_path:
        road_lats = [coord[0] for coord in road_path]
        road_lons = [coord[1] for coord in road_path]

    # Рисуем дорожную сеть
    ox.plot_graph(
        scenario_graph,
        ax=ax,
        node_size=0,
        edge_linewidth=0.3,
        edge_color='lightgray',
        bgcolor='white',
        show=False,
        close=False
    )

    # Рисуем склады
    ax.scatter(warehouses_df['longitude'], warehouses_df['latitude'],
              c='lightgray', s=20, alpha=0.3, edgecolors='none', zorder=2)

    # Рисуем маршрут по дорогам
    if road_path and len(road_lats) > 1 and len(road_lons) > 1:
        ax.plot(road_lons, road_lats,
               color='blue', linewidth=2.5, alpha=0.8, zorder=3,
               label='Маршрут по дорогам')

        # Добавляем стрелки направления
        for i in range(0, len(road_lats)-1, max(1, len(road_lats)//20)):
            ax.annotate('', xy=(road_lons[i+1], road_lats[i+1]),
                       xytext=(road_lons[i], road_lats[i]),
                       arrowprops=dict(arrowstyle='->', color='red',
                                     lw=2, alpha=0.6))

    # Рисуем склады из маршрута
    for i, idx_point in enumerate(best_route[:-1]):  # Исключаем последний (дубликат начала)
        if idx_point < len(warehouses_df):
            warehouse = warehouses_df.iloc[idx_point]
            color = 'red' if i == 0 else 'green'
            size = 100 if i == 0 else 60
            marker = '*' if i == 0 else 'o'

            ax.scatter(warehouse['longitude'], warehouse['latitude'],
                      c=color, s=size, marker=marker, edgecolors='black',
                      linewidth=1.5, zorder=5,
                      label='Начальная точка' if i == 0 else None)

    # Подписываем начальный склад
    if best_route and best_route[0] < len(warehouses_df):
        start_warehouse = warehouses_df.iloc[best_route[0]]
        ax.annotate(start_warehouse['name'][:15] + '...'
                   if len(start_warehouse['name']) > 15
                   else start_warehouse['name'],
                   (start_warehouse['longitude'], start_warehouse['latitude']),
                   xytext=(5, 5), textcoords='offset points',
                   fontsize=8, fontweight='bold',
                   bbox=dict(boxstyle="round,pad=0.2", facecolor="white", alpha=0.8))

    ax.set_title(f'{scenario_name}\n{best_method}\n{best_distance:.2f} км',
                fontsize=12, fontweight='bold')
    ax.set_xlabel('Долгота')
    ax.set_ylabel('Широта')
    ax.grid(True, alpha=0.2)

    # Добавляем легенду только для первого графика
    if idx == 0:
        from matplotlib.lines import Line2D
        legend_elements = [
            Line2D([0], [0], color='blue', lw=3, label='Маршрут по дорогам'),
            Line2D([0], [0], color='red', lw=0, marker='*', markersize=10,
                  label='Начальная точка', markeredgecolor='black'),
            Line2D([0], [0], color='green', lw=0, marker='o', markersize=8,
                  label='Склады в маршруте', markeredgecolor='black'),
            Line2D([0], [0], color='lightgray', lw=0, marker='o', markersize=6,
                  label='Все склады', alpha=0.3, markeredgecolor='none')
        ]
        ax.legend(handles=legend_elements, loc='upper right', fontsize=8)

plt.suptitle('Лучшие RL маршруты по сценариям (по дорогам)',
            fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('rl_best_routes_all_scenarios_roads.png', dpi=300, bbox_inches='tight')
plt.show()